In [7]:
import pandas as pd
import re
from datetime import datetime
import numpy as np
from tqdm.auto import tqdm
import os

# Improve pandas display for debugging
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Function to create log directory if it doesn't exist
def ensure_log_dir():
    log_dir = 'logs'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir

# Set up logging
log_dir = ensure_log_dir()
log_file = os.path.join(log_dir, f'data_cleaning_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')

def log_message(message):
    """Log a message to both console and file"""
    print(message)
    with open(log_file, 'a') as f:
        f.write(message + '\n')

log_message(f"Starting data cleaning process at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Load the data - use read_excel for .xlsx files
file_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Merged_ES_Cleaned.xlsx'
try:
    # Try reading the Excel file
    log_message(f"Loading Excel file: {file_path}")
    df = pd.read_excel(file_path)
    log_message(f"Successfully loaded Excel file with {len(df)} rows and {len(df.columns)} columns.")
except Exception as e:
    log_message(f"Error loading Excel file: {e}")
    # Try with different encoding as fallback
    try:
        # If it's actually a CSV disguised as .xlsx
        log_message("Attempting to load as CSV with latin1 encoding...")
        df = pd.read_csv(file_path, encoding='latin1')
        log_message("Loaded as CSV with latin1 encoding.")
    except Exception as e2:
        log_message(f"Second attempt failed: {e2}")
        raise Exception("Could not load the file. Please check the file format and path.")

# Print basic info to verify data was loaded
log_message(f"Data shape: {df.shape}")
log_message(f"First few column names: {df.columns[:5].tolist()}")

# 1. Extract course_date and course_year from course_name
log_message("\nSTEP 1: Extracting course date and year from course name")

def extract_date_from_course_name(course_name):
    if pd.isna(course_name):
        return None
    
    # Match date patterns like 1/12/22, 4/6/21, etc.
    date_pattern = r'(\d{1,2}/\d{1,2}/\d{2})'
    match = re.search(date_pattern, str(course_name))
    
    if match:
        return match.group(1)
    return None

# Apply the function to create course_date with a progress bar
log_message("Extracting course_date from course_name...")
tqdm.pandas(desc="Extracting dates")
df['course_date'] = df['course_name'].progress_apply(extract_date_from_course_name)

# Count successful extractions
date_count = df['course_date'].notna().sum()
log_message(f"Successfully extracted {date_count} dates from {len(df)} course names ({date_count/len(df)*100:.1f}%)")

# Convert course_date to datetime and extract year
def convert_date(date_str):
    if pd.isna(date_str):
        return None
    try:
        date_obj = datetime.strptime(str(date_str), '%m/%d/%y')
        return date_obj
    except Exception:
        # Try other formats if the first one fails
        try:
            date_obj = datetime.strptime(str(date_str), '%m/%d/%Y')
            return date_obj
        except:
            pass
        return None

# Apply with error handling and progress bar
log_message("Converting extracted dates to datetime objects...")
tqdm.pandas(desc="Converting dates")
df['course_date_dt'] = df['course_date'].progress_apply(convert_date)
df['course_year'] = df['course_date_dt'].apply(lambda x: x.year if pd.notnull(x) else None)

# Count successful conversions
year_count = df['course_year'].notna().sum()
log_message(f"Successfully extracted {year_count} years from {date_count} dates ({year_count/date_count*100:.1f}%)")

# 2. Create full_name column by combining first_name and last_name
log_message("\nSTEP 2: Creating full names from first and last names")

# Add safety checks to handle missing values
tqdm.pandas(desc="Creating full names")
df['full_name'] = df.progress_apply(
    lambda row: (str(row['first_name']) + ' ' + str(row['last_name'])) 
    if pd.notnull(row['first_name']) and pd.notnull(row['last_name']) 
    else row['first_name'] if pd.notnull(row['first_name']) 
    else row['last_name'] if pd.notnull(row['last_name']) 
    else None, 
    axis=1
)

# Count successful full name creations
name_count = df['full_name'].notna().sum()
log_message(f"Created {name_count} full names from {len(df)} records ({name_count/len(df)*100:.1f}%)")

# 3. Standardize date_of_birth to MM-DD-YYYY format
log_message("\nSTEP 3: Standardizing dates of birth")

def standardize_dob(dob_str):
    if pd.isna(dob_str):
        return None
    
    # Handle datetime objects that include time components (from Excel)
    if isinstance(dob_str, pd.Timestamp) or isinstance(dob_str, np.datetime64):
        return dob_str.strftime('%m-%d-%Y')
    
    # Convert to string for other processing
    dob_str = str(dob_str)
    
    # Check if the string contains a timestamp part (like '1947-09-14 00:00:00')
    if ' 00:00:00' in dob_str:
        dob_str = dob_str.split(' ')[0]  # Extract just the date part
    
    # Try different date formats
    formats = [
        '%m/%d/%y',   # 4/9/88
        '%m/%d/%Y',   # 4/9/1988
        '%d/%m/%y',   # 9/4/88
        '%d/%m/%Y',   # 9/4/1988
        '%Y-%m-%d',   # 1988-04-09
        '%Y/%m/%d',   # 1988/04/09
        '%m-%d-%Y',   # 04-09-1988
        '%d-%m-%Y'    # 09-04-1988
    ]
    
    for fmt in formats:
        try:
            date_obj = datetime.strptime(dob_str, fmt)
            
            # Fix for 2-digit years: if year > current year, assume it's 19xx not 20xx
            if fmt in ['%m/%d/%y', '%d/%m/%y'] and date_obj.year > datetime.now().year:
                date_obj = date_obj.replace(year=date_obj.year - 100)
                
            # Convert to MM-DD-YYYY format
            return date_obj.strftime('%m-%d-%Y')
        except:
            continue
    
    # Last resort: check if it's just YYYY-MM-DD format and try to extract
    yyyy_mm_dd_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2})'
    match = re.match(yyyy_mm_dd_pattern, dob_str)
    if match:
        year, month, day = match.groups()
        try:
            return f"{int(month):02d}-{int(day):02d}-{year}"
        except:
            pass
    
    return dob_str  # Return original string if unable to convert

# Apply with progress bar
tqdm.pandas(desc="Standardizing DOBs")
df['date_of_birth_standardized'] = df['date_of_birth'].progress_apply(standardize_dob)

# Count standardized dates
std_date_count = df['date_of_birth_standardized'].notna().sum()
log_message(f"Standardized {std_date_count} dates of birth from {len(df)} records ({std_date_count/len(df)*100:.1f}%)")

# Log a sample of original vs. standardized dates for verification
sample_size = min(5, len(df))
log_message("\nSample of date standardization:")
for i in range(sample_size):
    if pd.notna(df.iloc[i]['date_of_birth']) and pd.notna(df.iloc[i]['date_of_birth_standardized']):
        log_message(f"  Original: {df.iloc[i]['date_of_birth']} -> Standardized: {df.iloc[i]['date_of_birth_standardized']}")

# 4. Calculate age from date_of_birth
log_message("\nSTEP 4: Calculating ages from dates of birth")

date_format_errors = 0
age_range_errors = 0

def calculate_age(dob_str):
    global date_format_errors, age_range_errors
    if pd.isna(dob_str):
        return None
        
    # First, check if it's already a date object (from direct datetime conversion)
    if isinstance(dob_str, (datetime, pd.Timestamp, np.datetime64)):
        dob = pd.Timestamp(dob_str).to_pydatetime()
        today = datetime.now()
        age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
        
        # Sanity check
        if 0 <= age <= 120:
            return age
        else:
            age_range_errors += 1
            return None
    
    # If not a date object, process the string
    try:
        # Try various formats for standardized date string
        for fmt in ['%m-%d-%Y', '%Y-%m-%d', '%m/%d/%Y']:
            try:
                dob = datetime.strptime(str(dob_str), fmt)
                today = datetime.now()
                age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
                
                # Sanity check: ages should be reasonable (0-120)
                if 0 <= age <= 120:
                    return age
                else:
                    age_range_errors += 1
                    continue
            except:
                continue
                
        # If we get here, none of the formats worked or gave reasonable ages
        # Try direct parsing for timestamp-like strings
        if isinstance(dob_str, str) and ' 00:00:00' in dob_str:
            date_part = dob_str.split(' ')[0]
            try:
                dob = datetime.strptime(date_part, '%Y-%m-%d')
                today = datetime.now()
                age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
                if 0 <= age <= 120:
                    return age
                else:
                    age_range_errors += 1
            except:
                pass
        
        date_format_errors += 1
        return None
    except Exception:
        date_format_errors += 1
        return None

# Apply with progress bar
tqdm.pandas(desc="Calculating ages")
df['age'] = df['date_of_birth_standardized'].progress_apply(calculate_age)

# Count successful age calculations
age_count = df['age'].notna().sum()
log_message(f"Calculated {age_count} ages from {std_date_count} standardized dates ({age_count/std_date_count*100:.1f}%)")
log_message(f"Date format errors: {date_format_errors}, Age range errors: {age_range_errors}")

# Log age distribution for verification
age_stats = df['age'].describe()
log_message("\nAge statistics:")
log_message(f"  Count: {age_stats['count']:.0f}")
log_message(f"  Mean: {age_stats['mean']:.1f} years")
log_message(f"  Min: {age_stats['min']:.0f} years")
log_message(f"  Max: {age_stats['max']:.0f} years")

# 5. Fix zip codes (ensure they are 5 digits)
log_message("\nSTEP 5: Fixing zip codes to 5 digits")

zip_errors = 0

def fix_zip_code(zip_code):
    global zip_errors
    if pd.isna(zip_code):
        return None
    
    try:
        # Handle various types properly
        if isinstance(zip_code, (int, float)):
            zip_str = str(int(zip_code))
        else:
            # Remove any non-numeric characters
            zip_str = re.sub(r'[^0-9]', '', str(zip_code))
        
        # If zip code is less than 5 digits, pad with leading zeros
        if len(zip_str) < 5:
            return zip_str.zfill(5)
        
        # If zip code is more than 5 digits, truncate to first 5
        if len(zip_str) > 5:
            return zip_str[:5]
            
        return zip_str
    except Exception:
        zip_errors += 1
        return str(zip_code)  # Return original as string if conversion fails

# Apply with progress bar
tqdm.pandas(desc="Fixing zip codes")
df['zip_code_fixed'] = df['zip_code'].progress_apply(fix_zip_code)

# Count fixed zip codes
zip_count = df['zip_code_fixed'].notna().sum()
log_message(f"Standardized {zip_count} zip codes from {len(df)} records ({zip_count/len(df)*100:.1f}%)")
log_message(f"Zip code errors: {zip_errors}")

# Preview the data to verify changes
log_message("\nPreview of processed data:")
try:
    preview_columns = ['course_name', 'course_date', 'course_year', 'first_name', 'last_name', 
                      'full_name', 'date_of_birth', 'date_of_birth_standardized', 'age', 
                      'zip_code', 'zip_code_fixed']
    
    # Filter to only include columns that actually exist
    existing_columns = [col for col in preview_columns if col in df.columns]
    preview_df = df[existing_columns].head()
    log_message(str(preview_df))
except Exception as e:
    log_message(f"Error generating preview: {e}")
    # Show what columns are actually available
    log_message(f"Available columns: {df.columns.tolist()}")

# Generate summary statistics
log_message("\nData cleaning summary:")
log_message(f"Total records processed: {len(df)}")
log_message(f"Course dates extracted: {date_count} ({date_count/len(df)*100:.1f}%)")
log_message(f"Course years extracted: {year_count} ({year_count/len(df)*100:.1f}%)")
log_message(f"Full names created: {name_count} ({name_count/len(df)*100:.1f}%)")
log_message(f"Dates of birth standardized: {std_date_count} ({std_date_count/len(df)*100:.1f}%)")
log_message(f"Ages calculated: {age_count} ({age_count/len(df)*100:.1f}%)")
log_message(f"Zip codes fixed: {zip_count} ({zip_count/len(df)*100:.1f}%)")

# Save the cleaned data as Excel
output_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/2nd_cleaned_data_excel.xlsx'
try:
    log_message(f"\nSaving data to Excel file: {output_path}")
    
    # Create a writer object for Excel
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # Write the DataFrame to Excel
        df.to_excel(writer, index=False, sheet_name='Cleaned Data')
        
    log_message(f"Successfully saved cleaned data to '{output_path}'")
except Exception as e:
    log_message(f"Error saving Excel file: {e}")
    
    # Try with different path as fallback
    try:
        fallback_path = 'cleaned_data_excel.xlsx'
        log_message(f"Trying to save to alternate path: {fallback_path}")
        
        with pd.ExcelWriter(fallback_path, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='Cleaned Data')
            
        log_message(f"Saved to alternate path: {fallback_path}")
    except Exception as e2:
        log_message(f"Could not save file: {e2}")

log_message(f"\nData cleaning process completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Starting data cleaning process at 2025-05-07 17:20:36
Loading Excel file: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Merged_ES_Cleaned.xlsx
Successfully loaded Excel file with 29700 rows and 151 columns.
Data shape: (29700, 151)
First few column names: ['name_of_siteschool', 'course_type', 'course_name', 'first_name', 'last_name']

STEP 1: Extracting course date and year from course name
Extracting course_date from course_name...


Extracting dates:   0%|          | 0/29700 [00:00<?, ?it/s]

Successfully extracted 19373 dates from 29700 course names (65.2%)
Converting extracted dates to datetime objects...


Converting dates:   0%|          | 0/29700 [00:00<?, ?it/s]

Successfully extracted 19363 years from 19373 dates (99.9%)

STEP 2: Creating full names from first and last names


...

Standardized 29611 dates of birth from 29700 records (99.7%)

Sample of date standardization:
  Original: 4/9/88 -> Standardized: 04-09-1988
  Original: 9/22/66 -> Standardized: 09-22-1966
  Original: 3/5/81 -> Standardized: 03-05-1981
  Original: 2/7/81 -> Standardized: 02-07-1981
  Original: 12/18/92 -> Standardized: 12-18-1992

STEP 4: Calculating ages from dates of birth


Calculating ages:   0%|          | 0/29700 [00:00<?, ?it/s]

Calculated 29559 ages from 29611 standardized dates (99.8%)
Date format errors: 52, Age range errors: 42

Age statistics:
  Count: 29559
  Mean: 46.5 years
  Min: 0 years
  Max: 99 years

STEP 5: Fixing zip codes to 5 digits


Fixing zip codes:   0%|          | 0/29700 [00:00<?, ?it/s]

Standardized 29610 zip codes from 29700 records (99.7%)
Zip code errors: 0

Preview of processed data:
                                         course_name course_date  course_year  first_name        last_name                 full_name date_of_birth date_of_birth_standardized   age  zip_code zip_code_fixed
0  The Latino Support Network Inc- Hugo Carvajal ...     1/12/22       2022.0       Flori           Robles              Flori Robles        4/9/88                 04-09-1988  37.0    1906.0          01906
1  Codman Square NDC- Prince Charles   SB 4/6/21 ...      4/6/21       2021.0    Penelope  Ragland- Godwin  Penelope Ragland- Godwin       9/22/66                 09-22-1966  58.0    2114.0          02114
2  MakeIT Haverhill- Tim Haynes Frank Vasquez SB ...     4/13/22       2022.0     Johanna          Hidalgo           Johanna Hidalgo        3/5/81                 03-05-1981  44.0    1835.0          01835
3  YMCA International Learning Ce- Sarah Poole SB...     4/12/21       2021.0

In [1]:
#POST course data cleaning

import pandas as pd
import re
import os
import numpy as np
from datetime import datetime

def clean_excel_data(input_file):
    """
    Clean and transform Excel data according to the specified requirements.
    
    Parameters:
    input_file (str): Path to the input Excel file
    """
    print(f"Reading data from {input_file}...")
    
    # Read the Excel file
    df = pd.read_excel(input_file)
    original_row_count = len(df)
    print(f"Found {original_row_count} rows of data.")
    
    # Create output file path in the same directory
    input_dir = os.path.dirname(input_file)
    input_filename = os.path.basename(input_file)
    base_name = os.path.splitext(input_filename)[0]
    output_file = os.path.join(input_dir, f"{base_name}_2cleaned.xlsx")
    
    # 1. Extract course_date and course_year from course_name
    print("Extracting course date and year from course names...")
    
    def extract_date_from_course_name(course_name):
        if pd.isna(course_name):
            return None
            
        course_str = str(course_name)
        
        # Look for standard date patterns like M/D/YY or MM/DD/YY
        date_pattern = r'\b(\d{1,2}/\d{1,2}/\d{2})\b'
        match = re.search(date_pattern, course_str)
        
        if match:
            date_str = match.group(1)
            try:
                date_obj = pd.to_datetime(date_str, format='%m/%d/%y')
                return date_obj.strftime('%m/%d/%Y')
            except:
                return date_str
        
        # Look for dates without separators like 21524 (2/15/24)
        no_sep_pattern = r'\b(\d{5,6})\b'
        matches = re.findall(no_sep_pattern, course_str)
        
        for match in matches:
            if len(match) == 5:  # Format: MDDYY
                try:
                    month = int(match[0:1])
                    day = int(match[1:3])
                    year = int(match[3:5])
                    
                    if 1 <= month <= 12 and 1 <= day <= 31:
                        date_str = f"{month}/{day}/{year}"
                        date_obj = pd.to_datetime(date_str, format='%m/%d/%y')
                        return date_obj.strftime('%m/%d/%Y')
                except:
                    pass
            
            elif len(match) == 6:  # Format: MMDDYY
                try:
                    month = int(match[0:2])
                    day = int(match[2:4])
                    year = int(match[4:6])
                    
                    if 1 <= month <= 12 and 1 <= day <= 31:
                        date_str = f"{month}/{day}/{year}"
                        date_obj = pd.to_datetime(date_str, format='%m/%d/%y')
                        return date_obj.strftime('%m/%d/%Y')
                except:
                    pass
        
        return None
    
    def extract_year_from_date(date_str):
        if pd.isna(date_str):
            return None
            
        try:
            # Extract the year from the standardized date (MM/DD/YYYY)
            date_parts = date_str.split('/')
            if len(date_parts) == 3:
                return int(date_parts[2])
            return None
        except:
            return None
    
    # Apply the date extraction and conversion
    df['course_date'] = df['course_name'].apply(extract_date_from_course_name)
    df['course_year'] = df['course_date'].apply(extract_year_from_date)
    
    # 2. Create full_name by joining first_name and last_name
    print("Creating full names...")
    
    # Handle potential missing values
    df['first_name'] = df['first_name'].fillna('')
    df['last_name'] = df['last_name'].fillna('')
    df['full_name'] = df.apply(lambda row: (row['first_name'] + ' ' + row['last_name']).strip(), axis=1)
    
    # 3. Standardize date of birth column
    print("Standardizing date of birth formats...")
    
    def standardize_dob(dob):
        if pd.isna(dob):
            return None
            
        dob_str = str(dob).strip().replace('-', '/').replace('.', '/').replace(' ', '')
        
        # Try to handle various formats
        
        # Check if it's already in a standard format
        try:
            date_obj = pd.to_datetime(dob_str, errors='raise')
            # For two-digit years, ensure proper century (1900s not 2000s for older dates)
            if date_obj.year > datetime.now().year:
                date_obj = date_obj.replace(year=date_obj.year - 100)
            return date_obj.strftime('%m-%d-%Y')
        except:
            pass
        
        # Handle numeric-only formats without separators
        if dob_str.isdigit():
            # MMDDYYYY format (8 digits)
            if len(dob_str) == 8:
                month = int(dob_str[0:2])
                day = int(dob_str[2:4])
                year = int(dob_str[4:8])
                
                if 1 <= month <= 12 and 1 <= day <= 31:
                    try:
                        return f"{month:02d}-{day:02d}-{year}"
                    except:
                        pass
            
            # MMDDYY format (6 digits)
            elif len(dob_str) == 6:
                month = int(dob_str[0:2])
                day = int(dob_str[2:4])
                year = int(dob_str[4:6])
                
                if 1 <= month <= 12 and 1 <= day <= 31:
                    # Adjust years to 19XX or 20XX
                    full_year = 1900 + year if year >= 50 else 2000 + year
                    try:
                        return f"{month:02d}-{day:02d}-{full_year}"
                    except:
                        pass
            
            # MDDYYYY format (7 digits)
            elif len(dob_str) == 7:
                month = int(dob_str[0:1])
                day = int(dob_str[1:3])
                year = int(dob_str[3:7])
                
                if 1 <= month <= 12 and 1 <= day <= 31:
                    try:
                        return f"{month:02d}-{day:02d}-{year}"
                    except:
                        pass
        
        # Formats with separators (M/D/YY, MM/DD/YYYY, etc.)
        if '/' in dob_str:
            parts = dob_str.split('/')
            if len(parts) == 3:
                month, day, year = parts
                
                # Handle 2-digit years
                if len(year) == 2:
                    year_int = int(year)
                    full_year = 1900 + year_int if year_int >= 50 else 2000 + year_int
                else:
                    full_year = int(year)
                
                try:
                    month_int = int(month)
                    day_int = int(day)
                    if 1 <= month_int <= 12 and 1 <= day_int <= 31:
                        return f"{month_int:02d}-{day_int:02d}-{full_year}"
                except:
                    pass
        
        # If we couldn't parse it properly, return the original string
        return dob_str
    
    df['date_of_birth'] = df['date_of_birth'].apply(standardize_dob)
    
    # 4. Standardize email addresses
    print("Standardizing email addresses...")
    
    def standardize_email(email):
        if pd.isna(email):
            return None
        return str(email).lower().strip()
    
    # Ensure we're using the correct column name for email
    if 'email_address' in df.columns:
        df['email_address'] = df['email_address'].apply(standardize_email)
    
    # 5. Calculate age at course
    print("Calculating age at course...")
    
    def calculate_age(dob, course_year):
        if pd.isna(dob) or pd.isna(course_year):
            return None
        
        try:
            # Parse the DOB
            dob_obj = pd.to_datetime(dob, format='%m-%d-%Y')
            
            # Calculate age based on the course year
            age = course_year - dob_obj.year
            
            # Adjust age if the birthday hasn't occurred yet in the course year
            # We'll consider the person's age at December 31st of the course year
            # as we don't have specific course dates for all entries
            return age
        except:
            return None
    
    df['age_at_course'] = df.apply(lambda row: calculate_age(row['date_of_birth'], row['course_year']), axis=1)
    
    # 6. Count responses for each column
    print("Calculating response counts for each column...")
    response_counts = df.count()
    
    # Save the processed data
    print(f"Saving cleaned data to {output_file}...")
    
    # Create a new Excel writer
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Write the main data
        df.to_excel(writer, sheet_name='Cleaned Data', index=False)
        
        # Create a summary DataFrame for the counts
        summary_df = pd.DataFrame({
            'Column': response_counts.index,
            'Response Count': response_counts.values,
            'Percentage': (response_counts.values / original_row_count * 100).round(2),
            'Missing Values': original_row_count - response_counts.values
        })
        
        # Write the summary to a new sheet
        summary_df.to_excel(writer, sheet_name='Response Summary', index=False)
    
    print("Data cleaning complete!")
    print("\nColumn Response Summary:")
    for col, count, pct in zip(summary_df['Column'], summary_df['Response Count'], summary_df['Percentage']):
        print(f"{col}: {count} responses ({pct}%)")
    
    return df, summary_df

if __name__ == "__main__":
    input_file = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS.xlsx'
    df, summary = clean_excel_data(input_file)
    print(f"\nCleaned data has been saved successfully!")

Reading data from /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS.xlsx...
Found 7806 rows of data.
Extracting course date and year from course names...
Creating full names...
Standardizing date of birth formats...
Standardizing email addresses...
Calculating age at course...
Calculating response counts for each column...
Saving cleaned data to /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx...
Data cleaning complete!

Column Response Summary:
submitted_date: 7647 responses (97.96%)
course_name: 7686 responses (98.46%)
course_type: 7700 responses (98.64%)
first_name: 7806 responses (100.0%)
middle_initial: 3165 responses (40.55%)
last_name: 7806 responses (100.0%)
date_of_birth: 7748 responses (99.26%)
email_address: 7746 responses (99.23%)
please_describe_your_confidence_level_using_the_internet_to_find_information_you_need: 7748 responses (99.26%)
creating_and_se

In [3]:
#pre and post merge stats

import pandas as pd
import os

def analyze_survey_matches(pre_course_path, post_course_path):
    """
    Analyze how many learners completed both pre-enrollment and post-course surveys
    by matching email addresses between the two datasets.
    
    Parameters:
    pre_course_path (str): Path to the pre-enrollment survey Excel file
    post_course_path (str): Path to the post-course survey Excel file
    """
    print("Loading pre-enrollment survey data...")
    pre_df = pd.read_excel(pre_course_path)
    
    print("Loading post-course survey data...")
    post_df = pd.read_excel(post_course_path)
    
    # Get total counts
    pre_total = len(pre_df)
    post_total = len(post_df)
    
    print(f"Pre-enrollment survey: {pre_total} entries")
    print(f"Post-course survey: {post_total} entries")
    
    # Check if email_address column exists in both datasets
    if 'email_address' not in pre_df.columns:
        print("Error: 'email_address' column not found in pre-enrollment dataset")
        print(f"Available columns: {pre_df.columns.tolist()}")
        return
    
    if 'email_address' not in post_df.columns:
        print("Error: 'email_address' column not found in post-course dataset")
        print(f"Available columns: {post_df.columns.tolist()}")
        return
    
    # Clean email addresses to ensure fair comparison
    def clean_email(email):
        if pd.isna(email):
            return None
        return str(email).lower().strip()
    
    pre_df['email_clean'] = pre_df['email_address'].apply(clean_email)
    post_df['email_clean'] = post_df['email_address'].apply(clean_email)
    
    # Remove null emails
    pre_df_valid = pre_df.dropna(subset=['email_clean'])
    post_df_valid = post_df.dropna(subset=['email_clean'])
    
    pre_valid_count = len(pre_df_valid)
    post_valid_count = len(post_df_valid)
    
    print(f"Pre-enrollment survey: {pre_valid_count} entries with valid emails")
    print(f"Post-course survey: {post_valid_count} entries with valid emails")
    
    # Get unique emails
    pre_emails = set(pre_df_valid['email_clean'])
    post_emails = set(post_df_valid['email_clean'])
    
    pre_unique_count = len(pre_emails)
    post_unique_count = len(post_emails)
    
    print(f"Pre-enrollment survey: {pre_unique_count} unique emails")
    print(f"Post-course survey: {post_unique_count} unique emails")
    
    # Find matching emails
    matching_emails = pre_emails.intersection(post_emails)
    matching_count = len(matching_emails)
    
    print(f"\nFound {matching_count} matching emails between datasets")
    print(f"Match rate: {(matching_count / pre_unique_count * 100):.2f}% of pre-enrollment emails")
    print(f"Match rate: {(matching_count / post_unique_count * 100):.2f}% of post-course emails")
    
    # Create a merged dataset with only the matching records
    pre_matched = pre_df_valid[pre_df_valid['email_clean'].isin(matching_emails)]
    post_matched = post_df_valid[post_df_valid['email_clean'].isin(matching_emails)]
    
    # Get a sample of matched emails for verification
    sample_size = min(5, matching_count)
    sample_emails = list(matching_emails)[:sample_size]
    
    print(f"\nSample of matched emails (first {sample_size}):")
    for i, email in enumerate(sample_emails, 1):
        print(f"{i}. {email}")
    
    # Save match report
    output_dir = os.path.dirname(pre_course_path)
    output_file = os.path.join(output_dir, "survey_match_analysis.xlsx")
    
    # Create a summary dataframe
    summary_data = {
        'Metric': [
            'Total Pre-Enrollment Records',
            'Total Post-Course Records',
            'Valid Pre-Enrollment Emails',
            'Valid Post-Course Emails',
            'Unique Pre-Enrollment Emails',
            'Unique Post-Course Emails',
            'Matching Emails',
            'Pre-Enrollment Match Rate',
            'Post-Course Match Rate'
        ],
        'Value': [
            pre_total,
            post_total,
            pre_valid_count,
            post_valid_count,
            pre_unique_count,
            post_unique_count,
            matching_count,
            f"{(matching_count / pre_unique_count * 100):.2f}%",
            f"{(matching_count / post_unique_count * 100):.2f}%"
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    
    # Create a DataFrame with the list of matching emails
    matches_df = pd.DataFrame({'Matching Emails': list(matching_emails)})
    
    # Save to Excel
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        summary_df.to_excel(writer, sheet_name='Match Summary', index=False)
        matches_df.to_excel(writer, sheet_name='Matching Emails', index=False)
        
        # Also save a sample of pre and post data for matched emails
        if matching_count > 0:
            sample_email = sample_emails[0]
            pre_sample = pre_df_valid[pre_df_valid['email_clean'] == sample_email]
            post_sample = post_df_valid[post_df_valid['email_clean'] == sample_email]
            
            sample_comparison = pd.DataFrame({
                'Field': ['Email Address'],
                'Pre-Enrollment Value': [sample_email],
                'Post-Course Value': [sample_email]
            })
            
            sample_comparison.to_excel(writer, sheet_name='Sample Comparison', index=False)
    
    print(f"\nSaved match analysis to: {output_file}")
    
    return {
        'pre_total': pre_total,
        'post_total': post_total,
        'matching_count': matching_count,
        'matching_emails': matching_emails,
        'pre_df': pre_df,
        'post_df': post_df
    }

def merge_survey_data(results, output_path=None):
    """
    Merge the pre-enrollment and post-course survey data based on matching emails.
    
    Parameters:
    results (dict): Results from analyze_survey_matches
    output_path (str, optional): Path to save the merged data
    """
    if results['matching_count'] == 0:
        print("No matching emails found. Cannot merge datasets.")
        return None
    
    pre_df = results['pre_df']
    post_df = results['post_df']
    
    # Prepare for merge
    pre_df_for_merge = pre_df.copy()
    post_df_for_merge = post_df.copy()
    
    # Rename columns in post_df to avoid duplicates
    post_columns = post_df_for_merge.columns
    post_columns_renamed = [f"post_{col}" if col != 'email_clean' and col != 'email_address' else col for col in post_columns]
    post_df_for_merge.columns = post_columns_renamed
    
    # Merge datasets on cleaned email
    merged_df = pd.merge(
        pre_df_for_merge, 
        post_df_for_merge,
        on='email_clean',
        how='inner',
        suffixes=('_pre', '_post')
    )
    
    # If there are duplicate email_address columns, keep only one
    if 'email_address_pre' in merged_df.columns and 'email_address_post' in merged_df.columns:
        merged_df.drop('email_address_post', axis=1, inplace=True)
        merged_df.rename(columns={'email_address_pre': 'email_address'}, inplace=True)
    
    # Save to Excel if output path provided
    if output_path:
        merged_df.to_excel(output_path, index=False)
        print(f"Merged data saved to: {output_path}")
    
    return merged_df

if __name__ == "__main__":
    pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
    post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
    
    # Analyze matches
    results = analyze_survey_matches(pre_course_path, post_course_path)
    
    # If you want to merge the datasets, uncomment this section
    # output_dir = os.path.dirname(pre_course_path)
    # merged_output_path = os.path.join(output_dir, "Merged_Surveys.xlsx")
    # merged_df = merge_survey_data(results, merged_output_path)

Loading pre-enrollment survey data...
Loading post-course survey data...
Pre-enrollment survey: 29700 entries
Post-course survey: 7806 entries
Pre-enrollment survey: 24446 entries with valid emails
Post-course survey: 7746 entries with valid emails
Pre-enrollment survey: 24446 unique emails
Post-course survey: 7746 unique emails

Found 3189 matching emails between datasets
Match rate: 13.05% of pre-enrollment emails
Match rate: 41.17% of post-course emails

Sample of matched emails (first 5):
1. milliee38@gmail.com
2. shanakawap@yahoo.com
3. griseldacruz49@hotmail.com
4. johnrossfrancis9@gmail.com
5. zpt00200@gmail.com

Saved match analysis to: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/survey_match_analysis.xlsx


In [5]:
pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.4/156.4 kB 3.1 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
#using fuzzy matching for further matching

import pandas as pd
import os
import re
from fuzzywuzzy import fuzz, process  # For fuzzy matching

def analyze_survey_matches(pre_course_path, post_course_path, fuzzy_match=True, similarity_threshold=85):
    """
    Analyze how many learners completed both pre-enrollment and post-course surveys
    by matching email addresses between the two datasets with enhanced normalization.
    
    Parameters:
    pre_course_path (str): Path to the pre-enrollment survey Excel file
    post_course_path (str): Path to the post-course survey Excel file
    fuzzy_match (bool): Whether to use fuzzy matching for emails
    similarity_threshold (int): Threshold for fuzzy matching (0-100)
    """
    print("Loading pre-enrollment survey data...")
    pre_df = pd.read_excel(pre_course_path)
    
    print("Loading post-course survey data...")
    post_df = pd.read_excel(post_course_path)
    
    # Get total counts
    pre_total = len(pre_df)
    post_total = len(post_df)
    
    print(f"Pre-enrollment survey: {pre_total} entries")
    print(f"Post-course survey: {post_total} entries")
    
    # Check if email_address column exists in both datasets
    if 'email_address' not in pre_df.columns:
        print("Error: 'email_address' column not found in pre-enrollment dataset")
        print(f"Available columns: {pre_df.columns.tolist()}")
        return
    
    if 'email_address' not in post_df.columns:
        print("Error: 'email_address' column not found in post-course dataset")
        print(f"Available columns: {post_df.columns.tolist()}")
        return
    
    # Enhanced email normalization
    def normalize_email(email):
        if pd.isna(email):
            return None
        
        # Convert to string and lowercase
        email_str = str(email).lower().strip()
        
        # Remove any spaces
        email_str = email_str.replace(' ', '')
        
        # Remove common typos or variations
        email_str = email_str.replace('gmail.con', 'gmail.com')
        email_str = email_str.replace('yaho.com', 'yahoo.com')
        email_str = email_str.replace('yahooo.com', 'yahoo.com')
        email_str = email_str.replace('hotmial.com', 'hotmail.com')
        email_str = email_str.replace('hotmal.com', 'hotmail.com')
        email_str = email_str.replace('gamil.com', 'gmail.com')
        email_str = email_str.replace('gnail.com', 'gmail.com')
        
        # Remove any non-email characters that might have been added
        email_str = re.sub(r'[^\w@.-]', '', email_str)
        
        # If it doesn't look like an email at all, return None
        if '@' not in email_str or '.' not in email_str:
            return None
            
        return email_str
    
    pre_df['email_normalized'] = pre_df['email_address'].apply(normalize_email)
    post_df['email_normalized'] = post_df['email_address'].apply(normalize_email)
    
    # Remove null emails
    pre_df_valid = pre_df.dropna(subset=['email_normalized'])
    post_df_valid = post_df.dropna(subset=['email_normalized'])
    
    pre_valid_count = len(pre_df_valid)
    post_valid_count = len(post_df_valid)
    
    print(f"Pre-enrollment survey: {pre_valid_count} entries with valid emails")
    print(f"Post-course survey: {post_valid_count} entries with valid emails")
    
    # Get unique emails
    pre_emails = set(pre_df_valid['email_normalized'])
    post_emails = set(post_df_valid['email_normalized'])
    
    pre_unique_count = len(pre_emails)
    post_unique_count = len(post_emails)
    
    print(f"Pre-enrollment survey: {pre_unique_count} unique emails")
    print(f"Post-course survey: {post_unique_count} unique emails")
    
    # First find exact matches
    exact_matching_emails = pre_emails.intersection(post_emails)
    exact_matching_count = len(exact_matching_emails)
    
    print(f"\nFound {exact_matching_count} exact matching emails between datasets")
    
    # Additional matches through fuzzy matching if requested
    fuzzy_matches = {}
    additional_matches_count = 0
    
    if fuzzy_match:
        print("\nPerforming fuzzy matching to find additional potential matches...")
        
        # Only try to fuzzy match emails that didn't get an exact match
        pre_unmatched = pre_emails - exact_matching_emails
        post_unmatched = post_emails - exact_matching_emails
        
        # Convert to lists for fuzzy matching
        pre_unmatched_list = list(pre_unmatched)
        post_unmatched_list = list(post_unmatched)
        
        # If either list is too large, limit the fuzzy matching
        max_fuzzy_compare = 5000  # Adjust based on performance needs
        
        if len(pre_unmatched_list) > max_fuzzy_compare or len(post_unmatched_list) > max_fuzzy_compare:
            print(f"Warning: Too many emails for full fuzzy matching. Limiting comparison to {max_fuzzy_compare} emails.")
            if len(pre_unmatched_list) > max_fuzzy_compare:
                pre_unmatched_list = pre_unmatched_list[:max_fuzzy_compare]
            if len(post_unmatched_list) > max_fuzzy_compare:
                post_unmatched_list = post_unmatched_list[:max_fuzzy_compare]
        
        for pre_email in pre_unmatched_list:
            # Use domain-based grouping to reduce comparisons
            pre_domain = pre_email.split('@')[-1] if '@' in pre_email else ''
            post_domain_group = [email for email in post_unmatched_list if email.endswith(pre_domain)]
            
            # Only perform fuzzy matching within same domain group
            if post_domain_group:
                matches = process.extractBests(pre_email, post_domain_group, 
                                             score_cutoff=similarity_threshold, 
                                             limit=1)
                if matches:
                    best_match, score = matches[0]
                    fuzzy_matches[pre_email] = (best_match, score)
                    additional_matches_count += 1
        
        print(f"Found {additional_matches_count} additional potential matches through fuzzy matching")
    
    total_matches = exact_matching_count + additional_matches_count
    
    print(f"\nTotal matching emails: {total_matches}")
    print(f"Exact match rate: {(exact_matching_count / pre_unique_count * 100):.2f}% of pre-enrollment emails")
    print(f"Exact match rate: {(exact_matching_count / post_unique_count * 100):.2f}% of post-course emails")
    print(f"Total match rate: {(total_matches / pre_unique_count * 100):.2f}% of pre-enrollment emails")
    print(f"Total match rate: {(total_matches / post_unique_count * 100):.2f}% of post-course emails")
    
    # Check for common patterns in unmatched emails
    print("\nAnalyzing unmatched emails for patterns...")
    
    # Extract domains from pre and post emails
    def get_domain(email):
        parts = email.split('@')
        return parts[1] if len(parts) > 1 else None
    
    pre_domains = [get_domain(email) for email in pre_emails if get_domain(email)]
    post_domains = [get_domain(email) for email in post_emails if get_domain(email)]
    
    # Count domain frequencies
    from collections import Counter
    pre_domain_counts = Counter(pre_domains)
    post_domain_counts = Counter(post_domains)
    
    # Get top 5 domains in each dataset
    print("\nTop 5 domains in pre-enrollment emails:")
    for domain, count in pre_domain_counts.most_common(5):
        print(f"  {domain}: {count} emails")
    
    print("\nTop 5 domains in post-course emails:")
    for domain, count in post_domain_counts.most_common(5):
        print(f"  {domain}: {count} emails")
    
    # Create a merged dataset with matched records
    # Combine exact and fuzzy matches
    all_matched_pairs = {}
    for email in exact_matching_emails:
        all_matched_pairs[email] = email
    
    for pre_email, (post_email, score) in fuzzy_matches.items():
        all_matched_pairs[pre_email] = post_email
    
    # Save match report
    output_dir = os.path.dirname(pre_course_path)
    output_file = os.path.join(output_dir, "enhanced_survey_match_analysis.xlsx")
    
    # Create a summary dataframe
    summary_data = {
        'Metric': [
            'Total Pre-Enrollment Records',
            'Total Post-Course Records',
            'Valid Pre-Enrollment Emails',
            'Valid Post-Course Emails',
            'Unique Pre-Enrollment Emails',
            'Unique Post-Course Emails',
            'Exact Matching Emails',
            'Fuzzy Matching Emails',
            'Total Matching Emails',
            'Pre-Enrollment Exact Match Rate',
            'Post-Course Exact Match Rate',
            'Pre-Enrollment Total Match Rate',
            'Post-Course Total Match Rate'
        ],
        'Value': [
            pre_total,
            post_total,
            pre_valid_count,
            post_valid_count,
            pre_unique_count,
            post_unique_count,
            exact_matching_count,
            additional_matches_count,
            total_matches,
            f"{(exact_matching_count / pre_unique_count * 100):.2f}%",
            f"{(exact_matching_count / post_unique_count * 100):.2f}%",
            f"{(total_matches / pre_unique_count * 100):.2f}%",
            f"{(total_matches / post_unique_count * 100):.2f}%"
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    
    # Create a DataFrame with exact matches
    exact_matches_df = pd.DataFrame({'Email': list(exact_matching_emails)})
    
    # Create a DataFrame with fuzzy matches
    if fuzzy_matches:
        fuzzy_matches_data = []
        for pre_email, (post_email, score) in fuzzy_matches.items():
            fuzzy_matches_data.append({
                'Pre-Enrollment Email': pre_email,
                'Post-Course Email': post_email,
                'Similarity Score': score
            })
        fuzzy_matches_df = pd.DataFrame(fuzzy_matches_data)
    else:
        fuzzy_matches_df = pd.DataFrame({'Pre-Enrollment Email': [], 'Post-Course Email': [], 'Similarity Score': []})
    
    # Save to Excel
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        summary_df.to_excel(writer, sheet_name='Match Summary', index=False)
        exact_matches_df.to_excel(writer, sheet_name='Exact Matches', index=False)
        fuzzy_matches_df.to_excel(writer, sheet_name='Fuzzy Matches', index=False)
        
        # Also save domain frequency information
        pre_domain_df = pd.DataFrame({
            'Domain': list(pre_domain_counts.keys()),
            'Count': list(pre_domain_counts.values()),
            'Percentage': [(count/pre_unique_count*100) for count in pre_domain_counts.values()]
        }).sort_values('Count', ascending=False)
        
        post_domain_df = pd.DataFrame({
            'Domain': list(post_domain_counts.keys()),
            'Count': list(post_domain_counts.values()),
            'Percentage': [(count/post_unique_count*100) for count in post_domain_counts.values()]
        }).sort_values('Count', ascending=False)
        
        pre_domain_df.to_excel(writer, sheet_name='Pre-Enrollment Domains', index=False)
        post_domain_df.to_excel(writer, sheet_name='Post-Course Domains', index=False)
    
    print(f"\nSaved enhanced match analysis to: {output_file}")
    
    return {
        'pre_total': pre_total,
        'post_total': post_total,
        'exact_matching_count': exact_matching_count,
        'additional_matches_count': additional_matches_count,
        'total_matches': total_matches,
        'exact_matching_emails': exact_matching_emails,
        'fuzzy_matches': fuzzy_matches,
        'all_matched_pairs': all_matched_pairs,
        'pre_df': pre_df,
        'post_df': post_df
    }

def merge_survey_data(results, output_path=None, include_fuzzy_matches=True):
    """
    Merge the pre-enrollment and post-course survey data based on matching emails.
    
    Parameters:
    results (dict): Results from analyze_survey_matches
    output_path (str, optional): Path to save the merged data
    include_fuzzy_matches (bool): Whether to include fuzzy matches in the merge
    """
    if results['total_matches'] == 0:
        print("No matching emails found. Cannot merge datasets.")
        return None
    
    pre_df = results['pre_df']
    post_df = results['post_df']
    all_matched_pairs = results['all_matched_pairs']
    
    print(f"Merging datasets with {len(all_matched_pairs)} matched email pairs...")
    
    # Create a mapping DataFrame
    mapping_data = []
    for pre_email, post_email in all_matched_pairs.items():
        mapping_data.append({
            'pre_email': pre_email,
            'post_email': post_email
        })
    
    mapping_df = pd.DataFrame(mapping_data)
    
    # Merge using the mapping
    # First, merge mapping with pre_df
    pre_df_norm = pre_df.dropna(subset=['email_normalized'])
    post_df_norm = post_df.dropna(subset=['email_normalized'])
    
    merged_with_pre = pd.merge(
        mapping_df,
        pre_df_norm,
        left_on='pre_email',
        right_on='email_normalized',
        how='left'
    )
    
    # Then merge with post_df
    merged_full = pd.merge(
        merged_with_pre,
        post_df_norm,
        left_on='post_email',
        right_on='email_normalized',
        how='left',
        suffixes=('_pre', '_post')
    )
    
    # Clean up the merged dataset
    # Remove mapping columns
    merged_full.drop(['pre_email', 'post_email'], axis=1, inplace=True)
    
    # Handle duplicate columns
    # If there are duplicate email_address columns, keep only one
    if 'email_address_pre' in merged_full.columns and 'email_address_post' in merged_full.columns:
        # Create a new column that takes the pre value, but if it's missing, takes the post value
        merged_full['email_address'] = merged_full['email_address_pre'].fillna(merged_full['email_address_post'])
        merged_full.drop(['email_address_pre', 'email_address_post'], axis=1, inplace=True)
    
    # Also clean up the email_normalized columns
    if 'email_normalized_pre' in merged_full.columns and 'email_normalized_post' in merged_full.columns:
        merged_full.drop(['email_normalized_pre', 'email_normalized_post'], axis=1, inplace=True)
    
    print(f"Merged dataset has {len(merged_full)} rows and {len(merged_full.columns)} columns")
    
    # Save to Excel if output path provided
    if output_path:
        merged_full.to_excel(output_path, index=False)
        print(f"Merged data saved to: {output_path}")
    
    return merged_full

if __name__ == "__main__":
    pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
    post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
    
    # Analyze matches with enhanced normalization and fuzzy matching
    results = analyze_survey_matches(pre_course_path, post_course_path, fuzzy_match=True, similarity_threshold=85)
    
    # If you want to merge the datasets, uncomment this section
    # output_dir = os.path.dirname(pre_course_path)
    # merged_output_path = os.path.join(output_dir, "Enhanced_Merged_Surveys.xlsx")
    # merged_df = merge_survey_data(results, merged_output_path, include_fuzzy_matches=True)


Loading pre-enrollment survey data...
Loading post-course survey data...
Pre-enrollment survey: 29700 entries
Post-course survey: 7806 entries
Pre-enrollment survey: 24433 entries with valid emails
Post-course survey: 5462 entries with valid emails
Pre-enrollment survey: 24412 unique emails
Post-course survey: 5462 unique emails

Found 3208 exact matching emails between datasets

Performing fuzzy matching to find additional potential matches...
Found 4319 additional potential matches through fuzzy matching

Total matching emails: 7527
Exact match rate: 13.14% of pre-enrollment emails
Exact match rate: 58.73% of post-course emails
Total match rate: 30.83% of pre-enrollment emails
Total match rate: 137.81% of post-course emails

Analyzing unmatched emails for patterns...

Top 5 domains in pre-enrollment emails:
  gmail.com: 18997 emails
  yahoo.com: 1742 emails
  hotmail.com: 1173 emails
  icloud.com: 432 emails
  aol.com: 219 emails

Top 5 domains in post-course emails:
  gmail.com: 469

In [11]:
#Fuzzy Matching analysis

import pandas as pd
import os
import re

def analyze_survey_matches(pre_course_path, post_course_path):
    """
    Analyze how many learners completed both pre-enrollment and post-course surveys
    by matching email addresses between the two datasets with strict normalization.
    
    Parameters:
    pre_course_path (str): Path to the pre-enrollment survey Excel file
    post_course_path (str): Path to the post-course survey Excel file
    """
    print("Loading pre-enrollment survey data...")
    pre_df = pd.read_excel(pre_course_path)
    
    print("Loading post-course survey data...")
    post_df = pd.read_excel(post_course_path)
    
    # Get total counts
    pre_total = len(pre_df)
    post_total = len(post_df)
    
    print(f"Pre-enrollment survey: {pre_total} entries")
    print(f"Post-course survey: {post_total} entries")
    
    # Check if email_address column exists in both datasets
    if 'email_address' not in pre_df.columns:
        print("Error: 'email_address' column not found in pre-enrollment dataset")
        print(f"Available columns: {pre_df.columns.tolist()}")
        return
    
    if 'email_address' not in post_df.columns:
        print("Error: 'email_address' column not found in post-course dataset")
        print(f"Available columns: {post_df.columns.tolist()}")
        return
    
    # Enhanced email normalization
    def normalize_email(email):
        if pd.isna(email):
            return None
        
        # Convert to string and lowercase
        email_str = str(email).lower().strip()
        
        # Remove any spaces
        email_str = email_str.replace(' ', '')
        
        # Fix common typos in domains
        email_str = email_str.replace('gmail.con', 'gmail.com')
        email_str = email_str.replace('yaho.com', 'yahoo.com')
        email_str = email_str.replace('yahooo.com', 'yahoo.com')
        email_str = email_str.replace('hotmial.com', 'hotmail.com')
        email_str = email_str.replace('hotmal.com', 'hotmail.com')
        email_str = email_str.replace('gamil.com', 'gmail.com')
        email_str = email_str.replace('gnail.com', 'gmail.com')
        
        # Remove any non-email characters that might have been added
        email_str = re.sub(r'[^\w@.-]', '', email_str)
        
        # If it doesn't look like an email at all, return None
        if '@' not in email_str or '.' not in email_str:
            return None
            
        return email_str
    
    # Apply normalization
    pre_df['email_normalized'] = pre_df['email_address'].apply(normalize_email)
    post_df['email_normalized'] = post_df['email_address'].apply(normalize_email)
    
    # Remove null emails
    pre_df_valid = pre_df.dropna(subset=['email_normalized'])
    post_df_valid = post_df.dropna(subset=['email_normalized'])
    
    pre_valid_count = len(pre_df_valid)
    post_valid_count = len(post_df_valid)
    
    print(f"Pre-enrollment survey: {pre_valid_count} entries with valid emails")
    print(f"Post-course survey: {post_valid_count} entries with valid emails")
    
    # Get unique emails
    pre_emails = set(pre_df_valid['email_normalized'].unique())
    post_emails = set(post_df_valid['email_normalized'].unique())
    
    pre_unique_count = len(pre_emails)
    post_unique_count = len(post_emails)
    
    print(f"Pre-enrollment survey: {pre_unique_count} unique emails")
    print(f"Post-course survey: {post_unique_count} unique emails")
    
    # Find exact matches only
    matching_emails = pre_emails.intersection(post_emails)
    matching_count = len(matching_emails)
    
    print(f"\nFound {matching_count} matching emails between datasets")
    print(f"Match rate: {(matching_count / pre_unique_count * 100):.2f}% of pre-enrollment emails")
    print(f"Match rate: {(matching_count / post_unique_count * 100):.2f}% of post-course emails")
    
    # Find potential near-matches with very minimal differences
    # These would be typo corrections that are extremely likely to be the same person
    # This is much stricter than fuzzy matching
    typo_corrections = {}
    for pre_email in pre_emails:
        username, domain = pre_email.split('@')
        for post_email in post_emails:
            if '@' in post_email:
                post_username, post_domain = post_email.split('@')
                
                # Only consider if domains match exactly
                if domain == post_domain:
                    # Look for extremely similar usernames with just one character difference
                    # This catches things like john.smith vs johnsmith or jsmith vs j.smith
                    # But avoids matching completely different people
                    
                    # Check for dot differences
                    pre_no_dots = username.replace('.', '')
                    post_no_dots = post_username.replace('.', '')
                    
                    if pre_no_dots == post_no_dots and pre_email != post_email:
                        typo_corrections[pre_email] = post_email
                    
                    # Check for single digit difference at the end (e.g., john1 vs john)
                    if (username.rstrip('0123456789') == post_username or 
                        post_username.rstrip('0123456789') == username):
                        typo_corrections[pre_email] = post_email
    
    print(f"Found {len(typo_corrections)} additional potential matches with minor differences")
    
    # Get a sample of matched emails for verification
    sample_size = min(5, matching_count)
    sample_emails = list(matching_emails)[:sample_size]
    
    print(f"\nSample of matched emails (first {sample_size}):")
    for i, email in enumerate(sample_emails, 1):
        print(f"{i}. {email}")
    
    # Save match report
    output_dir = os.path.dirname(pre_course_path)
    output_file = os.path.join(output_dir, "strict_match_analysis.xlsx")
    
    # Create a summary dataframe
    summary_data = {
        'Metric': [
            'Total Pre-Enrollment Records',
            'Total Post-Course Records',
            'Valid Pre-Enrollment Emails',
            'Valid Post-Course Emails',
            'Unique Pre-Enrollment Emails',
            'Unique Post-Course Emails',
            'Matching Emails',
            'Potential Minor Typo Matches',
            'Pre-Enrollment Match Rate',
            'Post-Course Match Rate'
        ],
        'Value': [
            pre_total,
            post_total,
            pre_valid_count,
            post_valid_count,
            pre_unique_count,
            post_unique_count,
            matching_count,
            len(typo_corrections),
            f"{(matching_count / pre_unique_count * 100):.2f}%",
            f"{(matching_count / post_unique_count * 100):.2f}%"
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    
    # Create a DataFrame with the list of matching emails
    matches_df = pd.DataFrame({'Exact Matching Emails': list(matching_emails)})
    
    # Create a DataFrame with potential typo corrections
    if typo_corrections:
        typo_data = []
        for pre_email, post_email in typo_corrections.items():
            typo_data.append({
                'Pre-Enrollment Email': pre_email,
                'Post-Course Email': post_email
            })
        typo_df = pd.DataFrame(typo_data)
    else:
        typo_df = pd.DataFrame({'Pre-Enrollment Email': [], 'Post-Course Email': []})
    
    # Extract domains for analysis
    def get_domain(email):
        parts = email.split('@')
        return parts[1] if len(parts) > 1 else None
    
    pre_domains = [get_domain(email) for email in pre_emails if get_domain(email)]
    post_domains = [get_domain(email) for email in post_emails if get_domain(email)]
    
    # Count domain frequencies
    from collections import Counter
    pre_domain_counts = Counter(pre_domains)
    post_domain_counts = Counter(post_domains)
    
    # Get top domains in each dataset
    print("\nTop 5 domains in pre-enrollment emails:")
    for domain, count in pre_domain_counts.most_common(5):
        print(f"  {domain}: {count} emails")
    
    print("\nTop 5 domains in post-course emails:")
    for domain, count in post_domain_counts.most_common(5):
        print(f"  {domain}: {count} emails")
    
    # Save to Excel
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        summary_df.to_excel(writer, sheet_name='Match Summary', index=False)
        matches_df.to_excel(writer, sheet_name='Exact Matches', index=False)
        typo_df.to_excel(writer, sheet_name='Potential Typo Matches', index=False)
        
        # Also save domain frequency information
        pre_domain_df = pd.DataFrame({
            'Domain': list(pre_domain_counts.keys()),
            'Count': list(pre_domain_counts.values()),
            'Percentage': [(count/pre_unique_count*100) for count in pre_domain_counts.values()]
        }).sort_values('Count', ascending=False)
        
        post_domain_df = pd.DataFrame({
            'Domain': list(post_domain_counts.keys()),
            'Count': list(post_domain_counts.values()),
            'Percentage': [(count/post_unique_count*100) for count in post_domain_counts.values()]
        }).sort_values('Count', ascending=False)
        
        pre_domain_df.to_excel(writer, sheet_name='Pre-Enrollment Domains', index=False)
        post_domain_df.to_excel(writer, sheet_name='Post-Course Domains', index=False)
    
    print(f"\nSaved match analysis to: {output_file}")
    
    return {
        'pre_total': pre_total,
        'post_total': post_total,
        'matching_count': matching_count,
        'matching_emails': matching_emails,
        'typo_corrections': typo_corrections,
        'pre_df': pre_df,
        'post_df': post_df
    }

def merge_survey_data(results, output_path=None, include_typo_matches=False):
    """
    Merge the pre-enrollment and post-course survey data based on matching emails.
    
    Parameters:
    results (dict): Results from analyze_survey_matches
    output_path (str, optional): Path to save the merged data
    include_typo_matches (bool): Whether to include typo-corrected matches
    """
    if results['matching_count'] == 0 and not (include_typo_matches and results['typo_corrections']):
        print("No matching emails found. Cannot merge datasets.")
        return None
    
    pre_df = results['pre_df']
    post_df = results['post_df']
    matching_emails = results['matching_emails']
    typo_corrections = results['typo_corrections'] if include_typo_matches else {}
    
    # Prepare for merge
    pre_df_for_merge = pre_df.copy()
    post_df_for_merge = post_df.copy()
    
    # Filter datasets to only include matched records
    if include_typo_matches:
        all_pre_emails = list(matching_emails) + list(typo_corrections.keys())
        all_post_emails = list(matching_emails) + list(typo_corrections.values())
    else:
        all_pre_emails = list(matching_emails)
        all_post_emails = list(matching_emails)
    
    pre_matched_df = pre_df_for_merge[pre_df_for_merge['email_normalized'].isin(all_pre_emails)]
    post_matched_df = post_df_for_merge[post_df_for_merge['email_normalized'].isin(all_post_emails)]
    
    print(f"Filtered to {len(pre_matched_df)} pre-enrollment records and {len(post_matched_df)} post-course records")
    
    # Create a mapping for exact matches
    mapping_data = []
    
    # Add exact matches
    for email in matching_emails:
        mapping_data.append({
            'pre_email': email,
            'post_email': email,
            'match_type': 'exact'
        })
    
    # Add typo corrections if requested
    if include_typo_matches:
        for pre_email, post_email in typo_corrections.items():
            mapping_data.append({
                'pre_email': pre_email,
                'post_email': post_email,
                'match_type': 'typo'
            })
    
    mapping_df = pd.DataFrame(mapping_data)
    
    # Merge using the mapping
    # First, merge mapping with pre_df
    merged_with_pre = pd.merge(
        mapping_df,
        pre_matched_df,
        left_on='pre_email',
        right_on='email_normalized',
        how='left'
    )
    
    # Then merge with post_df
    merged_full = pd.merge(
        merged_with_pre,
        post_matched_df,
        left_on='post_email',
        right_on='email_normalized',
        how='left',
        suffixes=('_pre', '_post')
    )
    
    # Clean up the merged dataset
    # Remove mapping columns
    merged_full.drop(['pre_email', 'post_email'], axis=1, inplace=True)
    
    # Handle duplicate columns
    if 'email_address_pre' in merged_full.columns and 'email_address_post' in merged_full.columns:
        # Create a new column that takes the pre value, but if it's missing, takes the post value
        merged_full['email_address'] = merged_full['email_address_pre'].fillna(merged_full['email_address_post'])
        merged_full.drop(['email_address_pre', 'email_address_post'], axis=1, inplace=True)
    
    # Also clean up the email_normalized columns
    if 'email_normalized_pre' in merged_full.columns and 'email_normalized_post' in merged_full.columns:
        merged_full.drop(['email_normalized_pre', 'email_normalized_post'], axis=1, inplace=True)
    
    print(f"Merged dataset has {len(merged_full)} rows and {len(merged_full.columns)} columns")
    
    # Save to Excel if output path provided
    if output_path:
        merged_full.to_excel(output_path, index=False)
        print(f"Merged data saved to: {output_path}")
    
    return merged_full

if __name__ == "__main__":
    pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
    post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
    
    # Analyze matches with enhanced normalization but no fuzzy matching
    results = analyze_survey_matches(pre_course_path, post_course_path)
    
    # If you want to merge the datasets, uncomment this section
    # output_dir = os.path.dirname(pre_course_path)
    # merged_output_path = os.path.join(output_dir, "Strict_Merged_Surveys.xlsx")
    # merged_df = merge_survey_data(results, merged_output_path, include_typo_matches=True)

Loading pre-enrollment survey data...
Loading post-course survey data...
Pre-enrollment survey: 29700 entries
Post-course survey: 7806 entries
Pre-enrollment survey: 24433 entries with valid emails
Post-course survey: 5462 entries with valid emails
Pre-enrollment survey: 24412 unique emails
Post-course survey: 5462 unique emails

Found 3208 matching emails between datasets
Match rate: 13.14% of pre-enrollment emails
Match rate: 58.73% of post-course emails
Found 1264 additional potential matches with minor differences

Sample of matched emails (first 5):
1. milliee38@gmail.com
2. shanakawap@yahoo.com
3. griseldacruz49@hotmail.com
4. johnrossfrancis9@gmail.com
5. zpt00200@gmail.com

Top 5 domains in pre-enrollment emails:
  gmail.com: 18997 emails
  yahoo.com: 1742 emails
  hotmail.com: 1173 emails
  icloud.com: 432 emails
  aol.com: 219 emails

Top 5 domains in post-course emails:
  gmail.com: 4699 emails
  yahoo.com: 219 emails
  hotmail.com: 135 emails
  icloud.com: 59 emails
  bpsad

In [18]:
#matching name and dob to improing the matching
import pandas as pd
import os
import re
from datetime import datetime

def analyze_survey_matches(pre_course_path, post_course_path):
    """
    Analyze matches between pre-enrollment and post-course surveys
    using multiple identifiers with comprehensive domain correction.
    
    Parameters:
    pre_course_path (str): Path to the pre-enrollment survey Excel file
    post_course_path (str): Path to the post-course survey Excel file
    """
    # Store statistics for reporting
    stats = {}
    
    print("Loading pre-enrollment survey data...")
    pre_df = pd.read_excel(pre_course_path)
    
    print("Loading post-course survey data...")
    post_df = pd.read_excel(post_course_path)
    
    # Get total counts
    pre_total = len(pre_df)
    post_total = len(post_df)
    
    stats['pre_total'] = pre_total
    stats['post_total'] = post_total
    
    print(f"Pre-enrollment survey: {pre_total} entries")
    print(f"Post-course survey: {post_total} entries")
    
    # Step 1: Standardize and prepare data for matching
    
    # Fix field name discrepancy for date of birth
    if 'date_of_birth_standardized' in pre_df.columns and 'date_of_birth' in post_df.columns:
        print("Aligning date of birth column names...")
        pre_df.rename(columns={'date_of_birth_standardized': 'date_of_birth'}, inplace=True)
    
    # Comprehensive domain correction dictionary
    domain_corrections = {
        # Gmail variations
        'gmai.com': 'gmail.com',
        'gmil.com': 'gmail.com',
        'gmal.com': 'gmail.com',
        'gamail.com': 'gmail.com',
        'gimail.com': 'gmail.com',
        'gmaill.com': 'gmail.com',
        'gmail.comm': 'gmail.com',
        'gmial.com': 'gmail.com',
        'gmail.cm': 'gmail.com',
        'gmail.org': 'gmail.com',
        'gemail.com': 'gmail.com',
        'gmail.com.com': 'gmail.com',
        'gmai.coml': 'gmail.com',
        'testgmail.com': 'gmail.com',
        'gmail.coom': 'gmail.com',
        'jmail.com': 'gmail.com',
        'gmeil.com': 'gmail.com',
        'gmall.com': 'gmail.com',
        'gail.com': 'gmail.com',
        'gmail.co': 'gmail.com',
        'gmaiil.com': 'gmail.com',
        'gmaim.com': 'gmail.com',
        'gmail.om': 'gmail.com',
        'gmail.ccom': 'gmail.com',
        'g.com': 'gmail.com',
        'gmail.vom': 'gmail.com',
        'gmali.com': 'gmail.com',
        'gmaio.com': 'gmail.com',
        'g-mail.com': 'gmail.com',
        'gamill.com': 'gmail.com',
        'gmaol.com': 'gmail.com',
        'fakegmail.com': 'gmail.com',
        'gmsil.com': 'gmail.com',
        'gmaii.com': 'gmail.com',
        'gmail.oeg': 'gmail.com',
        '27gmail.com': 'gmail.com',
        'gmanil.com': 'gmail.com',
        'gmaili.com': 'gmail.com',
        'gmail.crom': 'gmail.com',
        'gmail.com11': 'gmail.com',
        'gmill.com': 'gmail.com',
        'gmail.com3': 'gmail.com',
        'gnmail.com': 'gmail.com',
        '7gmail.com': 'gmail.com',
        '08gmail.com': 'gmail.com',
        'gmaij.com': 'gmail.com',
        '729gmail.com': 'gmail.com',
        'gmail.coml.com': 'gmail.com',
        '23gmail.com': 'gmail.com',
        
        # Yahoo variations
        'yahoo.con': 'yahoo.com',
        'yaoo.com': 'yahoo.com',
        'hayoo.com': 'yahoo.com',
        'yhoo.com': 'yahoo.com',
        'yahoo.comm': 'yahoo.com',
        'myyahoo.com': 'yahoo.com',
        'yahoo.comb': 'yahoo.com',
        'yahool.com': 'yahoo.com',
        'yaho.cm': 'yahoo.com',
        'yahio.un': 'yahoo.com',
        
        # Hotmail variations
        'hmail.com': 'hotmail.com',
        'hotmai.com': 'hotmail.com',
        'hotamil.com': 'hotmail.com',
        'hotmail.con': 'hotmail.com',
        'hoo.com': 'hotmail.com',
        'hotmiail.com': 'hotmail.com',
        'htomail.com': 'hotmail.com',
        'hiotmail.com': 'hotmail.com',
        'hitmail.com': 'hotmail.com',
        
        # iCloud variations
        'icould.com': 'icloud.com',
        'iclod.com': 'icloud.com',
        'cloud.com': 'icloud.com',
        'icoud.com': 'icloud.com',
        'ichoud.com': 'icloud.com',
        'iclou.com': 'icloud.com',
        
        # AOL variations
        'ao.com': 'aol.com',
        'alo.com': 'aol.com',
        'aol.co': 'aol.com',
        'aol.om': 'aol.com',
        
        # Boston Public Schools variations
        'bostonk12.com': 'bostonk12.org',
        'boston12.org': 'bostonk12.org',
        'bostobpublicschools.org': 'bostonpublicschools.org',
        'bostonnpublicschools.org': 'bostonpublicschools.org',
        'bostonnk12.org': 'bostonk12.org',
        'bostonpulicschools.org': 'bostonpublicschools.org',
        'bostonpublicschool.org.org': 'bostonpublicschools.org',
        
        # BPS Adult Ed variations
        'pbsadulted.com': 'bpsadulted.com',
        'bpsadulred.com': 'bpsadulted.com',
        'adulted.com': 'bpsadulted.com',
        
        # Other common corrections
        'live.con': 'live.com',
        'live.cm': 'live.com',
        'outlook.pt': 'outlook.com',
        'verizon.com': 'verizon.net',
        'comcast.com': 'comcast.net',
        'mail.com': 'gmail.com',  # Often a typo for gmail
        'email.com': 'gmail.com', # Often a typo for gmail
        'cmail.com': 'gmail.com', # Often a typo for gmail
        'ail.com': 'gmail.com'    # Often a typo for gmail
    }
    
    # Email standardization with domain correction
    def normalize_email(email):
        if pd.isna(email):
            return None
        
        # Convert to string and lowercase
        email_str = str(email).lower().strip()
        
        # Remove any spaces and special characters
        email_str = email_str.replace(' ', '')
        email_str = re.sub(r'[^\w@.-]', '', email_str)
        
        # Basic validation
        if '@' not in email_str or '.' not in email_str:
            return None
            
        # Extract domain for correction
        username, domain = email_str.split('@', 1)
        
        # Apply domain correction if needed
        corrected_domain = domain_corrections.get(domain, domain)
        
        # Rebuild email with potentially corrected domain
        corrected_email = f"{username}@{corrected_domain}"
        
        return corrected_email
    
    # Name standardization
    def normalize_name(name):
        if pd.isna(name):
            return None
        
        # Convert to string, lowercase, and strip spaces
        name_str = str(name).lower().strip()
        
        # Remove extra spaces
        name_str = re.sub(r'\s+', ' ', name_str)
        
        # Remove non-alphabetic characters except spaces
        name_str = re.sub(r'[^a-z ]', '', name_str)
        
        return name_str
    
    # Date of birth standardization - extract year, month, and day components
    def standardize_dob(dob):
        if pd.isna(dob):
            return None, None, None
            
        # Try to parse the date
        try:
            if isinstance(dob, str):
                # Replace common separators with a standard one
                dob_str = dob.replace('-', '/').replace('.', '/').replace(' ', '')
                
                # Try various date formats
                formats = ['%m/%d/%Y', '%m/%d/%y', '%Y/%m/%d', '%d/%m/%Y', '%m-%d-%Y']
                
                for fmt in formats:
                    try:
                        date_obj = datetime.strptime(dob_str, fmt)
                        # For two-digit years, adjust century if needed
                        if '%y' in fmt and date_obj.year > datetime.now().year:
                            date_obj = date_obj.replace(year=date_obj.year - 100)
                        return date_obj.year, date_obj.month, date_obj.day
                    except:
                        continue
                
                # If we couldn't parse with standard formats, try handling numeric-only strings
                if dob_str.isdigit():
                    if len(dob_str) == 8:  # MMDDYYYY
                        month = int(dob_str[0:2])
                        day = int(dob_str[2:4])
                        year = int(dob_str[4:8])
                        if 1 <= month <= 12 and 1 <= day <= 31:
                            return year, month, day
                    elif len(dob_str) == 6:  # MMDDYY
                        month = int(dob_str[0:2])
                        day = int(dob_str[2:4])
                        year = int(dob_str[4:6])
                        if 1 <= month <= 12 and 1 <= day <= 31:
                            full_year = 1900 + year if year >= 50 else 2000 + year
                            return full_year, month, day
            else:
                # If it's already a datetime or similar type
                try:
                    date_obj = pd.to_datetime(dob)
                    return date_obj.year, date_obj.month, date_obj.day
                except:
                    pass
        except:
            pass
            
        return None, None, None
    
    # Apply standardization to both datasets
    print("Standardizing data for matching...")
    
    # Process pre-enrollment data
    print("Processing pre-enrollment emails...")
    pre_df['original_email'] = pre_df['email_address']
    pre_df['email_normalized'] = pre_df['email_address'].apply(normalize_email)
    
    # Get email stats before domain correction
    pre_valid_email_before = pre_df.dropna(subset=['email_address']).shape[0]
    pre_valid_normalized_before = pre_df.dropna(subset=['email_normalized']).shape[0]
    
    stats['pre_valid_email_before'] = pre_valid_email_before
    stats['pre_valid_normalized_before'] = pre_valid_normalized_before
    
    print(f"Pre-enrollment emails before normalization: {pre_valid_email_before}")
    print(f"Pre-enrollment emails after normalization: {pre_valid_normalized_before}")
    
    # Check if 'full_name' exists, if not, try to create it
    if 'full_name' not in pre_df.columns:
        if 'first_name' in pre_df.columns and 'last_name' in pre_df.columns:
            pre_df['first_name'] = pre_df['first_name'].fillna('')
            pre_df['last_name'] = pre_df['last_name'].fillna('')
            pre_df['full_name'] = pre_df['first_name'] + ' ' + pre_df['last_name']
            pre_df['full_name'] = pre_df['full_name'].str.strip()
    
    pre_df['name_normalized'] = pre_df['full_name'].apply(normalize_name)
    pre_df['first_name_norm'] = pre_df['first_name'].apply(normalize_name) if 'first_name' in pre_df.columns else None
    pre_df['last_name_norm'] = pre_df['last_name'].apply(normalize_name) if 'last_name' in pre_df.columns else None
    
    # Extract DOB components
    print("Processing pre-enrollment dates of birth...")
    dob_results = pre_df['date_of_birth'].apply(standardize_dob)
    pre_df['dob_year'], pre_df['dob_month'], pre_df['dob_day'] = zip(*dob_results)
    
    # Process post-course data
    print("Processing post-course emails...")
    post_df['original_email'] = post_df['email_address']
    post_df['email_normalized'] = post_df['email_address'].apply(normalize_email)
    
    # Get email stats before domain correction
    post_valid_email_before = post_df.dropna(subset=['email_address']).shape[0]
    post_valid_normalized_before = post_df.dropna(subset=['email_normalized']).shape[0]
    
    stats['post_valid_email_before'] = post_valid_email_before
    stats['post_valid_normalized_before'] = post_valid_normalized_before
    
    print(f"Post-course emails before normalization: {post_valid_email_before}")
    print(f"Post-course emails after normalization: {post_valid_normalized_before}")
    
    if 'full_name' not in post_df.columns:
        if 'first_name' in post_df.columns and 'last_name' in post_df.columns:
            post_df['first_name'] = post_df['first_name'].fillna('')
            post_df['last_name'] = post_df['last_name'].fillna('')
            post_df['full_name'] = post_df['first_name'] + ' ' + post_df['last_name']
            post_df['full_name'] = post_df['full_name'].str.strip()
    
    post_df['name_normalized'] = post_df['full_name'].apply(normalize_name)
    post_df['first_name_norm'] = post_df['first_name'].apply(normalize_name) if 'first_name' in post_df.columns else None
    post_df['last_name_norm'] = post_df['last_name'].apply(normalize_name) if 'last_name' in post_df.columns else None
    
    # Extract DOB components
    print("Processing post-course dates of birth...")
    dob_results = post_df['date_of_birth'].apply(standardize_dob)
    post_df['dob_year'], post_df['dob_month'], post_df['dob_day'] = zip(*dob_results)
    
    # Step 2: Perform matching across different identifiers
    
    # 2.1 Start with email matching
    print("\nPerforming email matching...")
    
    # Remove invalid emails
    pre_df_valid_email = pre_df.dropna(subset=['email_normalized'])
    post_df_valid_email = post_df.dropna(subset=['email_normalized'])
    
    pre_valid_email_count = len(pre_df_valid_email)
    post_valid_email_count = len(post_df_valid_email)
    
    stats['pre_valid_email_count'] = pre_valid_email_count
    stats['post_valid_email_count'] = post_valid_email_count
    
    print(f"Pre-enrollment records with valid emails: {pre_valid_email_count} ({pre_valid_email_count/pre_total*100:.2f}%)")
    print(f"Post-course records with valid emails: {post_valid_email_count} ({post_valid_email_count/post_total*100:.2f}%)")
    
    # Get unique normalized emails
    pre_emails = set(pre_df_valid_email['email_normalized'].unique())
    post_emails = set(post_df_valid_email['email_normalized'].unique())
    
    pre_unique_email_count = len(pre_emails)
    post_unique_email_count = len(post_emails)
    
    stats['pre_unique_email_count'] = pre_unique_email_count
    stats['post_unique_email_count'] = post_unique_email_count
    
    print(f"Unique emails in pre-enrollment: {pre_unique_email_count}")
    print(f"Unique emails in post-course: {post_unique_email_count}")
    
    # Find matching emails
    email_matches = pre_emails.intersection(post_emails)
    email_match_count = len(email_matches)
    
    stats['email_match_count'] = email_match_count
    
    print(f"Found {email_match_count} matches by email")
    print(f"Email match rate: {email_match_count/pre_unique_email_count*100:.2f}% of pre-enrollment emails")
    print(f"Email match rate: {email_match_count/post_unique_email_count*100:.2f}% of post-course emails")
    
    # Extract domains before and after correction for analysis
    def extract_domain(email):
        if pd.isna(email):
            return None
        try:
            return email.split('@')[1]
        except:
            return None
    
    pre_df['original_domain'] = pre_df['original_email'].apply(extract_domain)
    pre_df['corrected_domain'] = pre_df['email_normalized'].apply(extract_domain)
    
    post_df['original_domain'] = post_df['original_email'].apply(extract_domain)
    post_df['corrected_domain'] = post_df['email_normalized'].apply(extract_domain)
    
    # Count domain corrections
    pre_domain_changes = pre_df[pre_df['original_domain'] != pre_df['corrected_domain']].dropna(subset=['original_domain', 'corrected_domain'])
    post_domain_changes = post_df[post_df['original_domain'] != post_df['corrected_domain']].dropna(subset=['original_domain', 'corrected_domain'])
    
    pre_domain_corrections_count = len(pre_domain_changes)
    post_domain_corrections_count = len(post_domain_changes)
    
    stats['pre_domain_corrections_count'] = pre_domain_corrections_count
    stats['post_domain_corrections_count'] = post_domain_corrections_count
    
    print(f"\nDomain corrections applied:")
    print(f"Pre-enrollment: {pre_domain_corrections_count} domains corrected")
    print(f"Post-course: {post_domain_corrections_count} domains corrected")
    
    # 2.2 Next, try matching by full name + DOB for records without email matches
    print("\nPerforming full name + DOB matching...")
    
    # Get records without email matches
    pre_unmatched_email = pre_df[~pre_df['email_normalized'].isin(email_matches)]
    post_unmatched_email = post_df[~post_df['email_normalized'].isin(email_matches)]
    
    # Create a composite key for name+DOB matching
    def create_name_dob_key(row):
        if pd.isna(row['name_normalized']) or pd.isna(row['dob_year']):
            return None
        return f"{row['name_normalized']}_{row['dob_year']}_{row['dob_month']}_{row['dob_day']}"
    
    pre_unmatched_email['name_dob_key'] = pre_unmatched_email.apply(create_name_dob_key, axis=1)
    post_unmatched_email['name_dob_key'] = post_unmatched_email.apply(create_name_dob_key, axis=1)
    
    # Remove invalid keys
    pre_valid_name_dob = pre_unmatched_email.dropna(subset=['name_dob_key'])
    post_valid_name_dob = post_unmatched_email.dropna(subset=['name_dob_key'])
    
    pre_valid_name_dob_count = len(pre_valid_name_dob)
    post_valid_name_dob_count = len(post_valid_name_dob)
    
    stats['pre_valid_name_dob_count'] = pre_valid_name_dob_count
    stats['post_valid_name_dob_count'] = post_valid_name_dob_count
    
    print(f"Pre-enrollment records with valid name+DOB (unmatched by email): {pre_valid_name_dob_count}")
    print(f"Post-course records with valid name+DOB (unmatched by email): {post_valid_name_dob_count}")
    
    # Get unique keys
    pre_name_dob_keys = set(pre_valid_name_dob['name_dob_key'].unique())
    post_name_dob_keys = set(post_valid_name_dob['name_dob_key'].unique())
    
    # Find matching name+DOB combinations
    name_dob_matches = pre_name_dob_keys.intersection(post_name_dob_keys)
    name_dob_match_count = len(name_dob_matches)
    
    stats['name_dob_match_count'] = name_dob_match_count
    
    print(f"Found {name_dob_match_count} additional matches by full name + DOB")
    
    # 2.3 Try matching by first name, last name, and birth year for remaining records
    print("\nPerforming first name + last name + birth year matching...")
    
    # Get records without email or name+DOB matches
    pre_matched_ids = set(pre_df[pre_df['email_normalized'].isin(email_matches)].index).union(
                       set(pre_valid_name_dob[pre_valid_name_dob['name_dob_key'].isin(name_dob_matches)].index))
    
    post_matched_ids = set(post_df[post_df['email_normalized'].isin(email_matches)].index).union(
                        set(post_valid_name_dob[post_valid_name_dob['name_dob_key'].isin(name_dob_matches)].index))
    
    pre_unmatched = pre_df[~pre_df.index.isin(pre_matched_ids)]
    post_unmatched = post_df[~post_df.index.isin(post_matched_ids)]
    
    # Create a composite key for first+last+year matching
    def create_name_year_key(row):
        if (pd.isna(row['first_name_norm']) or pd.isna(row['last_name_norm']) or 
            pd.isna(row['dob_year'])):
            return None
        return f"{row['first_name_norm']}_{row['last_name_norm']}_{row['dob_year']}"
    
    pre_unmatched['name_year_key'] = pre_unmatched.apply(create_name_year_key, axis=1)
    post_unmatched['name_year_key'] = post_unmatched.apply(create_name_year_key, axis=1)
    
    # Remove invalid keys
    pre_valid_name_year = pre_unmatched.dropna(subset=['name_year_key'])
    post_valid_name_year = post_unmatched.dropna(subset=['name_year_key'])
    
    pre_valid_name_year_count = len(pre_valid_name_year)
    post_valid_name_year_count = len(post_valid_name_year)
    
    stats['pre_valid_name_year_count'] = pre_valid_name_year_count
    stats['post_valid_name_year_count'] = post_valid_name_year_count
    
    print(f"Pre-enrollment records with valid name+year (still unmatched): {pre_valid_name_year_count}")
    print(f"Post-course records with valid name+year (still unmatched): {post_valid_name_year_count}")
    
    # Get unique keys
    pre_name_year_keys = set(pre_valid_name_year['name_year_key'].unique())
    post_name_year_keys = set(post_valid_name_year['name_year_key'].unique())
    
    # Find matching name+year combinations
    name_year_matches = pre_name_year_keys.intersection(post_name_year_keys)
    name_year_match_count = len(name_year_matches)
    
    stats['name_year_match_count'] = name_year_match_count
    
    print(f"Found {name_year_match_count} additional matches by first name + last name + birth year")
    
    # Calculate total matches
    total_matches = email_match_count + name_dob_match_count + name_year_match_count
    stats['total_matches'] = total_matches
    
    print(f"\nTotal matched records: {total_matches}")
    print(f"Overall match rate: {total_matches/pre_unique_email_count*100:.2f}% of pre-enrollment emails")
    print(f"Overall match rate: {total_matches/post_unique_email_count*100:.2f}% of post-course emails")
    
    # 3. Create matching pairs for merging
    print("\nCreating matching pairs...")
    
    # 3.1 Email matches
    email_match_pairs = []
    for email in email_matches:
        # There could be multiple records with the same email in either dataset
        pre_matches = pre_df_valid_email[pre_df_valid_email['email_normalized'] == email]
        post_matches = post_df_valid_email[post_df_valid_email['email_normalized'] == email]
        
        # Create all possible pairs
        for pre_idx, pre_row in pre_matches.iterrows():
            for post_idx, post_row in post_matches.iterrows():
                email_match_pairs.append({
                    'pre_idx': pre_idx,
                    'post_idx': post_idx,
                    'match_type': 'email',
                    'pre_email': pre_row['original_email'],
                    'post_email': post_row['original_email'],
                    'pre_normalized': pre_row['email_normalized'],
                    'post_normalized': post_row['email_normalized'],
                    'pre_name': pre_row['full_name'] if 'full_name' in pre_row else '',
                    'post_name': post_row['full_name'] if 'full_name' in post_row else ''
                })
    
    # 3.2 Name+DOB matches
    name_dob_match_pairs = []
    for key in name_dob_matches:
        # Get matching records
        pre_matches = pre_valid_name_dob[pre_valid_name_dob['name_dob_key'] == key]
        post_matches = post_valid_name_dob[post_valid_name_dob['name_dob_key'] == key]
        
        # Create all possible pairs
        for pre_idx, pre_row in pre_matches.iterrows():
            for post_idx, post_row in post_matches.iterrows():
                name_dob_match_pairs.append({
                    'pre_idx': pre_idx,
                    'post_idx': post_idx,
                    'match_type': 'name_dob',
                    'pre_email': pre_row['original_email'],
                    'post_email': post_row['original_email'],
                    'pre_normalized': pre_row['email_normalized'] if not pd.isna(pre_row['email_normalized']) else '',
                    'post_normalized': post_row['email_normalized'] if not pd.isna(post_row['email_normalized']) else '',
                    'pre_name': pre_row['full_name'] if 'full_name' in pre_row else '',
                    'post_name': post_row['full_name'] if 'full_name' in post_row else '',
                    'match_key': key
                })
    
    # 3.3 First name + last name + birth year matches
    name_year_match_pairs = []
    for key in name_year_matches:
        # Get matching records
        pre_matches = pre_valid_name_year[pre_valid_name_year['name_year_key'] == key]
        post_matches = post_valid_name_year[post_valid_name_year['name_year_key'] == key]
        
        # Create all possible pairs
        for pre_idx, pre_row in pre_matches.iterrows():
            for post_idx, post_row in post_matches.iterrows():
                name_year_match_pairs.append({
                    'pre_idx': pre_idx,
                    'post_idx': post_idx,
                    'match_type': 'name_year',
                    'pre_email': pre_row['original_email'],
                    'post_email': post_row['original_email'],
                    'pre_normalized': pre_row['email_normalized'] if not pd.isna(pre_row['email_normalized']) else '',
                    'post_normalized': post_row['email_normalized'] if not pd.isna(post_row['email_normalized']) else '',
                    'pre_name': pre_row['full_name'] if 'full_name' in pre_row else '',
                    'post_name': post_row['full_name'] if 'full_name' in post_row else '',
                    'match_key': key
                })
    
    # Combine all match pairs
    all_match_pairs = email_match_pairs + name_dob_match_pairs + name_year_match_pairs
    
    # Create a DataFrame with all match pairs
    match_pairs_df = pd.DataFrame(all_match_pairs)
    
    # Count matches by type
    match_type_counts = match_pairs_df['match_type'].value_counts()
    
    stats['email_match_pairs'] = len(email_match_pairs)
    stats['name_dob_match_pairs'] = len(name_dob_match_pairs)
    stats['name_year_match_pairs'] = len(name_year_match_pairs)
    stats['total_match_pairs'] = len(all_match_pairs)
    
    print("\nMatches by type:")
    for match_type, count in match_type_counts.items():
        print(f"  {match_type}: {count} match pairs")
    
    # 4. Analyze domain corrections that led to successful matches
    print("\nAnalyzing domain corrections that led to successful matches...")
    
    # Find emails that matched due to domain correction
    corrected_matches = []
    for idx, row in match_pairs_df.iterrows():
        if row['match_type'] == 'email':
            pre_email = row['pre_email']
            post_email = row['post_email']
            pre_norm = row['pre_normalized']
            post_norm = row['post_normalized']
            
            # Check if the original emails were different but normalized ones match
            if pre_email != post_email and pre_norm == post_norm:
                corrected_matches.append({
                    'pre_email': pre_email,
                    'post_email': post_email,
                    'normalized_email': pre_norm,
                    'pre_domain': extract_domain(pre_email),
                    'post_domain': extract_domain(post_email),
                    'corrected_domain': extract_domain(pre_norm)
                })
    
    corrected_matches_df = pd.DataFrame(corrected_matches)
    stats['corrected_domain_matches'] = len(corrected_matches_df)
    
    print(f"Found {len(corrected_matches_df)} matches due to domain correction")
    
    # 5. Save match analysis
    print("\nSaving match analysis...")
    
    output_dir = os.path.dirname(pre_course_path)
    output_file = os.path.join(output_dir, "comprehensive_match_analysis.xlsx")
    
    # Create summary dataframe
    summary_data = [
        {'Category': 'Dataset Sizes', 'Metric': 'Total Pre-Enrollment Records', 'Value': stats['pre_total']},
        {'Category': 'Dataset Sizes', 'Metric': 'Total Post-Course Records', 'Value': stats['post_total']},
        
        {'Category': 'Email Standardization', 'Metric': 'Pre-Enrollment Records with Original Email', 'Value': stats['pre_valid_email_before']},
        {'Category': 'Email Standardization', 'Metric': 'Pre-Enrollment Records with Normalized Email', 'Value': stats['pre_valid_normalized_before']},
        {'Category': 'Email Standardization', 'Metric': 'Post-Course Records with Original Email', 'Value': stats['post_valid_email_before']},
        {'Category': 'Email Standardization', 'Metric': 'Post-Course Records with Normalized Email', 'Value': stats['post_valid_normalized_before']},
        
        {'Category': 'Domain Correction', 'Metric': 'Pre-Enrollment Domains Corrected', 'Value': stats['pre_domain_corrections_count']},
        {'Category': 'Domain Correction', 'Metric': 'Post-Course Domains Corrected', 'Value': stats['post_domain_corrections_count']},
        {'Category': 'Domain Correction', 'Metric': 'Matches Due to Domain Correction', 'Value': stats['corrected_domain_matches']},
        
        {'Category': 'Valid Identifiers', 'Metric': 'Pre-Enrollment Records with Valid Email', 'Value': stats['pre_valid_email_count']},
        {'Category': 'Valid Identifiers', 'Metric': 'Post-Course Records with Valid Email', 'Value': stats['post_valid_email_count']},
        {'Category': 'Valid Identifiers', 'Metric': 'Pre-Enrollment Unique Emails', 'Value': stats['pre_unique_email_count']},
        {'Category': 'Valid Identifiers', 'Metric': 'Post-Course Unique Emails', 'Value': stats['post_unique_email_count']},
        {'Category': 'Valid Identifiers', 'Metric': 'Pre-Enrollment Unmatched Records with Valid Name+DOB', 'Value': stats['pre_valid_name_dob_count']},
        {'Category': 'Valid Identifiers', 'Metric': 'Post-Course Unmatched Records with Valid Name+DOB', 'Value': stats['post_valid_name_dob_count']},
        {'Category': 'Valid Identifiers', 'Metric': 'Pre-Enrollment Remaining Records with Valid Name+Year', 'Value': stats['pre_valid_name_year_count']},
        {'Category': 'Valid Identifiers', 'Metric': 'Post-Course Remaining Records with Valid Name+Year', 'Value': stats['post_valid_name_year_count']},
        
        {'Category': 'Matches by Type', 'Metric': 'Matches by Email', 'Value': stats['email_match_count']},
        {'Category': 'Matches by Type', 'Metric': 'Additional Matches by Full Name + DOB', 'Value': stats['name_dob_match_count']},
        {'Category': 'Matches by Type', 'Metric': 'Additional Matches by Name + Birth Year', 'Value': stats['name_year_match_count']},
        {'Category': 'Matches by Type', 'Metric': 'Total Unique Matches', 'Value': stats['total_matches']},
        
        {'Category': 'Match Pairs', 'Metric': 'Email Match Pairs', 'Value': stats['email_match_pairs']},
        {'Category': 'Match Pairs', 'Metric': 'Name+DOB Match Pairs', 'Value': stats['name_dob_match_pairs']},
        {'Category': 'Match Pairs', 'Metric': 'Name+Year Match Pairs', 'Value': stats['name_year_match_pairs']},
        {'Category': 'Match Pairs', 'Metric': 'Total Match Pairs', 'Value': stats['total_match_pairs']},
        
        {'Category': 'Match Rates', 'Metric': 'Email Match Rate (% of Pre-Enrollment Emails)', 'Value': f"{email_match_count/pre_unique_email_count*100:.2f}%"},
        {'Category': 'Match Rates', 'Metric': 'Email Match Rate (% of Post-Course Emails)', 'Value': f"{email_match_count/post_unique_email_count*100:.2f}%"},
        {'Category': 'Match Rates', 'Metric': 'Overall Match Rate (% of Pre-Enrollment)', 'Value': f"{total_matches/stats['pre_unique_email_count']*100:.2f}%"},
        {'Category': 'Match Rates', 'Metric': 'Overall Match Rate (% of Post-Course)', 'Value': f"{total_matches/stats['post_unique_email_count']*100:.2f}%"}
    ]
    
    summary_df = pd.DataFrame(summary_data)
    
    # Save to Excel
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        summary_df.to_excel(writer, sheet_name='Match Summary', index=False)
        match_pairs_df.to_excel(writer, sheet_name='All Match Pairs', index=False)
        
        # Save domain correction analysis
        corrected_matches_df.to_excel(writer, sheet_name='Domain Correction Matches', index=False)
        
        # Sample of each match type
        for match_type in match_pairs_df['match_type'].unique():
            sample_df = match_pairs_df[match_pairs_df['match_type'] == match_type].head(500)
            sample_df.to_excel(writer, sheet_name=f'{match_type}_matches_sample', index=False)
        
        # Domain analysis
        # Original pre-enrollment domains
        pre_original_domains = pre_df['original_domain'].value_counts().reset_index()
        pre_original_domains.columns = ['Domain', 'Count']
        pre_original_domains['Percentage'] = pre_original_domains['Count'] / pre_original_domains['Count'].sum() * 100
        
        # Corrected pre-enrollment domains
        pre_corrected_domains = pre_df['corrected_domain'].value_counts().reset_index()
        pre_corrected_domains.columns = ['Domain', 'Count']
        pre_corrected_domains['Percentage'] = pre_corrected_domains['Count'] / pre_corrected_domains['Count'].sum() * 100
        
        # Original post-course domains
        post_original_domains = post_df['original_domain'].value_counts().reset_index()
        post_original_domains.columns = ['Domain', 'Count']
        post_original_domains['Percentage'] = post_original_domains['Count'] / post_original_domains['Count'].sum() * 100
        
        # Corrected post-course domains
        post_corrected_domains = post_df['corrected_domain'].value_counts().reset_index()
        post_corrected_domains.columns = ['Domain', 'Count']
        post_corrected_domains['Percentage'] = post_corrected_domains['Count'] / post_corrected_domains['Count'].sum() * 100
        
        # Save domain analysis
        pre_original_domains.to_excel(writer, sheet_name='Pre Original Domains', index=False)
        pre_corrected_domains.to_excel(writer, sheet_name='Pre Corrected Domains', index=False)
        post_original_domains.to_excel(writer, sheet_name='Post Original Domains', index=False)
        post_corrected_domains.to_excel(writer, sheet_name='Post Corrected Domains', index=False)
        
        # Domain correction details
        pre_domain_changes_df = pre_df[pre_df['original_domain'] != pre_df['corrected_domain']].dropna(subset=['original_domain', 'corrected_domain'])
        pre_domain_changes_df = pre_domain_changes_df[['original_email', 'email_normalized', 'original_domain', 'corrected_domain']]
        pre_domain_changes_df.to_excel(writer, sheet_name='Pre Domain Corrections', index=False)
        
        post_domain_changes_df = post_df[post_df['original_domain'] != post_df['corrected_domain']].dropna(subset=['original_domain', 'corrected_domain'])
        post_domain_changes_df = post_domain_changes_df[['original_email', 'email_normalized', 'original_domain', 'corrected_domain']]
        post_domain_changes_df.to_excel(writer, sheet_name='Post Domain Corrections', index=False)
    
    print(f"Saved comprehensive match analysis to: {output_file}")
    
    # Return results
    return {
        'stats': stats,
        'match_pairs_df': match_pairs_df,
        'corrected_matches_df': corrected_matches_df,
        'pre_df': pre_df,
        'post_df': post_df,
        'email_matches': email_matches,
        'name_dob_matches': name_dob_matches,
        'name_year_matches': name_year_matches
    }

def merge_survey_data(results, output_path=None, include_match_types=None):
    """
    Merge the pre-enrollment and post-course survey data based on matching records.
    
    Parameters:
    results (dict): Results from analyze_survey_matches
    output_path (str, optional): Path to save the merged data
    include_match_types (list, optional): Types of matches to include ('email', 'name_dob', 'name_year')
    """
    # If match types not specified, use all
    if include_match_types is None:
        include_match_types = ['email', 'name_dob', 'name_year']
    
    # Validate match types
    valid_match_types = ['email', 'name_dob', 'name_year']
    for match_type in include_match_types:
        if match_type not in valid_match_types:
            print(f"Warning: Invalid match type '{match_type}'. Ignoring.")
            include_match_types.remove(match_type)
    
    if not include_match_types:
        print("Error: No valid match types specified.")
        return None
    
    print(f"Merging datasets using match types: {include_match_types}")
    
    # Filter match pairs by type
    match_pairs_df = results['match_pairs_df']
    filtered_pairs = match_pairs_df[match_pairs_df['match_type'].isin(include_match_types)]
    
    if len(filtered_pairs) == 0:
        print("No matching pairs found with specified match types.")
        return None
    
    print(f"Using {len(filtered_pairs)} matching pairs for merging.")
    
    # Get original dataframes
    pre_df = results['pre_df'].copy()
    post_df = results['post_df'].copy()
    
    # Create a list of unique pre and post indices
    pre_indices = filtered_pairs['pre_idx'].unique()
    post_indices = filtered_pairs['post_idx'].unique()
    
    # Filter dataframes to only include matched records
    pre_matched = pre_df.loc[pre_indices].copy()
    post_matched = post_df.loc[post_indices].copy()
    
    print(f"Filtered to {len(pre_matched)} pre-enrollment records and {len(post_matched)} post-course records")
    
    # Create a mapping table for merging
    # This table will map each pre_idx to the corresponding post_idx
    # In case of multiple matches, we'll use the first one for each pre_idx
    mapping = {}
    for _, group in filtered_pairs.groupby('pre_idx'):
        # Use the first match for each pre_idx (prioritizing email matches if available)
        priority_order = {'email': 0, 'name_dob': 1, 'name_year': 2}
        group_sorted = group.sort_values(by='match_type', key=lambda x: x.map(priority_order))
        mapping[group_sorted.iloc[0]['pre_idx']] = group_sorted.iloc[0]['post_idx']
    
    # Create a new column in pre_matched with the corresponding post_idx
    pre_matched['matched_post_idx'] = pre_matched.index.map(lambda x: mapping.get(x, None))
    
    # Drop rows without a match
    pre_matched = pre_matched.dropna(subset=['matched_post_idx'])
    
    # Reset the index to make it a regular column
    pre_matched = pre_matched.reset_index()
    post_matched = post_matched.reset_index()
    
    # Rename columns to avoid conflicts
    pre_cols = pre_matched.columns
    post_cols = post_matched.columns
    
    pre_renamed = [col if col not in post_cols or col == 'index' else f"pre_{col}" for col in pre_cols]
    post_renamed = [col if col not in pre_cols or col == 'index' else f"post_{col}" for col in post_cols]
    
    pre_matched.columns = pre_renamed
    post_matched.columns = post_renamed
    
    # Merge the datasets
    merged_df = pd.merge(
        pre_matched,
        post_matched,
        left_on='matched_post_idx',
        right_on='index',
        how='inner'
    )
    
    # Clean up the merged dataset
    # Remove index and matched_post_idx columns
    merged_df = merged_df.drop(['matched_post_idx', 'index_x', 'index_y'], axis=1, errors='ignore')
    
    # Remove normalized and temporary columns
    columns_to_drop = [col for col in merged_df.columns if any(x in col for x in 
                     ['_normalized', '_norm', 'dob_year', 'dob_month', 'dob_day', 
                      'name_dob_key', 'name_year_key', 'original_domain', 'corrected_domain'])]
    merged_df = merged_df.drop(columns_to_drop, axis=1, errors='ignore')
    
    print(f"Merged dataset has {len(merged_df)} rows and {len(merged_df.columns)} columns")
    
    # Add a column indicating the match type
    match_types = {}
    for _, row in filtered_pairs.iterrows():
        if row['pre_idx'] in mapping and mapping[row['pre_idx']] == row['post_idx']:
            match_types[row['pre_idx']] = row['match_type']
    
    # Convert pre_idx back from string to original type if needed
    if 'pre_index' in merged_df.columns:
        merged_df['match_type'] = merged_df['pre_index'].map(match_types)
    
    # Save to Excel if output path provided
    if output_path:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        merged_df.to_excel(output_path, index=False)
        print(f"Merged data saved to: {output_path}")
        
        # Also save a version with match types separated
        output_dir = os.path.dirname(output_path)
        base_name = os.path.splitext(os.path.basename(output_path))[0]
        
        with pd.ExcelWriter(os.path.join(output_dir, f"{base_name}_by_match_type.xlsx"), engine='openpyxl') as writer:
            merged_df.to_excel(writer, sheet_name='All Matches', index=False)
            
            # Split by match type
            for match_type in include_match_types:
                if match_type in merged_df['match_type'].values:
                    type_df = merged_df[merged_df['match_type'] == match_type]
                    type_df.to_excel(writer, sheet_name=f'{match_type}_matches', index=False)
    
    return merged_df

if __name__ == "__main__":
    pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
    post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
    
    # Analyze matches with comprehensive domain correction and multiple identifiers
    results = analyze_survey_matches(pre_course_path, post_course_path)
    
    # If you want to merge the datasets, uncomment this section
    # By default, include all match types
    output_dir = os.path.dirname(pre_course_path)
    merged_output_path = os.path.join(output_dir, "Comprehensive_Merged_Surveys.xlsx")
    merged_df = merge_survey_data(results, merged_output_path)
    
    # Or, to only include email and name+DOB matches (excluding the less reliable name+year matches)
    # merged_df = merge_survey_data(results, merged_output_path, include_match_types=['email', 'name_dob'])

Loading pre-enrollment survey data...
Loading post-course survey data...
Pre-enrollment survey: 29700 entries
Post-course survey: 7806 entries
Aligning date of birth column names...
Standardizing data for matching...
Processing pre-enrollment emails...
Pre-enrollment emails before normalization: 24446
Pre-enrollment emails after normalization: 24433
Processing pre-enrollment dates of birth...
Processing post-course emails...
Post-course emails before normalization: 7746
Post-course emails after normalization: 5462
Processing post-course dates of birth...

Performing email matching...
Pre-enrollment records with valid emails: 24433 (82.27%)
Post-course records with valid emails: 5462 (69.97%)
Unique emails in pre-enrollment: 24368
Unique emails in post-course: 5456
Found 3226 matches by email
Email match rate: 13.24% of pre-enrollment emails
Email match rate: 59.13% of post-course emails

Domain corrections applied:
Pre-enrollment: 334 domains corrected
Post-course: 58 domains corrected

/var/folders/q6/v__5dqvn4fsgnld9cp_x0v1m0000gn/T/ipykernel_7198/52385201.py:401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_unmatched_email['name_dob_key'] = pre_unmatched_email.apply(create_name_dob_key, axis=1)
/var/folders/q6/v__5dqvn4fsgnld9cp_x0v1m0000gn/T/ipykernel_7198/52385201.py:402: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_unmatched_email['name_dob_key'] = post_unmatched_email.apply(create_name_dob_key, axis=1)


Pre-enrollment records with valid name+DOB (unmatched by email): 26364
Post-course records with valid name+DOB (unmatched by email): 4488
Found 2324 additional matches by full name + DOB

Performing first name + last name + birth year matching...


/var/folders/q6/v__5dqvn4fsgnld9cp_x0v1m0000gn/T/ipykernel_7198/52385201.py:449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_unmatched['name_year_key'] = pre_unmatched.apply(create_name_year_key, axis=1)
/var/folders/q6/v__5dqvn4fsgnld9cp_x0v1m0000gn/T/ipykernel_7198/52385201.py:450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  post_unmatched['name_year_key'] = post_unmatched.apply(create_name_year_key, axis=1)


Pre-enrollment records with valid name+year (still unmatched): 23885
Post-course records with valid name+year (still unmatched): 2139
Found 97 additional matches by first name + last name + birth year

Total matched records: 5647
Overall match rate: 23.17% of pre-enrollment emails
Overall match rate: 103.50% of post-course emails

Creating matching pairs...

Matches by type:
  email: 3247 match pairs
  name_dob: 2506 match pairs
  name_year: 100 match pairs

Analyzing domain corrections that led to successful matches...
Found 60 matches due to domain correction

Saving match analysis...
Saved comprehensive match analysis to: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/comprehensive_match_analysis.xlsx
Merging datasets using match types: ['email', 'name_dob', 'name_year']
Using 5853 matching pairs for merging.
Filtered to 5820 pre-enrollment records and 5676 post-course records
Merged dataset has 5820 rows and 229 columns
Merged data saved to: /Use

KeyError: 'match_type'

In [20]:
#merge data quality analysis
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_data_quality(merged_file_path):
    """
    Comprehensive analysis of data quality in the merged dataset
    
    Parameters:
    merged_file_path (str): Path to the merged Excel file
    """
    print(f"Analyzing data quality for: {merged_file_path}")
    
    # Load the merged data
    print("Loading data...")
    merged_df = pd.read_excel(merged_file_path)
    print(f"Loaded {len(merged_df)} records with {len(merged_df.columns)} columns")
    
    # Create output directory for reports
    output_dir = os.path.dirname(merged_file_path)
    analysis_dir = os.path.join(output_dir, "Data_Quality_Analysis")
    os.makedirs(analysis_dir, exist_ok=True)
    
    # Initialize report
    report_data = []
    
    # 1. Basic Dataset Information
    print("\n1. Basic Dataset Information")
    report_data.append({"Category": "Basic Info", "Metric": "Total Records", "Value": len(merged_df)})
    report_data.append({"Category": "Basic Info", "Metric": "Total Columns", "Value": len(merged_df.columns)})
    
    # Check for null values in key columns
    key_columns = [
        'pre_email_address', 'post_email_address', 
        'pre_full_name', 'post_full_name',
        'pre_date_of_birth', 'post_date_of_birth',
        'match_type'
    ]
    
    # Adjust column names if they don't exist exactly as above
    existing_columns = []
    for col in key_columns:
        if col in merged_df.columns:
            existing_columns.append(col)
        else:
            print(f"Warning: Column '{col}' not found in dataset")
    
    for col in existing_columns:
        null_count = merged_df[col].isna().sum()
        null_percent = (null_count / len(merged_df)) * 100
        report_data.append({"Category": "Missing Values", "Metric": f"Null in {col}", 
                           "Value": f"{null_count} ({null_percent:.2f}%)"})
        print(f"Column {col}: {null_count} null values ({null_percent:.2f}%)")
    
    # 2. Match Type Analysis
    print("\n2. Match Type Analysis")
    if 'match_type' in merged_df.columns:
        match_counts = merged_df['match_type'].value_counts()
        print("Match type distribution:")
        for match_type, count in match_counts.items():
            percent = (count / len(merged_df)) * 100
            report_data.append({"Category": "Match Types", "Metric": f"{match_type}", 
                               "Value": f"{count} ({percent:.2f}%)"})
            print(f"  {match_type}: {count} ({percent:.2f}%)")
    else:
        print("No match_type column found. Adding match type inference based on available data...")
        
        # Infer match type based on available data
        def infer_match_type(row):
            # Check if email addresses match
            if 'pre_email_address' in merged_df.columns and 'post_email_address' in merged_df.columns:
                if pd.notna(row['pre_email_address']) and pd.notna(row['post_email_address']):
                    if str(row['pre_email_address']).lower().strip() == str(row['post_email_address']).lower().strip():
                        return 'email'
            
            # Check if full names and DOB match
            if ('pre_full_name' in merged_df.columns and 'post_full_name' in merged_df.columns and
                'pre_date_of_birth' in merged_df.columns and 'post_date_of_birth' in merged_df.columns):
                if (pd.notna(row['pre_full_name']) and pd.notna(row['post_full_name']) and
                    pd.notna(row['pre_date_of_birth']) and pd.notna(row['post_date_of_birth'])):
                    if (str(row['pre_full_name']).lower().strip() == str(row['post_full_name']).lower().strip() and
                        str(row['pre_date_of_birth']) == str(row['post_date_of_birth'])):
                        return 'name_dob'
            
            # Check if first and last names and birth year match
            if ('pre_first_name' in merged_df.columns and 'post_first_name' in merged_df.columns and
                'pre_last_name' in merged_df.columns and 'post_last_name' in merged_df.columns and
                'pre_date_of_birth' in merged_df.columns and 'post_date_of_birth' in merged_df.columns):
                if (pd.notna(row['pre_first_name']) and pd.notna(row['post_first_name']) and
                    pd.notna(row['pre_last_name']) and pd.notna(row['post_last_name'])):
                    if (str(row['pre_first_name']).lower().strip() == str(row['post_first_name']).lower().strip() and
                        str(row['pre_last_name']).lower().strip() == str(row['post_last_name']).lower().strip()):
                        return 'name_year'
            
            return 'unknown'
        
        merged_df['inferred_match_type'] = merged_df.apply(infer_match_type, axis=1)
        
        inferred_counts = merged_df['inferred_match_type'].value_counts()
        print("Inferred match type distribution:")
        for match_type, count in inferred_counts.items():
            percent = (count / len(merged_df)) * 100
            report_data.append({"Category": "Inferred Match Types", "Metric": f"{match_type}", 
                               "Value": f"{count} ({percent:.2f}%)"})
            print(f"  {match_type}: {count} ({percent:.2f}%)")
    
    # 3. Email Quality Analysis
    print("\n3. Email Quality Analysis")
    email_cols = [col for col in merged_df.columns if 'email' in col.lower()]
    print(f"Found {len(email_cols)} email-related columns: {email_cols}")
    
    pre_email_col = None
    post_email_col = None
    
    # Identify the main email columns
    for col in email_cols:
        if 'pre' in col.lower() and 'address' in col.lower():
            pre_email_col = col
        elif 'post' in col.lower() and 'address' in col.lower():
            post_email_col = col
    
    if not pre_email_col:
        pre_email_col = 'pre_email_address' if 'pre_email_address' in merged_df.columns else 'pre_original_email'
    
    if not post_email_col:
        post_email_col = 'post_email_address' if 'post_email_address' in merged_df.columns else 'post_original_email'
    
    print(f"Using {pre_email_col} and {post_email_col} for email analysis")
    
    # Check for duplicate emails
    if pre_email_col in merged_df.columns:
        pre_email_counts = merged_df[pre_email_col].value_counts()
        pre_duplicates = pre_email_counts[pre_email_counts > 1]
        print(f"Found {len(pre_duplicates)} pre-enrollment emails with duplicates")
        report_data.append({"Category": "Duplicate Analysis", 
                           "Metric": "Pre-enrollment Emails with Duplicates", 
                           "Value": len(pre_duplicates)})
        
        if len(pre_duplicates) > 0:
            print("Top 10 pre-enrollment duplicated emails:")
            for email, count in pre_duplicates.head(10).items():
                if pd.notna(email):
                    print(f"  {email}: {count} occurrences")
            
            # Save duplicates to file
            pre_dup_df = pd.DataFrame({'Email': pre_duplicates.index, 'Count': pre_duplicates.values})
            pre_dup_df.to_excel(os.path.join(analysis_dir, "pre_duplicate_emails.xlsx"), index=False)
    
    if post_email_col in merged_df.columns:
        post_email_counts = merged_df[post_email_col].value_counts()
        post_duplicates = post_email_counts[post_email_counts > 1]
        print(f"Found {len(post_duplicates)} post-course emails with duplicates")
        report_data.append({"Category": "Duplicate Analysis", 
                           "Metric": "Post-course Emails with Duplicates", 
                           "Value": len(post_duplicates)})
        
        if len(post_duplicates) > 0:
            print("Top 10 post-course duplicated emails:")
            for email, count in post_duplicates.head(10).items():
                if pd.notna(email):
                    print(f"  {email}: {count} occurrences")
            
            # Save duplicates to file
            post_dup_df = pd.DataFrame({'Email': post_duplicates.index, 'Count': post_duplicates.values})
            post_dup_df.to_excel(os.path.join(analysis_dir, "post_duplicate_emails.xlsx"), index=False)
    
    # Analyze email format inconsistencies
    if pre_email_col in merged_df.columns and post_email_col in merged_df.columns:
        print("\nAnalyzing email format consistency between pre and post...")
        
        # Define a function to extract domain from email
        def extract_domain(email):
            if pd.isna(email):
                return None
            try:
                return str(email).lower().split('@')[1]
            except (IndexError, AttributeError):
                return None
        
        # Add domain columns
        merged_df['pre_email_domain'] = merged_df[pre_email_col].apply(extract_domain)
        merged_df['post_email_domain'] = merged_df[post_email_col].apply(extract_domain)
        
        # Count records where domains don't match
        domain_mismatch = merged_df[(merged_df['pre_email_domain'].notna()) & 
                                    (merged_df['post_email_domain'].notna()) & 
                                    (merged_df['pre_email_domain'] != merged_df['post_email_domain'])]
        
        print(f"Found {len(domain_mismatch)} records where pre and post email domains don't match")
        report_data.append({"Category": "Email Consistency", 
                           "Metric": "Records with Different Email Domains", 
                           "Value": len(domain_mismatch)})
        
        if len(domain_mismatch) > 0:
            print("Sample of domain mismatches:")
            sample = domain_mismatch.head(10)
            for _, row in sample.iterrows():
                print(f"  {row[pre_email_col]} -> {row[post_email_col]}")
            
            # Save domain mismatches to file
            domain_mismatch[['pre_email_domain', 'post_email_domain', pre_email_col, post_email_col]].to_excel(
                os.path.join(analysis_dir, "email_domain_mismatches.xlsx"), index=False)
    
    # 4. Name and DOB Consistency Analysis
    print("\n4. Name and DOB Consistency Analysis")
    
    # Check name consistency
    if 'pre_full_name' in merged_df.columns and 'post_full_name' in merged_df.columns:
        # Normalize names for comparison
        def normalize_name(name):
            if pd.isna(name):
                return None
            return re.sub(r'[^a-zA-Z ]', '', str(name).lower()).strip()
        
        merged_df['pre_name_norm'] = merged_df['pre_full_name'].apply(normalize_name)
        merged_df['post_name_norm'] = merged_df['post_full_name'].apply(normalize_name)
        
        # Count exact matches and near matches
        exact_name_matches = sum(merged_df['pre_name_norm'] == merged_df['post_name_norm'])
        exact_match_pct = (exact_name_matches / len(merged_df)) * 100
        
        print(f"Exact name matches: {exact_name_matches} ({exact_match_pct:.2f}%)")
        report_data.append({"Category": "Name Consistency", 
                           "Metric": "Exact Name Matches", 
                           "Value": f"{exact_name_matches} ({exact_match_pct:.2f}%)"})
        
        # Analyze name differences
        name_diff_df = merged_df[merged_df['pre_name_norm'] != merged_df['post_name_norm']].copy()
        
        if len(name_diff_df) > 0:
            print(f"Found {len(name_diff_df)} records with name differences")
            
            # Categorize name differences
            def categorize_name_diff(pre, post):
                if pd.isna(pre) or pd.isna(post):
                    return "Missing Name"
                
                pre_parts = pre.split()
                post_parts = post.split()
                
                if len(pre_parts) != len(post_parts):
                    return "Different Number of Name Parts"
                
                if len(pre_parts) == 1 or len(post_parts) == 1:
                    return "Single Name Part"
                
                # Check for first name match only
                if pre_parts[0] == post_parts[0] and pre_parts[1:] != post_parts[1:]:
                    return "First Name Match Only"
                
                # Check for last name match only
                if pre_parts[-1] == post_parts[-1] and pre_parts[:-1] != post_parts[:-1]:
                    return "Last Name Match Only"
                
                # Check for swapped first/last
                if len(pre_parts) >= 2 and len(post_parts) >= 2:
                    if pre_parts[0] == post_parts[-1] and pre_parts[-1] == post_parts[0]:
                        return "First/Last Swapped"
                
                return "Other Difference"
            
            name_diff_df['name_diff_type'] = name_diff_df.apply(
                lambda row: categorize_name_diff(row['pre_name_norm'], row['post_name_norm']), axis=1)
            
            diff_counts = name_diff_df['name_diff_type'].value_counts()
            print("Name difference categories:")
            for diff_type, count in diff_counts.items():
                pct = (count / len(name_diff_df)) * 100
                print(f"  {diff_type}: {count} ({pct:.2f}%)")
                report_data.append({"Category": "Name Differences", 
                                   "Metric": diff_type, 
                                   "Value": f"{count} ({pct:.2f}%)"})
            
            # Save name differences to file
            name_cols = [col for col in merged_df.columns if 'name' in col.lower()]
            name_diff_df[name_cols + ['name_diff_type']].to_excel(
                os.path.join(analysis_dir, "name_differences.xlsx"), index=False)
    
    # Check DOB consistency
    if 'pre_date_of_birth' in merged_df.columns and 'post_date_of_birth' in merged_df.columns:
        # Convert dates to string format for comparison
        merged_df['pre_dob_str'] = merged_df['pre_date_of_birth'].astype(str)
        merged_df['post_dob_str'] = merged_df['post_date_of_birth'].astype(str)
        
        # Count exact DOB matches
        exact_dob_matches = sum(merged_df['pre_dob_str'] == merged_df['post_dob_str'])
        exact_dob_pct = (exact_dob_matches / len(merged_df)) * 100
        
        print(f"Exact DOB matches: {exact_dob_matches} ({exact_dob_pct:.2f}%)")
        report_data.append({"Category": "DOB Consistency", 
                           "Metric": "Exact DOB Matches", 
                           "Value": f"{exact_dob_matches} ({exact_dob_pct:.2f}%)"})
        
        # Analyze DOB differences
        dob_diff_df = merged_df[merged_df['pre_dob_str'] != merged_df['post_dob_str']].copy()
        
        if len(dob_diff_df) > 0:
            print(f"Found {len(dob_diff_df)} records with DOB differences")
            
            # Try to parse DOBs to detect patterns
            def parse_dob(dob_str):
                if pd.isna(dob_str):
                    return None, None, None
                
                # Try multiple date formats
                formats = ['%Y-%m-%d', '%m/%d/%Y', '%m-%d-%Y', '%d/%m/%Y', '%d-%m-%Y']
                
                for fmt in formats:
                    try:
                        dt = pd.to_datetime(dob_str, format=fmt)
                        return dt.year, dt.month, dt.day
                    except:
                        continue
                
                return None, None, None
            
            # Extract year, month, day components
            dob_diff_df['pre_year'], dob_diff_df['pre_month'], dob_diff_df['pre_day'] = zip(
                *dob_diff_df['pre_dob_str'].apply(parse_dob))
            
            dob_diff_df['post_year'], dob_diff_df['post_month'], dob_diff_df['post_day'] = zip(
                *dob_diff_df['post_dob_str'].apply(parse_dob))
            
            # Categorize differences
            year_match = sum((dob_diff_df['pre_year'] == dob_diff_df['post_year']) & 
                             (dob_diff_df['pre_year'].notna()))
            
            month_day_swapped = sum((dob_diff_df['pre_month'] == dob_diff_df['post_day']) & 
                                   (dob_diff_df['pre_day'] == dob_diff_df['post_month']) &
                                   (dob_diff_df['pre_month'].notna()))
            
            print(f"  Same year, different month/day: {year_match} records")
            print(f"  Month/day swapped: {month_day_swapped} records")
            
            report_data.append({"Category": "DOB Differences", 
                               "Metric": "Same Year, Different Month/Day", 
                               "Value": year_match})
            
            report_data.append({"Category": "DOB Differences", 
                               "Metric": "Month/Day Swapped", 
                               "Value": month_day_swapped})
            
            # Save DOB differences to file
            dob_cols = [col for col in merged_df.columns if 'birth' in col.lower() or 'dob' in col.lower()]
            dob_analysis_cols = ['pre_dob_str', 'post_dob_str', 'pre_year', 'pre_month', 'pre_day',
                               'post_year', 'post_month', 'post_day']
            
            dob_diff_df[dob_cols + dob_analysis_cols].to_excel(
                os.path.join(analysis_dir, "dob_differences.xlsx"), index=False)
    
    # 5. Match Quality Analysis - Confidence Scoring
    print("\n5. Match Quality Analysis")
    
    # Create a confidence score for each match
    def calculate_match_confidence(row):
        score = 0
        max_score = 0
        
        # Email match (highest confidence)
        if pre_email_col in merged_df.columns and post_email_col in merged_df.columns:
            max_score += 5
            if pd.notna(row[pre_email_col]) and pd.notna(row[post_email_col]):
                pre_email = str(row[pre_email_col]).lower().strip()
                post_email = str(row[post_email_col]).lower().strip()
                
                if pre_email == post_email:
                    score += 5  # Exact match
                elif pre_email.split('@')[0] == post_email.split('@')[0]:
                    score += 4  # Username matches but different domain
        
        # Name match
        if 'pre_name_norm' in merged_df.columns and 'post_name_norm' in merged_df.columns:
            max_score += 3
            if pd.notna(row['pre_name_norm']) and pd.notna(row['post_name_norm']):
                if row['pre_name_norm'] == row['post_name_norm']:
                    score += 3  # Exact match
                elif 'pre_first_name' in merged_df.columns and 'post_first_name' in merged_df.columns:
                    # First name match
                    pre_first = normalize_name(row['pre_first_name'])
                    post_first = normalize_name(row['post_first_name'])
                    if pd.notna(pre_first) and pd.notna(post_first) and pre_first == post_first:
                        score += 1.5
                    
                    # Last name match
                    if 'pre_last_name' in merged_df.columns and 'post_last_name' in merged_df.columns:
                        pre_last = normalize_name(row['pre_last_name'])
                        post_last = normalize_name(row['post_last_name'])
                        if pd.notna(pre_last) and pd.notna(post_last) and pre_last == post_last:
                            score += 1.5
        
        # DOB match
        if 'pre_dob_str' in merged_df.columns and 'post_dob_str' in merged_df.columns:
            max_score += 3
            if pd.notna(row['pre_dob_str']) and pd.notna(row['post_dob_str']):
                if row['pre_dob_str'] == row['post_dob_str']:
                    score += 3  # Exact match
                elif hasattr(row, 'pre_year') and hasattr(row, 'post_year'):
                    # Year match
                    if pd.notna(row['pre_year']) and pd.notna(row['post_year']) and row['pre_year'] == row['post_year']:
                        score += 1.5
                    
                    # Month match
                    if pd.notna(row['pre_month']) and pd.notna(row['post_month']) and row['pre_month'] == row['post_month']:
                        score += 0.75
                    
                    # Day match
                    if pd.notna(row['pre_day']) and pd.notna(row['post_day']) and row['pre_day'] == row['post_day']:
                        score += 0.75
        
        # Calculate percentage
        confidence = (score / max_score * 100) if max_score > 0 else 0
        
        # Round to 2 decimal places
        return round(confidence, 2)
    
    # Add confidence score
    merged_df['match_confidence'] = merged_df.apply(calculate_match_confidence, axis=1)
    
    # Analyze confidence distribution
    confidence_bins = [0, 50, 70, 80, 90, 95, 100]
    merged_df['confidence_category'] = pd.cut(merged_df['match_confidence'], confidence_bins, 
                                             labels=['Very Low', 'Low', 'Moderate', 'Good', 'High', 'Perfect'])
    
    conf_counts = merged_df['confidence_category'].value_counts().sort_index()
    
    print("Match confidence distribution:")
    for category, count in conf_counts.items():
        pct = (count / len(merged_df)) * 100
        print(f"  {category}: {count} ({pct:.2f}%)")
        report_data.append({"Category": "Match Confidence", 
                           "Metric": str(category), 
                           "Value": f"{count} ({pct:.2f}%)"})
    
    # Export low confidence matches for review
    low_conf = merged_df[merged_df['match_confidence'] < 70].copy()
    if len(low_conf) > 0:
        print(f"Found {len(low_conf)} potentially problematic matches with confidence < 70%")
        report_data.append({"Category": "Data Quality Concerns", 
                           "Metric": "Low Confidence Matches (<70%)", 
                           "Value": len(low_conf)})
        
        # Select relevant columns for review
        review_cols = [
            pre_email_col, post_email_col,
            'pre_full_name', 'post_full_name',
            'pre_first_name', 'post_first_name',
            'pre_last_name', 'post_last_name',
            'pre_date_of_birth', 'post_date_of_birth',
            'match_confidence', 'confidence_category'
        ]
        
        # Filter to only include columns that exist
        review_cols = [col for col in review_cols if col in merged_df.columns]
        
        # Add match type if it exists
        if 'match_type' in merged_df.columns:
            review_cols.append('match_type')
        elif 'inferred_match_type' in merged_df.columns:
            review_cols.append('inferred_match_type')
        
        low_conf[review_cols].to_excel(
            os.path.join(analysis_dir, "low_confidence_matches.xlsx"), index=False)
    
    # 6. Additional Data Quality Checks
    print("\n6. Additional Data Quality Checks")
    
    # Check for outliers in age_at_course
    if 'age_at_course' in merged_df.columns:
        age_stats = merged_df['age_at_course'].describe()
        
        print("Age at course statistics:")
        print(f"  Min: {age_stats['min']}")
        print(f"  Max: {age_stats['max']}")
        print(f"  Mean: {age_stats['mean']:.1f}")
        
        # Identify potentially incorrect ages
        young_outliers = merged_df[merged_df['age_at_course'] < 5]
        old_outliers = merged_df[merged_df['age_at_course'] > 90]
        
        if len(young_outliers) > 0:
            print(f"  Found {len(young_outliers)} records with age < 5")
            report_data.append({"Category": "Data Quality Concerns", 
                               "Metric": "Very Young Age (<5)", 
                               "Value": len(young_outliers)})
        
        if len(old_outliers) > 0:
            print(f"  Found {len(old_outliers)} records with age > 90")
            report_data.append({"Category": "Data Quality Concerns", 
                               "Metric": "Very Old Age (>90)", 
                               "Value": len(old_outliers)})
        
        # Save age outliers to file
        if len(young_outliers) > 0 or len(old_outliers) > 0:
            age_outliers = pd.concat([young_outliers, old_outliers])
            
            age_cols = ['pre_full_name', 'post_full_name', 'pre_date_of_birth', 'post_date_of_birth', 
                      'age_at_course', 'pre_course_year', 'post_course_year']
            
            age_cols = [col for col in age_cols if col in merged_df.columns]
            
            age_outliers[age_cols].to_excel(
                os.path.join(analysis_dir, "age_outliers.xlsx"), index=False)
    
    # Check for inconsistencies in course years
    if 'pre_course_year' in merged_df.columns and 'post_course_year' in merged_df.columns:
        year_diff = merged_df['post_course_year'] - merged_df['pre_course_year']
        
        future_pre = merged_df[year_diff < 0]
        if len(future_pre) > 0:
            print(f"Found {len(future_pre)} records where pre-course year is after post-course year")
            report_data.append({"Category": "Data Quality Concerns", 
                               "Metric": "Pre-course Year After Post-course Year", 
                               "Value": len(future_pre)})
            
            year_cols = ['pre_course_year', 'post_course_year', 'pre_course_date', 'post_course_date']
            year_cols = [col for col in year_cols if col in merged_df.columns]
            
            future_pre[year_cols].to_excel(
                os.path.join(analysis_dir, "year_inconsistencies.xlsx"), index=False)
    
    # 7. Generate Summary Report
    print("\n7. Generating Summary Report")
    
    # Create a DataFrame of all reported metrics
    report_df = pd.DataFrame(report_data)
    
    # Save the full report
    report_df.to_excel(os.path.join(analysis_dir, "data_quality_report.xlsx"), index=False)
    
    print(f"Data quality analysis complete. Reports saved to: {analysis_dir}")
    
    # Return dataset with quality analysis columns
    return merged_df

# Run the analysis with your file
merged_file_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Comprehensive_Merged_Surveys.xlsx'
analyzed_df = analyze_data_quality(merged_file_path)
def visualize_data_quality(analyzed_df, analysis_dir):
    """
    Generate visualizations of data quality metrics
    
    Parameters:
    analyzed_df (DataFrame): The dataset with quality analysis columns added
    analysis_dir (str): Directory to save visualizations
    """
    print("Generating data quality visualizations...")
    
    # Create visualizations directory
    viz_dir = os.path.join(analysis_dir, "Visualizations")
    os.makedirs(viz_dir, exist_ok=True)
    
    # Set style
    plt.style.use('ggplot')
    
    # 1. Match Type Distribution
    if 'match_type' in analyzed_df.columns:
        match_type_col = 'match_type'
    elif 'inferred_match_type' in analyzed_df.columns:
        match_type_col = 'inferred_match_type'
    else:
        match_type_col = None
    
    if match_type_col:
        plt.figure(figsize=(10, 6))
        match_counts = analyzed_df[match_type_col].value_counts()
        
        # Plot
        ax = sns.barplot(x=match_counts.index, y=match_counts.values)
        
        # Add count labels
        for i, count in enumerate(match_counts.values):
            ax.text(i, count + 50, f"{count}", ha='center')
        
        plt.title('Distribution of Match Types', fontsize=15)
        plt.xlabel('Match Type', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, "match_type_distribution.png"), dpi=300)
        plt.close()
    
    # 2. Match Confidence Distribution
    if 'confidence_category' in analyzed_df.columns:
        plt.figure(figsize=(12, 6))
        conf_counts = analyzed_df['confidence_category'].value_counts().sort_index()
        
        # Custom color map based on confidence level
        colors = ['#d53e4f', '#fc8d59', '#fee08b', '#e6f598', '#99d594', '#3288bd']
        
        # Plot
        ax = sns.barplot(x=conf_counts.index, y=conf_counts.values, palette=colors)
        
        # Add count labels
        for i, count in enumerate(conf_counts.values):
            ax.text(i, count + 50, f"{count}", ha='center')
        
        plt.title('Distribution of Match Confidence', fontsize=15)
        plt.xlabel('Confidence Level', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, "match_confidence_distribution.png"), dpi=300)
        plt.close()
    
    # 3. Email Consistency Analysis
    email_consistency_data = []
    
    # Pre-enrollment email duplicates
    pre_email_col = [col for col in analyzed_df.columns if 'pre' in col.lower() and 'email' in col.lower() and 'address' in col.lower()]
    if pre_email_col:
        pre_email_col = pre_email_col[0]
        pre_dup_count = sum(analyzed_df[pre_email_col].duplicated())
        email_consistency_data.append({'Category': 'Pre-enrollment Email Duplicates', 'Count': pre_dup_count})
    
    # Post-course email duplicates
    post_email_col = [col for col in analyzed_df.columns if 'post' in col.lower() and 'email' in col.lower() and 'address' in col.lower()]
    if post_email_col:
        post_email_col = post_email_col[0]
        post_dup_count = sum(analyzed_df[post_email_col].duplicated())
        email_consistency_data.append({'Category': 'Post-course Email Duplicates', 'Count': post_dup_count})
    
    # Domain mismatches
    if 'pre_email_domain' in analyzed_df.columns and 'post_email_domain' in analyzed_df.columns:
        domain_mismatch_count = sum((analyzed_df['pre_email_domain'].notna()) & 
                                   (analyzed_df['post_email_domain'].notna()) & 
                                   (analyzed_df['pre_email_domain'] != analyzed_df['post_email_domain']))
        email_consistency_data.append({'Category': 'Email Domain Mismatches', 'Count': domain_mismatch_count})
    
    if email_consistency_data:
        email_df = pd.DataFrame(email_consistency_data)
        
        plt.figure(figsize=(10, 6))
        ax = sns.barplot(x='Category', y='Count', data=email_df)
        
        # Add count labels
        for i, count in enumerate(email_df['Count']):
            ax.text(i, count + 5, f"{count}", ha='center')
        
        plt.title('Email Consistency Issues', fontsize=15)
        plt.xlabel('Issue Type', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, "email_consistency_issues.png"), dpi=300)
        plt.close()
    
    # 4. Name and DOB Consistency
    name_dob_data = []
    
    # Name consistency
    if 'pre_name_norm' in analyzed_df.columns and 'post_name_norm' in analyzed_df.columns:
        exact_name_matches = sum(analyzed_df['pre_name_norm'] == analyzed_df['post_name_norm'])
        name_mismatches = len(analyzed_df) - exact_name_matches
        
        name_dob_data.append({'Category': 'Exact Name Matches', 'Count': exact_name_matches})
        name_dob_data.append({'Category': 'Name Mismatches', 'Count': name_mismatches})
    
    # DOB consistency
    if 'pre_dob_str' in analyzed_df.columns and 'post_dob_str' in analyzed_df.columns:
        exact_dob_matches = sum(analyzed_df['pre_dob_str'] == analyzed_df['post_dob_str'])
        dob_mismatches = len(analyzed_df) - exact_dob_matches
        
        name_dob_data.append({'Category': 'Exact DOB Matches', 'Count': exact_dob_matches})
        name_dob_data.append({'Category': 'DOB Mismatches', 'Count': dob_mismatches})
    
    if name_dob_data:
        name_dob_df = pd.DataFrame(name_dob_data)
        
        plt.figure(figsize=(10, 6))
        ax = sns.barplot(x='Category', y='Count', data=name_dob_df)
        
        # Add count labels
        for i, count in enumerate(name_dob_df['Count']):
            ax.text(i, count + 100, f"{count}", ha='center')
        
        plt.title('Name and DOB Consistency', fontsize=15)
        plt.xlabel('Category', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, "name_dob_consistency.png"), dpi=300)
        plt.close()
    
    # 5. Data Quality Concerns
    quality_concerns = []
    
    # Low confidence matches
    if 'match_confidence' in analyzed_df.columns:
        low_conf_count = sum(analyzed_df['match_confidence'] < 70)
        quality_concerns.append({'Concern': 'Low Confidence Matches', 'Count': low_conf_count})
    
    # Age outliers
    if 'age_at_course' in analyzed_df.columns:
        young_outliers = sum(analyzed_df['age_at_course'] < 5)
        old_outliers = sum(analyzed_df['age_at_course'] > 90)
        
        if young_outliers > 0:
            quality_concerns.append({'Concern': 'Very Young Age (<5)', 'Count': young_outliers})
        
        if old_outliers > 0:
            quality_concerns.append({'Concern': 'Very Old Age (>90)', 'Count': old_outliers})
    
    # Course year inconsistencies
    if 'pre_course_year' in analyzed_df.columns and 'post_course_year' in analyzed_df.columns:
        year_issues = sum(analyzed_df['post_course_year'] < analyzed_df['pre_course_year'])
        if year_issues > 0:
            quality_concerns.append({'Concern': 'Year Inconsistencies', 'Count': year_issues})
    
    if quality_concerns:
        concerns_df = pd.DataFrame(quality_concerns)
        
        plt.figure(figsize=(12, 6))
        ax = sns.barplot(x='Concern', y='Count', data=concerns_df, color='tomato')
        
        # Add count labels
        for i, count in enumerate(concerns_df['Count']):
            ax.text(i, count + 5, f"{count}", ha='center')
        
        plt.title('Data Quality Concerns', fontsize=15)
        plt.xlabel('Issue Type', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, "data_quality_concerns.png"), dpi=300)
        plt.close()
    
    print(f"Visualizations saved to: {viz_dir}")

# Update the analyze_data_quality function to call the visualization function
def analyze_data_quality(merged_file_path):
    # ... [all existing code] ...
    
    # At the end, add:
    # Generate visualizations
    visualize_data_quality(merged_df, analysis_dir)
    
    print(f"Data quality analysis complete. Reports and visualizations saved to: {analysis_dir}")
    
    # Return dataset with quality analysis columns
    return merged_df
def generate_recommendations(analyzed_df, analysis_dir):
    """
    Generate recommendations for data quality improvements
    
    Parameters:
    analyzed_df (DataFrame): The dataset with quality analysis columns added
    analysis_dir (str): Directory to save recommendations
    """
    print("\nGenerating recommendations based on data quality analysis...")
    
    recommendations = []
    
    # 1. Low confidence matches
    if 'match_confidence' in analyzed_df.columns:
        low_conf_count = sum(analyzed_df['match_confidence'] < 70)
        if low_conf_count > 0:
            low_conf_pct = (low_conf_count / len(analyzed_df)) * 100
            
            if low_conf_pct > 20:
                recommendations.append({
                    'Issue': 'High percentage of low confidence matches',
                    'Impact': 'HIGH',
                    'Recommendation': 'Consider reviewing and potentially excluding matches with confidence < 70%. These matches could significantly affect data analysis reliability.',
                    'Action Items': [
                        'Review low_confidence_matches.xlsx from the analysis',
                        'Consider creating a filtered dataset that excludes matches below a threshold',
                        'If keeping low confidence matches, flag them in analyses'
                    ]
                })
            else:
                recommendations.append({
                    'Issue': 'Some low confidence matches detected',
                    'Impact': 'MEDIUM',
                    'Recommendation': 'Review low confidence matches but they may not significantly impact overall analysis.',
                    'Action Items': [
                        'Review low_confidence_matches.xlsx from the analysis',
                        'Consider flagging these records in your analyses'
                    ]
                })
    
    # 2. Email duplicates
    pre_email_col = [col for col in analyzed_df.columns if 'pre' in col.lower() and 'email' in col.lower() and 'address' in col.lower()]
    if pre_email_col:
        pre_email_col = pre_email_col[0]
        pre_dup_count = sum(analyzed_df[pre_email_col].duplicated())
        
        if pre_dup_count > 0:
            pre_dup_pct = (pre_dup_count / len(analyzed_df)) * 100
            
            if pre_dup_pct > 10:
                recommendations.append({
                    'Issue': 'Significant number of duplicate pre-enrollment emails',
                    'Impact': 'HIGH',
                    'Recommendation': 'Duplicate emails suggest the same person may appear multiple times in the dataset.',
                    'Action Items': [
                        'Review pre_duplicate_emails.xlsx from the analysis',
                        'Consider consolidating records for the same person',
                        'Investigate why duplicates exist (multiple enrollments? data entry errors?)'
                    ]
                })
            else:
                recommendations.append({
                    'Issue': 'Some duplicate pre-enrollment emails',
                    'Impact': 'MEDIUM',
                    'Recommendation': 'Some duplication exists but likely won\'t significantly impact analysis.',
                    'Action Items': [
                        'Review pre_duplicate_emails.xlsx from the analysis',
                        'Flag duplicate records in your analyses'
                    ]
                })
    
    # 3. Name and DOB inconsistencies
    if 'pre_name_norm' in analyzed_df.columns and 'post_name_norm' in analyzed_df.columns:
        name_mismatch_count = sum(analyzed_df['pre_name_norm'] != analyzed_df['post_name_norm'])
        name_mismatch_pct = (name_mismatch_count / len(analyzed_df)) * 100
        
        if name_mismatch_pct > 30:
            recommendations.append({
                'Issue': 'High percentage of name mismatches',
                'Impact': 'HIGH',
                'Recommendation': 'Many records show different names between pre and post surveys, raising concerns about match accuracy.',
                'Action Items': [
                    'Review name_differences.xlsx from the analysis',
                    'Consider prioritizing email matches over name-based matches',
                    'Investigate patterns in name differences (nicknames, maiden/married names, etc.)'
                ]
            })
        elif name_mismatch_pct > 10:
            recommendations.append({
                'Issue': 'Moderate percentage of name mismatches',
                'Impact': 'MEDIUM',
                'Recommendation': 'Some name inconsistencies exist and should be reviewed.',
                'Action Items': [
                    'Review name_differences.xlsx from the analysis',
                    'Consider if differences follow patterns that could be standardized'
                ]
            })
    
    # 4. Overall data quality recommendation
    if 'match_confidence' in analyzed_df.columns:
        high_conf_count = sum(analyzed_df['match_confidence'] >= 90)
        high_conf_pct = (high_conf_count / len(analyzed_df)) * 100
        
        if high_conf_pct > 80:
            recommendations.append({
                'Issue': 'Overall data quality',
                'Impact': 'POSITIVE',
                'Recommendation': 'Data quality is generally good with most matches having high confidence.',
                'Action Items': [
                    'Proceed with analysis using the current merged dataset',
                    'Consider excluding or flagging only the most problematic matches'
                ]
            })
        elif high_conf_pct > 50:
            recommendations.append({
                'Issue': 'Overall data quality',
                'Impact': 'NEUTRAL',
                'Recommendation': 'Data quality is acceptable but has room for improvement.',
                'Action Items': [
                    'Consider creating a "gold standard" subset of high-confidence matches for key analyses',
                    'Use the full dataset for less sensitive analyses'
                ]
            })
        else:
            recommendations.append({
                'Issue': 'Overall data quality',
                'Impact': 'CONCERN',
                'Recommendation': 'Data quality issues may significantly impact analysis reliability.',
                'Action Items': [
                    'Consider redoing the matching process with stricter criteria',
                    'For now, work primarily with high-confidence matches only',
                    'Flag data quality issues in any reports or analyses'
                ]
            })
    
    # Save recommendations to Excel
    recommendations_df = pd.DataFrame(recommendations)
    recommendations_df.to_excel(os.path.join(analysis_dir, "data_quality_recommendations.xlsx"), index=False)
    
    # Also print summary of recommendations
    print("\nKey recommendations:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec['Issue']} - Impact: {rec['Impact']}")
        print(f"   {rec['Recommendation']}")
    
    print(f"\nDetailed recommendations saved to: {os.path.join(analysis_dir, 'data_quality_recommendations.xlsx')}")

# Update analyze_data_quality to include recommendations
def analyze_data_quality(merged_file_path):
    # ... [all existing code] ...
    
    # At the end, add:
    # Generate visualizations
    visualize_data_quality(merged_df, analysis_dir)
    
    # Generate recommendations
    generate_recommendations(merged_df, analysis_dir)
    
    print(f"Data quality analysis complete. Reports, visualizations, and recommendations saved to: {analysis_dir}")
    
    # Return dataset with quality analysis columns
    return merged_df

Analyzing data quality for: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Comprehensive_Merged_Surveys.xlsx
Loading data...
Loaded 5820 records with 229 columns

1. Basic Dataset Information
Column pre_email_address: 738 null values (12.68%)
Column post_email_address: 0 null values (0.00%)
Column pre_full_name: 0 null values (0.00%)
Column post_full_name: 0 null values (0.00%)
Column pre_date_of_birth: 0 null values (0.00%)
Column post_date_of_birth: 0 null values (0.00%)

2. Match Type Analysis
No match_type column found. Adding match type inference based on available data...
Inferred match type distribution:
  email: 3188 (54.78%)
  name_dob: 2476 (42.54%)
  name_year: 124 (2.13%)
  unknown: 32 (0.55%)

3. Email Quality Analysis
Found 14 email-related columns: ['pre_email_address', 'post_email_address', 'pre_creating_and_sending_emails', 'pre_opening_and_replying_to_emails', 'downloading_attachments_i_receive_in_an_email_documents_or_pictures', 'a

In [22]:
#High Quality Merged Data
import pandas as pd
import os
import re
from datetime import datetime
import numpy as np

def create_high_quality_merged_dataset(pre_course_path, post_course_path):
    """
    Create a high-quality merged dataset directly from pre-enrollment and post-course data,
    using strict matching criteria (email or full name + DOB).
    
    Parameters:
    pre_course_path (str): Path to the pre-enrollment Excel file
    post_course_path (str): Path to the post-course Excel file
    """
    print("Creating high-quality merged dataset from original files")
    
    # Step 1: Load the datasets
    print("\n1. Loading datasets")
    print(f"Loading pre-enrollment data from: {pre_course_path}")
    pre_df = pd.read_excel(pre_course_path)
    print(f"Loaded {len(pre_df)} pre-enrollment records")
    
    print(f"Loading post-course data from: {post_course_path}")
    post_df = pd.read_excel(post_course_path)
    print(f"Loaded {len(post_df)} post-course records")
    
    # Create output directory and file path
    output_dir = os.path.dirname(pre_course_path)
    output_file = os.path.join(output_dir, "Direct_High_Quality_Merged_Dataset.xlsx")
    
    # Step 2: Standardize and clean data for matching
    print("\n2. Standardizing and cleaning data")
    
    # Domain corrections dictionary
    domain_corrections = {
        # Gmail variations
        'gmai.com': 'gmail.com',
        'gmil.com': 'gmail.com',
        'gmal.com': 'gmail.com',
        'gamail.com': 'gmail.com',
        'gimail.com': 'gmail.com',
        'gmaill.com': 'gmail.com',
        'gmail.comm': 'gmail.com',
        'gmial.com': 'gmail.com',
        'gmail.cm': 'gmail.com',
        'gmail.org': 'gmail.com',
        'gemail.com': 'gmail.com',
        'gmail.com.com': 'gmail.com',
        'gmai.coml': 'gmail.com',
        'gmail.coom': 'gmail.com',
        'jmail.com': 'gmail.com',
        'gmeil.com': 'gmail.com',
        'gmall.com': 'gmail.com',
        'gail.com': 'gmail.com',
        'gmail.co': 'gmail.com',
        'gmaiil.com': 'gmail.com',
        'gmaim.com': 'gmail.com',
        'gmail.om': 'gmail.com',
        'gmail.ccom': 'gmail.com',
        'g.com': 'gmail.com',
        'gmail.vom': 'gmail.com',
        'gmali.com': 'gmail.com',
        'gmaio.com': 'gmail.com',
        'g-mail.com': 'gmail.com',
        'gamill.com': 'gmail.com',
        'gmaol.com': 'gmail.com',
        'gmsil.com': 'gmail.com',
        'gmaii.com': 'gmail.com',
        'gmail.oeg': 'gmail.com',
        'gmanil.com': 'gmail.com',
        'gmaili.com': 'gmail.com',
        'gmail.crom': 'gmail.com',
        'gmail.com11': 'gmail.com',
        'gmill.com': 'gmail.com',
        'gmail.com3': 'gmail.com',
        'gnmail.com': 'gmail.com',
        'gmaij.com': 'gmail.com',
        'gmail.coml.com': 'gmail.com',
        
        # Yahoo variations
        'yahoo.con': 'yahoo.com',
        'yaoo.com': 'yahoo.com',
        'hayoo.com': 'yahoo.com',
        'yhoo.com': 'yahoo.com',
        'yahoo.comm': 'yahoo.com',
        'myyahoo.com': 'yahoo.com',
        'yahoo.comb': 'yahoo.com',
        'yahool.com': 'yahoo.com',
        'yaho.cm': 'yahoo.com',
        'yahio.un': 'yahoo.com',
        
        # Hotmail variations
        'hmail.com': 'hotmail.com',
        'hotmai.com': 'hotmail.com',
        'hotamil.com': 'hotmail.com',
        'hotmail.con': 'hotmail.com',
        'hoo.com': 'hotmail.com',
        'hotmiail.com': 'hotmail.com',
        'htomail.com': 'hotmail.com',
        'hiotmail.com': 'hotmail.com',
        'hitmail.com': 'hotmail.com',
        
        # iCloud variations
        'icould.com': 'icloud.com',
        'iclod.com': 'icloud.com',
        'cloud.com': 'icloud.com',
        'icoud.com': 'icloud.com',
        'ichoud.com': 'icloud.com',
        'iclou.com': 'icloud.com',
        
        # AOL variations
        'ao.com': 'aol.com',
        'alo.com': 'aol.com',
        'aol.co': 'aol.com',
        'aol.om': 'aol.com',
        
        # Boston Public Schools variations
        'bostonk12.com': 'bostonk12.org',
        'boston12.org': 'bostonk12.org',
        'bostobpublicschools.org': 'bostonpublicschools.org',
        'bostonnpublicschools.org': 'bostonpublicschools.org',
        'bostonnk12.org': 'bostonk12.org',
        'bostonpulicschools.org': 'bostonpublicschools.org',
        'bostonpublicschool.org.org': 'bostonpublicschools.org',
        
        # Other common corrections
        'live.con': 'live.com',
        'live.cm': 'live.com',
        'outlook.pt': 'outlook.com',
        'verizon.com': 'verizon.net',
        'comcast.com': 'comcast.net',
    }
    
    # Function to standardize email
    def standardize_email(email):
        if pd.isna(email):
            return None
        
        # Convert to string, lowercase, and strip spaces
        email_str = str(email).lower().strip()
        
        # Remove any spaces
        email_str = email_str.replace(' ', '')
        
        # Basic validation
        if '@' not in email_str or '.' not in email_str:
            return None
        
        # Extract username and domain
        username, domain = email_str.split('@', 1)
        
        # Apply domain correction
        corrected_domain = domain_corrections.get(domain, domain)
        
        # Rebuild corrected email
        return f"{username}@{corrected_domain}"
    
    # Function to normalize name
    def normalize_name(name):
        if pd.isna(name):
            return None
        
        # Convert to string, lowercase, and strip spaces
        name_str = str(name).lower().strip()
        
        # Remove extra spaces
        name_str = re.sub(r'\s+', ' ', name_str)
        
        # Remove non-alphabetic characters except spaces
        name_str = re.sub(r'[^a-z ]', '', name_str)
        
        return name_str
    
    # Function to standardize date of birth
    def standardize_dob(dob):
        if pd.isna(dob):
            return None
        
        # Try to handle various date formats
        try:
            # If it's already a datetime
            if isinstance(dob, (pd.Timestamp, datetime)):
                return dob.strftime('%Y-%m-%d')
            
            # Convert to string
            dob_str = str(dob).strip()
            
            # Try parsing with pandas
            try:
                dob_date = pd.to_datetime(dob_str)
                
                # Adjust years for two-digit years (19xx vs 20xx)
                if dob_date.year > datetime.now().year and dob_date.year < 2100:
                    dob_date = dob_date.replace(year=dob_date.year - 100)
                
                return dob_date.strftime('%Y-%m-%d')
            except:
                pass
            
            # Handle numeric formats without separators
            if dob_str.isdigit():
                if len(dob_str) == 8:  # MMDDYYYY
                    month = int(dob_str[0:2])
                    day = int(dob_str[2:4])
                    year = int(dob_str[4:8])
                    
                    if 1 <= month <= 12 and 1 <= day <= 31:
                        return f"{year:04d}-{month:02d}-{day:02d}"
                
                elif len(dob_str) == 6:  # MMDDYY
                    month = int(dob_str[0:2])
                    day = int(dob_str[2:4])
                    year = int(dob_str[4:6])
                    
                    # Adjust years for two-digit years
                    full_year = 1900 + year if year >= 50 else 2000 + year
                    
                    if 1 <= month <= 12 and 1 <= day <= 31:
                        return f"{full_year:04d}-{month:02d}-{day:02d}"
        except:
            pass
        
        # If all parsing attempts fail, return None
        return None
    
    # Identify email columns
    pre_email_col = 'email_address' if 'email_address' in pre_df.columns else None
    if not pre_email_col:
        for col in pre_df.columns:
            if 'email' in col.lower() and 'pre' in col.lower():
                pre_email_col = col
                break
    
    post_email_col = 'email_address' if 'email_address' in post_df.columns else None
    if not post_email_col:
        for col in post_df.columns:
            if 'email' in col.lower() and 'post' in col.lower():
                post_email_col = col
                break
    
    print(f"Using pre-enrollment email column: {pre_email_col}")
    print(f"Using post-course email column: {post_email_col}")
    
    # Identify name columns
    pre_fullname_col = 'full_name' if 'full_name' in pre_df.columns else None
    if not pre_fullname_col:
        for col in pre_df.columns:
            if 'full_name' in col.lower() and 'pre' in col.lower():
                pre_fullname_col = col
                break
    
    # If no full_name column, try to create one from first and last name
    if not pre_fullname_col:
        if 'first_name' in pre_df.columns and 'last_name' in pre_df.columns:
            pre_df['full_name'] = pre_df['first_name'].fillna('') + ' ' + pre_df['last_name'].fillna('')
            pre_df['full_name'] = pre_df['full_name'].str.strip()
            pre_fullname_col = 'full_name'
    
    post_fullname_col = 'full_name' if 'full_name' in post_df.columns else None
    if not post_fullname_col:
        for col in post_df.columns:
            if 'full_name' in col.lower() and 'post' in col.lower():
                post_fullname_col = col
                break
    
    # If no full_name column, try to create one from first and last name
    if not post_fullname_col:
        if 'first_name' in post_df.columns and 'last_name' in post_df.columns:
            post_df['full_name'] = post_df['first_name'].fillna('') + ' ' + post_df['last_name'].fillna('')
            post_df['full_name'] = post_df['full_name'].str.strip()
            post_fullname_col = 'full_name'
    
    print(f"Using pre-enrollment full name column: {pre_fullname_col}")
    print(f"Using post-course full name column: {post_fullname_col}")
    
    # Identify DOB columns
    pre_dob_col = 'date_of_birth' if 'date_of_birth' in pre_df.columns else None
    if not pre_dob_col:
        for col in pre_df.columns:
            if ('birth' in col.lower() or 'dob' in col.lower()) and 'pre' in col.lower():
                pre_dob_col = col
                break
    
    post_dob_col = 'date_of_birth' if 'date_of_birth' in post_df.columns else None
    if not post_dob_col:
        for col in post_df.columns:
            if ('birth' in col.lower() or 'dob' in col.lower()) and 'post' in col.lower():
                post_dob_col = col
                break
    
    print(f"Using pre-enrollment DOB column: {pre_dob_col}")
    print(f"Using post-course DOB column: {post_dob_col}")
    
    # Apply standardization
    # Email
    if pre_email_col:
        pre_df['email_original'] = pre_df[pre_email_col]
        pre_df['email_standardized'] = pre_df[pre_email_col].apply(standardize_email)
        pre_email_changes = sum(pre_df['email_original'] != pre_df['email_standardized'])
        print(f"Standardized {pre_email_changes} pre-enrollment emails")
    
    if post_email_col:
        post_df['email_original'] = post_df[post_email_col]
        post_df['email_standardized'] = post_df[post_email_col].apply(standardize_email)
        post_email_changes = sum(post_df['email_original'] != post_df['email_standardized'])
        print(f"Standardized {post_email_changes} post-course emails")
    
    # Names
    if pre_fullname_col:
        pre_df['name_normalized'] = pre_df[pre_fullname_col].apply(normalize_name)
        print(f"Normalized {sum(pre_df['name_normalized'].notna())} pre-enrollment names")
    
    if post_fullname_col:
        post_df['name_normalized'] = post_df[post_fullname_col].apply(normalize_name)
        print(f"Normalized {sum(post_df['name_normalized'].notna())} post-course names")
    
    # DOB
    if pre_dob_col:
        pre_df['dob_standardized'] = pre_df[pre_dob_col].apply(standardize_dob)
        print(f"Standardized {sum(pre_df['dob_standardized'].notna())} pre-enrollment DOBs")
    
    if post_dob_col:
        post_df['dob_standardized'] = post_df[post_dob_col].apply(standardize_dob)
        print(f"Standardized {sum(post_df['dob_standardized'].notna())} post-course DOBs")
    
    # Step 3: Perform matching
    print("\n3. Performing high-quality matching")
    
    # 3.1: Email matching
    print("Matching by standardized email...")
    
    email_matches = []
    
    if 'email_standardized' in pre_df.columns and 'email_standardized' in post_df.columns:
        # Create dictionaries for faster lookup
        post_email_dict = {}
        for idx, row in post_df.iterrows():
            email = row['email_standardized']
            if pd.notna(email):
                if email not in post_email_dict:
                    post_email_dict[email] = []
                post_email_dict[email].append(idx)
        
        # Find matches
        for pre_idx, pre_row in pre_df.iterrows():
            pre_email = pre_row['email_standardized']
            
            if pd.notna(pre_email) and pre_email in post_email_dict:
                for post_idx in post_email_dict[pre_email]:
                    email_matches.append({
                        'pre_idx': pre_idx,
                        'post_idx': post_idx,
                        'match_type': 'email'
                    })
    
    print(f"Found {len(email_matches)} matches by email")
    
    # 3.2: Full name + DOB matching (only for records not matched by email)
    print("Matching by full name + DOB...")
    
    name_dob_matches = []
    
    if ('name_normalized' in pre_df.columns and 'name_normalized' in post_df.columns and
        'dob_standardized' in pre_df.columns and 'dob_standardized' in post_df.columns):
        
        # Get pre-enrollment indices already matched by email
        matched_pre_indices = set(match['pre_idx'] for match in email_matches)
        
        # Create a dictionary for post-course name+DOB
        post_name_dob_dict = {}
        for idx, row in post_df.iterrows():
            name = row['name_normalized']
            dob = row['dob_standardized']
            
            if pd.notna(name) and pd.notna(dob):
                key = f"{name}|{dob}"
                if key not in post_name_dob_dict:
                    post_name_dob_dict[key] = []
                post_name_dob_dict[key].append(idx)
        
        # Find matches for unmatched pre-enrollment records
        for pre_idx, pre_row in pre_df.iterrows():
            # Skip if already matched by email
            if pre_idx in matched_pre_indices:
                continue
            
            pre_name = pre_row['name_normalized']
            pre_dob = pre_row['dob_standardized']
            
            if pd.notna(pre_name) and pd.notna(pre_dob):
                key = f"{pre_name}|{pre_dob}"
                
                if key in post_name_dob_dict:
                    for post_idx in post_name_dob_dict[key]:
                        name_dob_matches.append({
                            'pre_idx': pre_idx,
                            'post_idx': post_idx,
                            'match_type': 'full_name_dob'
                        })
    
    print(f"Found {len(name_dob_matches)} additional matches by full name + DOB")
    
    # Step 4: Combine all matches
    all_matches = email_matches + name_dob_matches
    print(f"Total matches: {len(all_matches)}")
    
    # Step 5: Create merged dataset
    print("\n4. Creating merged dataset")
    
    # Check for duplicate matches (same pre_idx matched to multiple post_idx)
    pre_idx_counts = {}
    for match in all_matches:
        pre_idx = match['pre_idx']
        if pre_idx not in pre_idx_counts:
            pre_idx_counts[pre_idx] = 0
        pre_idx_counts[pre_idx] += 1
    
    duplicate_pre_matches = {pre_idx: count for pre_idx, count in pre_idx_counts.items() if count > 1}
    
    if duplicate_pre_matches:
        print(f"\nWarning: Found {len(duplicate_pre_matches)} pre-enrollment records matched to multiple post-course records")
        print("Resolving by keeping only the highest quality match for each pre-enrollment record")
        
        # Keep only one match per pre_idx, prioritizing email matches
        filtered_matches = []
        seen_pre_idx = set()
        
        # First add all email matches
        for match in email_matches:
            pre_idx = match['pre_idx']
            if pre_idx not in seen_pre_idx:
                filtered_matches.append(match)
                seen_pre_idx.add(pre_idx)
        
        # Then add name+DOB matches for unmatched pre_idx
        for match in name_dob_matches:
            pre_idx = match['pre_idx']
            if pre_idx not in seen_pre_idx:
                filtered_matches.append(match)
                seen_pre_idx.add(pre_idx)
        
        print(f"After resolving duplicates: {len(filtered_matches)} matches")
        all_matches = filtered_matches
    
    # Create a dataframe of matches
    matches_df = pd.DataFrame(all_matches)
    
    # Add match confidence score
    def calculate_match_confidence(row):
        if row['match_type'] == 'email':
            return 100
        elif row['match_type'] == 'full_name_dob':
            return 90
        return 50
    
    matches_df['match_confidence'] = matches_df.apply(calculate_match_confidence, axis=1)
    
    # Perform the merge
    merged_data = []
    
    # Prefix mappings to avoid column name conflicts
    pre_prefix_map = {}
    post_prefix_map = {}
    
    # Add prefixes to pre columns
    for col in pre_df.columns:
        if col.startswith('pre_'):
            pre_prefix_map[col] = col
        else:
            pre_prefix_map[col] = f"pre_{col}"
    
    # Add prefixes to post columns
    for col in post_df.columns:
        if col.startswith('post_'):
            post_prefix_map[col] = col
        else:
            post_prefix_map[col] = f"post_{col}"
    
    # Merge the data
    for _, match in matches_df.iterrows():
        pre_idx = match['pre_idx']
        post_idx = match['post_idx']
        
        pre_row = pre_df.loc[pre_idx].to_dict()
        post_row = post_df.loc[post_idx].to_dict()
        
        merged_row = {}
        
        # Add prefixed pre columns
        for col, value in pre_row.items():
            merged_row[pre_prefix_map[col]] = value
        
        # Add prefixed post columns
        for col, value in post_row.items():
            merged_row[post_prefix_map[col]] = value
        
        # Add match metadata
        merged_row['match_type'] = match['match_type']
        merged_row['match_confidence'] = match['match_confidence']
        
        merged_data.append(merged_row)
    
    # Create the final merged dataframe
    merged_df = pd.DataFrame(merged_data)
    
    # Step 6: Quality checking
    print("\n5. Performing quality checks on merged dataset")
    
    # Age calculation where possible
    if ('pre_date_of_birth' in merged_df.columns and 'post_course_year' in merged_df.columns):
        def calculate_age(row):
            try:
                dob = pd.to_datetime(row['pre_date_of_birth'])
                course_year = row['post_course_year']
                if pd.notna(dob) and pd.notna(course_year):
                    return course_year - dob.year
                return None
            except:
                return None
        
        merged_df['age_at_course'] = merged_df.apply(calculate_age, axis=1)
        
        # Identify age outliers
        age_stats = merged_df['age_at_course'].describe()
        print("\nAge at course statistics:")
        print(f"  Min: {age_stats['min']}")
        print(f"  Max: {age_stats['max']}")
        print(f"  Mean: {age_stats['mean']:.1f}")
        
        young_outliers = merged_df[merged_df['age_at_course'] < 5].shape[0]
        old_outliers = merged_df[merged_df['age_at_course'] > 90].shape[0]
        
        if young_outliers > 0 or old_outliers > 0:
            print(f"  Found {young_outliers} records with age < 5")
            print(f"  Found {old_outliers} records with age > 90")
            print("  Consider addressing these age outliers separately as mentioned")
    
    # Course year consistency
    if 'pre_course_year' in merged_df.columns and 'post_course_year' in merged_df.columns:
        timeline_issues = sum(merged_df['post_course_year'] < merged_df['pre_course_year'])
        
        if timeline_issues > 0:
            print(f"\nFound {timeline_issues} records with pre-course year after post-course year")
            print("Consider addressing these timeline issues separately as mentioned")
    
    # Step 7: Save the merged dataset
    print(f"\n6. Saving high-quality merged dataset to {output_file}")
    merged_df.to_excel(output_file, index=False)
    
    print("\nHigh-quality merged dataset created successfully!")
    print(f"Total records: {len(merged_df)}")
    print("Match breakdown:")
    print(f"  Email matches: {sum(merged_df['match_type'] == 'email')}")
    print(f"  Full name + DOB matches: {sum(merged_df['match_type'] == 'full_name_dob')}")
    
    # Return the merged dataframe
    return merged_df

# Run the function with your original files
pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
high_quality_merged_df = create_high_quality_merged_dataset(pre_course_path, post_course_path)

Creating high-quality merged dataset from original files

1. Loading datasets
Loading pre-enrollment data from: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx
Loaded 29700 pre-enrollment records
Loading post-course data from: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx
Loaded 7806 post-course records

2. Standardizing and cleaning data
Using pre-enrollment email column: email_address
Using post-course email column: email_address
Using pre-enrollment full name column: full_name
Using post-course full name column: full_name
Using pre-enrollment DOB column: None
Using post-course DOB column: date_of_birth
Standardized 5548 pre-enrollment emails
Standardized 2395 post-course emails
Normalized 29614 pre-enrollment names
Normalized 7749 post-course names
Standardized 7708 post-course DOBs

3. Performing high-quality matching
Matching by standardized em

In [24]:
#name + BOB match Debuging
import pandas as pd
import os
import re
from datetime import datetime
import numpy as np

def debug_matching_process(pre_course_path, post_course_path):
    """
    Debug the matching process, focusing on the full name + DOB matching.
    
    Parameters:
    pre_course_path (str): Path to the pre-enrollment Excel file
    post_course_path (str): Path to the post-course Excel file
    """
    print("Debugging matching process between pre-enrollment and post-course data")
    
    # Step 1: Load the datasets
    print("\n1. Loading datasets")
    pre_df = pd.read_excel(pre_course_path)
    print(f"Loaded {len(pre_df)} pre-enrollment records")
    
    post_df = pd.read_excel(post_course_path)
    print(f"Loaded {len(post_df)} post-course records")
    
    # Step 2: Examine column names
    print("\n2. Examining column names")
    
    # Check for DOB columns
    pre_dob_candidates = [col for col in pre_df.columns if 'birth' in col.lower() or 'dob' in col.lower()]
    post_dob_candidates = [col for col in post_df.columns if 'birth' in col.lower() or 'dob' in col.lower()]
    
    print("Pre-enrollment DOB column candidates:")
    for col in pre_dob_candidates:
        non_null = pre_df[col].notna().sum()
        print(f"  {col}: {non_null} non-null values ({non_null/len(pre_df)*100:.1f}%)")
    
    print("Post-course DOB column candidates:")
    for col in post_dob_candidates:
        non_null = post_df[col].notna().sum()
        print(f"  {col}: {non_null} non-null values ({non_null/len(post_df)*100:.1f}%)")
    
    # Check for name columns
    pre_name_candidates = [col for col in pre_df.columns if 'name' in col.lower()]
    post_name_candidates = [col for col in post_df.columns if 'name' in col.lower()]
    
    print("\nPre-enrollment name column candidates:")
    for col in pre_name_candidates:
        non_null = pre_df[col].notna().sum()
        print(f"  {col}: {non_null} non-null values ({non_null/len(pre_df)*100:.1f}%)")
    
    print("Post-course name column candidates:")
    for col in post_name_candidates:
        non_null = post_df[col].notna().sum()
        print(f"  {col}: {non_null} non-null values ({non_null/len(post_df)*100:.1f}%)")
    
    # Step 3: Select the appropriate columns
    pre_dob_col = None
    if pre_dob_candidates:
        # Choose the column with the most non-null values
        pre_dob_col = max(pre_dob_candidates, key=lambda col: pre_df[col].notna().sum())
    
    post_dob_col = None
    if post_dob_candidates:
        # Choose the column with the most non-null values
        post_dob_col = max(post_dob_candidates, key=lambda col: post_df[col].notna().sum())
    
    pre_fullname_col = 'full_name' if 'full_name' in pre_df.columns else None
    if not pre_fullname_col and 'pre_full_name' in pre_df.columns:
        pre_fullname_col = 'pre_full_name'
    
    post_fullname_col = 'full_name' if 'full_name' in post_df.columns else None
    if not post_fullname_col and 'post_full_name' in post_df.columns:
        post_fullname_col = 'post_full_name'
    
    print(f"\nSelected columns for matching:")
    print(f"Pre-enrollment DOB column: {pre_dob_col}")
    print(f"Post-course DOB column: {post_dob_col}")
    print(f"Pre-enrollment full name column: {pre_fullname_col}")
    print(f"Post-course full name column: {post_fullname_col}")
    
    # Step 4: Examine DOB formats
    if pre_dob_col and post_dob_col:
        print("\n3. Examining DOB formats")
        
        # Get sample values
        pre_dob_samples = pre_df[pre_dob_col].dropna().sample(min(5, pre_df[pre_dob_col].notna().sum())).tolist()
        post_dob_samples = post_df[post_dob_col].dropna().sample(min(5, post_df[post_dob_col].notna().sum())).tolist()
        
        print("Pre-enrollment DOB samples:")
        for sample in pre_dob_samples:
            print(f"  {sample} (type: {type(sample).__name__})")
        
        print("Post-course DOB samples:")
        for sample in post_dob_samples:
            print(f"  {sample} (type: {type(sample).__name__})")
    
    # Step 5: Standardize DOBs
    def standardize_dob(dob):
        if pd.isna(dob):
            return None
        
        # Try to handle various date formats
        try:
            # If it's already a datetime
            if isinstance(dob, (pd.Timestamp, datetime)):
                return dob.strftime('%Y-%m-%d')
            
            # Convert to string
            dob_str = str(dob).strip()
            
            # Try parsing with pandas
            try:
                dob_date = pd.to_datetime(dob_str)
                
                # Adjust years for two-digit years (19xx vs 20xx)
                if dob_date.year > datetime.now().year and dob_date.year < 2100:
                    dob_date = dob_date.replace(year=dob_date.year - 100)
                
                return dob_date.strftime('%Y-%m-%d')
            except:
                pass
            
            # Handle numeric formats without separators
            if dob_str.isdigit():
                if len(dob_str) == 8:  # MMDDYYYY
                    month = int(dob_str[0:2])
                    day = int(dob_str[2:4])
                    year = int(dob_str[4:8])
                    
                    if 1 <= month <= 12 and 1 <= day <= 31:
                        return f"{year:04d}-{month:02d}-{day:02d}"
                
                elif len(dob_str) == 6:  # MMDDYY
                    month = int(dob_str[0:2])
                    day = int(dob_str[2:4])
                    year = int(dob_str[4:6])
                    
                    # Adjust years for two-digit years
                    full_year = 1900 + year if year >= 50 else 2000 + year
                    
                    if 1 <= month <= 12 and 1 <= day <= 31:
                        return f"{full_year:04d}-{month:02d}-{day:02d}"
        except:
            pass
        
        # If all parsing attempts fail, return None
        return None
    
    if pre_dob_col and post_dob_col:
        pre_df['dob_standardized'] = pre_df[pre_dob_col].apply(standardize_dob)
        post_df['dob_standardized'] = post_df[post_dob_col].apply(standardize_dob)
        
        print("\n4. After DOB standardization")
        print(f"Pre-enrollment standardized DOBs: {pre_df['dob_standardized'].notna().sum()} non-null values")
        print(f"Post-course standardized DOBs: {post_df['dob_standardized'].notna().sum()} non-null values")
        
        # Show standardized samples
        pre_std_samples = pre_df[['dob_standardized', pre_dob_col]].dropna(subset=['dob_standardized']).sample(min(5, pre_df['dob_standardized'].notna().sum())).values.tolist()
        post_std_samples = post_df[['dob_standardized', post_dob_col]].dropna(subset=['dob_standardized']).sample(min(5, post_df['dob_standardized'].notna().sum())).values.tolist()
        
        print("\nPre-enrollment DOB standardization examples:")
        for std, orig in pre_std_samples:
            print(f"  Original: {orig} -> Standardized: {std}")
        
        print("Post-course DOB standardization examples:")
        for std, orig in post_std_samples:
            print(f"  Original: {orig} -> Standardized: {std}")
    
    # Step 6: Normalize names
    def normalize_name(name):
        if pd.isna(name):
            return None
        
        # Convert to string, lowercase, and strip spaces
        name_str = str(name).lower().strip()
        
        # Remove extra spaces
        name_str = re.sub(r'\s+', ' ', name_str)
        
        # Remove non-alphabetic characters except spaces
        name_str = re.sub(r'[^a-z ]', '', name_str)
        
        return name_str
    
    if pre_fullname_col and post_fullname_col:
        pre_df['name_normalized'] = pre_df[pre_fullname_col].apply(normalize_name)
        post_df['name_normalized'] = post_df[post_fullname_col].apply(normalize_name)
        
        print("\n5. Name normalization")
        print(f"Pre-enrollment normalized names: {pre_df['name_normalized'].notna().sum()} non-null values")
        print(f"Post-course normalized names: {post_df['name_normalized'].notna().sum()} non-null values")
        
        # Show normalized samples
        pre_name_samples = pre_df[[pre_fullname_col, 'name_normalized']].dropna().sample(min(5, pre_df['name_normalized'].notna().sum())).values.tolist()
        post_name_samples = post_df[[post_fullname_col, 'name_normalized']].dropna().sample(min(5, post_df['name_normalized'].notna().sum())).values.tolist()
        
        print("\nPre-enrollment name normalization examples:")
        for orig, norm in pre_name_samples:
            print(f"  Original: {orig} -> Normalized: {norm}")
        
        print("Post-course name normalization examples:")
        for orig, norm in post_name_samples:
            print(f"  Original: {orig} -> Normalized: {norm}")
    
    # Step 7: Attempt full name + DOB matching
    print("\n6. Attempting full name + DOB matching")
    
    if ('name_normalized' in pre_df.columns and 'name_normalized' in post_df.columns and
        'dob_standardized' in pre_df.columns and 'dob_standardized' in post_df.columns):
        
        # Create name+DOB keys for both datasets
        pre_df['name_dob_key'] = pre_df.apply(
            lambda row: f"{row['name_normalized']}|{row['dob_standardized']}" 
            if pd.notna(row['name_normalized']) and pd.notna(row['dob_standardized']) else None, 
            axis=1
        )
        
        post_df['name_dob_key'] = post_df.apply(
            lambda row: f"{row['name_normalized']}|{row['dob_standardized']}" 
            if pd.notna(row['name_normalized']) and pd.notna(row['dob_standardized']) else None, 
            axis=1
        )
        
        print(f"Pre-enrollment records with valid name+DOB keys: {pre_df['name_dob_key'].notna().sum()}")
        print(f"Post-course records with valid name+DOB keys: {post_df['name_dob_key'].notna().sum()}")
        
        # Find common keys
        pre_keys = set(pre_df['name_dob_key'].dropna())
        post_keys = set(post_df['name_dob_key'].dropna())
        
        common_keys = pre_keys.intersection(post_keys)
        print(f"Number of matching name+DOB combinations: {len(common_keys)}")
        
        # Show a few examples of matching records
        if common_keys:
            sample_keys = list(common_keys)[:min(5, len(common_keys))]
            
            print("\nSample matching records:")
            for key in sample_keys:
                pre_matches = pre_df[pre_df['name_dob_key'] == key]
                post_matches = post_df[post_df['name_dob_key'] == key]
                
                print(f"Key: {key}")
                print(f"  Pre-enrollment: {pre_matches[pre_fullname_col].iloc[0]} (DOB: {pre_matches[pre_dob_col].iloc[0] if pre_dob_col else 'N/A'})")
                print(f"  Post-course: {post_matches[post_fullname_col].iloc[0]} (DOB: {post_matches[post_dob_col].iloc[0]})")
    else:
        print("Cannot perform name+DOB matching due to missing data")
    
    # Step 8: Check for DOB value distribution
    if 'dob_standardized' in pre_df.columns and 'dob_standardized' in post_df.columns:
        print("\n7. Analyzing DOB distribution")
        
        # Count years
        if pre_df['dob_standardized'].notna().any():
            pre_years = pre_df['dob_standardized'].dropna().str[:4].value_counts().sort_index()
            print("Top 5 birth years in pre-enrollment data:")
            for year, count in pre_years.head().items():
                print(f"  {year}: {count}")
        else:
            print("No valid DOB years in pre-enrollment data")
        
        if post_df['dob_standardized'].notna().any():
            post_years = post_df['dob_standardized'].dropna().str[:4].value_counts().sort_index()
            print("Top 5 birth years in post-course data:")
            for year, count in post_years.head().items():
                print(f"  {year}: {count}")
        else:
            print("No valid DOB years in post-course data")
    
    print("\nDebugging completed. You can use this information to adjust the matching algorithm.")

# Run the debug function
pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
debug_matching_process(pre_course_path, post_course_path)


Debugging matching process between pre-enrollment and post-course data

1. Loading datasets
Loaded 29700 pre-enrollment records
Loaded 7806 post-course records

2. Examining column names
Pre-enrollment DOB column candidates:
  date_of_birth_standardized: 29611 non-null values (99.7%)
  childs_date_of_birth: 7097 non-null values (23.9%)
Post-course DOB column candidates:
  date_of_birth: 7748 non-null values (99.3%)

Pre-enrollment name column candidates:
  name_of_siteschool: 18047 non-null values (60.8%)
  course_name: 19991 non-null values (67.3%)
  first_name: 29614 non-null values (99.7%)
  last_name: 29613 non-null values (99.7%)
  full_name: 29614 non-null values (99.7%)
  childs_first_name: 7052 non-null values (23.7%)
  childs_last_name: 7050 non-null values (23.7%)
  name_of_business: 315 non-null values (1.1%)
  please_enter_anyall_of_your_social_media_handles_separated_by_a_comma_eg_techgoeshome_facebookcomyourbusinessname: 1877 non-null values (6.3%)
  middle_name: 7148 non

In [26]:
#enhanced matching with email, name + DOB
import pandas as pd
import os
import re
from datetime import datetime
import numpy as np

def create_high_quality_merged_dataset(pre_course_path, post_course_path):
    """
    Create a high-quality merged dataset directly from pre-enrollment and post-course data,
    using both email and full name + DOB matching.
    
    Parameters:
    pre_course_path (str): Path to the pre-enrollment Excel file
    post_course_path (str): Path to the post-course Excel file
    """
    print("Creating high-quality merged dataset from original files")
    
    # Step 1: Load the datasets
    print("\n1. Loading datasets")
    print(f"Loading pre-enrollment data from: {pre_course_path}")
    pre_df = pd.read_excel(pre_course_path)
    print(f"Loaded {len(pre_df)} pre-enrollment records")
    
    print(f"Loading post-course data from: {post_course_path}")
    post_df = pd.read_excel(post_course_path)
    print(f"Loaded {len(post_df)} post-course records")
    
    # Create output directory and file path
    output_dir = os.path.dirname(pre_course_path)
    output_file = os.path.join(output_dir, "Direct_High_Quality_Merged_Dataset.xlsx")
    
    # Step 2: Standardize and clean data for matching
    print("\n2. Standardizing and cleaning data")
    
    # Domain corrections dictionary
    domain_corrections = {
        # Gmail variations
        'gmai.com': 'gmail.com',
        'gmil.com': 'gmail.com',
        'gmal.com': 'gmail.com',
        'gamail.com': 'gmail.com',
        'gimail.com': 'gmail.com',
        'gmaill.com': 'gmail.com',
        'gmail.comm': 'gmail.com',
        'gmial.com': 'gmail.com',
        'gmail.cm': 'gmail.com',
        'gmail.org': 'gmail.com',
        'gemail.com': 'gmail.com',
        'gmail.com.com': 'gmail.com',
        'gmai.coml': 'gmail.com',
        'gmail.coom': 'gmail.com',
        'jmail.com': 'gmail.com',
        'gmeil.com': 'gmail.com',
        'gmall.com': 'gmail.com',
        'gail.com': 'gmail.com',
        'gmail.co': 'gmail.com',
        'gmaiil.com': 'gmail.com',
        'gmaim.com': 'gmail.com',
        'gmail.om': 'gmail.com',
        'gmail.ccom': 'gmail.com',
        'g.com': 'gmail.com',
        'gmail.vom': 'gmail.com',
        'gmali.com': 'gmail.com',
        'gmaio.com': 'gmail.com',
        'g-mail.com': 'gmail.com',
        'gamill.com': 'gmail.com',
        'gmaol.com': 'gmail.com',
        'gmsil.com': 'gmail.com',
        'gmaii.com': 'gmail.com',
        'gmail.oeg': 'gmail.com',
        'gmanil.com': 'gmail.com',
        'gmaili.com': 'gmail.com',
        'gmail.crom': 'gmail.com',
        'gmail.com11': 'gmail.com',
        'gmill.com': 'gmail.com',
        'gmail.com3': 'gmail.com',
        'gnmail.com': 'gmail.com',
        'gmaij.com': 'gmail.com',
        'gmail.coml.com': 'gmail.com',
        
        # Yahoo variations
        'yahoo.con': 'yahoo.com',
        'yaoo.com': 'yahoo.com',
        'hayoo.com': 'yahoo.com',
        'yhoo.com': 'yahoo.com',
        'yahoo.comm': 'yahoo.com',
        'myyahoo.com': 'yahoo.com',
        'yahoo.comb': 'yahoo.com',
        'yahool.com': 'yahoo.com',
        'yaho.cm': 'yahoo.com',
        'yahio.un': 'yahoo.com',
        
        # Hotmail variations
        'hmail.com': 'hotmail.com',
        'hotmai.com': 'hotmail.com',
        'hotamil.com': 'hotmail.com',
        'hotmail.con': 'hotmail.com',
        'hoo.com': 'hotmail.com',
        'hotmiail.com': 'hotmail.com',
        'htomail.com': 'hotmail.com',
        'hiotmail.com': 'hotmail.com',
        'hitmail.com': 'hotmail.com',
        
        # iCloud variations
        'icould.com': 'icloud.com',
        'iclod.com': 'icloud.com',
        'cloud.com': 'icloud.com',
        'icoud.com': 'icloud.com',
        'ichoud.com': 'icloud.com',
        'iclou.com': 'icloud.com',
        
        # AOL variations
        'ao.com': 'aol.com',
        'alo.com': 'aol.com',
        'aol.co': 'aol.com',
        'aol.om': 'aol.com',
        
        # Boston Public Schools variations
        'bostonk12.com': 'bostonk12.org',
        'boston12.org': 'bostonk12.org',
        'bostobpublicschools.org': 'bostonpublicschools.org',
        'bostonnpublicschools.org': 'bostonpublicschools.org',
        'bostonnk12.org': 'bostonk12.org',
        'bostonpulicschools.org': 'bostonpublicschools.org',
        'bostonpublicschool.org.org': 'bostonpublicschools.org',
        
        # Other common corrections
        'live.con': 'live.com',
        'live.cm': 'live.com',
        'outlook.pt': 'outlook.com',
        'verizon.com': 'verizon.net',
        'comcast.com': 'comcast.net',
    }
    
    # Email standardization
    def standardize_email(email):
        if pd.isna(email):
            return None
        
        # Convert to string, lowercase, and strip spaces
        email_str = str(email).lower().strip()
        
        # Remove any spaces
        email_str = email_str.replace(' ', '')
        
        # Basic validation
        if '@' not in email_str or '.' not in email_str:
            return None
        
        # Extract username and domain
        username, domain = email_str.split('@', 1)
        
        # Apply domain correction
        corrected_domain = domain_corrections.get(domain, domain)
        
        # Rebuild corrected email
        return f"{username}@{corrected_domain}"
    
    # Name normalization
    def normalize_name(name):
        if pd.isna(name):
            return None
        
        # Convert to string, lowercase, and strip spaces
        name_str = str(name).lower().strip()
        
        # Remove extra spaces
        name_str = re.sub(r'\s+', ' ', name_str)
        
        # Remove non-alphabetic characters except spaces
        name_str = re.sub(r'[^a-z ]', '', name_str)
        
        return name_str
    
    # DOB standardization
    def standardize_dob(dob):
        if pd.isna(dob):
            return None
        
        # Try to handle various date formats
        try:
            # If it's already a datetime
            if isinstance(dob, (pd.Timestamp, datetime)):
                return dob.strftime('%Y-%m-%d')
            
            # Convert to string
            dob_str = str(dob).strip()
            
            # Try parsing with pandas
            try:
                dob_date = pd.to_datetime(dob_str)
                
                # Adjust years for two-digit years (19xx vs 20xx)
                if dob_date.year > datetime.now().year and dob_date.year < 2100:
                    dob_date = dob_date.replace(year=dob_date.year - 100)
                
                # Additional sanity check for very old birth years
                if dob_date.year < 1900:
                    # This is likely an error, try to fix common issues
                    # For example, if the date looks like 01-01-0001, it might be a placeholder
                    if dob_date.year < 1800:
                        return None
                
                return dob_date.strftime('%Y-%m-%d')
            except:
                pass
            
            # Handle numeric formats without separators
            if dob_str.isdigit():
                if len(dob_str) == 8:  # MMDDYYYY
                    month = int(dob_str[0:2])
                    day = int(dob_str[2:4])
                    year = int(dob_str[4:8])
                    
                    if 1 <= month <= 12 and 1 <= day <= 31:
                        return f"{year:04d}-{month:02d}-{day:02d}"
                
                elif len(dob_str) == 6:  # MMDDYY
                    month = int(dob_str[0:2])
                    day = int(dob_str[2:4])
                    year = int(dob_str[4:6])
                    
                    # Adjust years for two-digit years
                    full_year = 1900 + year if year >= 50 else 2000 + year
                    
                    if 1 <= month <= 12 and 1 <= day <= 31:
                        return f"{full_year:04d}-{month:02d}-{day:02d}"
        except:
            pass
        
        # If all parsing attempts fail, return None
        return None
    
    # Select the correct columns based on the debugging results
    pre_email_col = 'email_address'
    post_email_col = 'email_address'
    pre_fullname_col = 'full_name'
    post_fullname_col = 'full_name'
    pre_dob_col = 'date_of_birth_standardized'
    post_dob_col = 'date_of_birth'
    
    print(f"Using pre-enrollment email column: {pre_email_col}")
    print(f"Using post-course email column: {post_email_col}")
    print(f"Using pre-enrollment full name column: {pre_fullname_col}")
    print(f"Using post-course full name column: {post_fullname_col}")
    print(f"Using pre-enrollment DOB column: {pre_dob_col}")
    print(f"Using post-course DOB column: {post_dob_col}")
    
    # Apply standardization
    # Email
    if pre_email_col in pre_df.columns:
        pre_df['email_original'] = pre_df[pre_email_col]
        pre_df['email_standardized'] = pre_df[pre_email_col].apply(standardize_email)
        pre_email_changes = sum((pre_df['email_original'] != pre_df['email_standardized']) & 
                               pre_df['email_standardized'].notna())
        print(f"Standardized {pre_email_changes} pre-enrollment emails")
    
    if post_email_col in post_df.columns:
        post_df['email_original'] = post_df[post_email_col]
        post_df['email_standardized'] = post_df[post_email_col].apply(standardize_email)
        post_email_changes = sum((post_df['email_original'] != post_df['email_standardized']) & 
                                post_df['email_standardized'].notna())
        print(f"Standardized {post_email_changes} post-course emails")
    
    # Names
    if pre_fullname_col in pre_df.columns:
        pre_df['name_normalized'] = pre_df[pre_fullname_col].apply(normalize_name)
        print(f"Normalized {sum(pre_df['name_normalized'].notna())} pre-enrollment names")
    
    if post_fullname_col in post_df.columns:
        post_df['name_normalized'] = post_df[post_fullname_col].apply(normalize_name)
        print(f"Normalized {sum(post_df['name_normalized'].notna())} post-course names")
    
    # DOB
    if pre_dob_col in pre_df.columns:
        pre_df['dob_standardized'] = pre_df[pre_dob_col].apply(standardize_dob)
        print(f"Standardized {sum(pre_df['dob_standardized'].notna())} pre-enrollment DOBs")
    
    if post_dob_col in post_df.columns:
        post_df['dob_standardized'] = post_df[post_dob_col].apply(standardize_dob)
        print(f"Standardized {sum(post_df['dob_standardized'].notna())} post-course DOBs")
    
    # Step 3: Perform matching independently using both methods
    print("\n3. Performing matching using multiple identifiers")
    
    # 3.1: Email matching
    print("Matching by standardized email...")
    
    email_matches = []
    
    if 'email_standardized' in pre_df.columns and 'email_standardized' in post_df.columns:
        # Create dictionaries for faster lookup
        post_email_dict = {}
        for idx, row in post_df.iterrows():
            email = row['email_standardized']
            if pd.notna(email):
                if email not in post_email_dict:
                    post_email_dict[email] = []
                post_email_dict[email].append(idx)
        
        # Find matches
        for pre_idx, pre_row in pre_df.iterrows():
            pre_email = pre_row['email_standardized']
            
            if pd.notna(pre_email) and pre_email in post_email_dict:
                for post_idx in post_email_dict[pre_email]:
                    email_matches.append({
                        'pre_idx': pre_idx,
                        'post_idx': post_idx,
                        'match_type': 'email'
                    })
    
    print(f"Found {len(email_matches)} matches by email")
    
    # 3.2: Full name + DOB matching (independently, not just for unmatched records)
    print("Matching by full name + DOB...")
    
    name_dob_matches = []
    
    if ('name_normalized' in pre_df.columns and 'name_normalized' in post_df.columns and
        'dob_standardized' in pre_df.columns and 'dob_standardized' in post_df.columns):
        
        # Create name+DOB keys for both datasets
        pre_df['name_dob_key'] = pre_df.apply(
            lambda row: f"{row['name_normalized']}|{row['dob_standardized']}" 
            if pd.notna(row['name_normalized']) and pd.notna(row['dob_standardized']) else None, 
            axis=1
        )
        
        post_df['name_dob_key'] = post_df.apply(
            lambda row: f"{row['name_normalized']}|{row['dob_standardized']}" 
            if pd.notna(row['name_normalized']) and pd.notna(row['dob_standardized']) else None, 
            axis=1
        )
        
        # Create dictionary for post-course name+DOB
        post_name_dob_dict = {}
        for idx, row in post_df.iterrows():
            key = row['name_dob_key']
            if pd.notna(key):
                if key not in post_name_dob_dict:
                    post_name_dob_dict[key] = []
                post_name_dob_dict[key].append(idx)
        
        # Find matches
        for pre_idx, pre_row in pre_df.iterrows():
            pre_key = pre_row['name_dob_key']
            
            if pd.notna(pre_key) and pre_key in post_name_dob_dict:
                for post_idx in post_name_dob_dict[pre_key]:
                    name_dob_matches.append({
                        'pre_idx': pre_idx,
                        'post_idx': post_idx,
                        'match_type': 'full_name_dob'
                    })
    
    print(f"Found {len(name_dob_matches)} matches by full name + DOB")
    
    # Step 4: Combine all matches
    all_matches = email_matches + name_dob_matches
    print(f"Total raw matches (before de-duplication): {len(all_matches)}")
    
    # Step 5: Resolve duplicates by creating a unique identifier for each match pair
    print("\n4. Resolving duplicate matches")
    
    match_df = pd.DataFrame(all_matches)
    match_df['match_pair'] = match_df['pre_idx'].astype(str) + '_' + match_df['post_idx'].astype(str)
    
    # Count duplicate pairs
    pair_counts = match_df['match_pair'].value_counts()
    duplicate_pairs = pair_counts[pair_counts > 1].index.tolist()
    print(f"Found {len(duplicate_pairs)} duplicate match pairs")
    
    # For duplicate pairs, keep the email match (higher confidence)
    if duplicate_pairs:
        # Sort by match type to prioritize email matches
        match_df['match_type_priority'] = match_df['match_type'].map({'email': 1, 'full_name_dob': 2})
        match_df = match_df.sort_values(['match_pair', 'match_type_priority'])
        
        # Keep only the first occurrence of each match pair (which will be email match if exists)
        match_df = match_df.drop_duplicates(subset=['match_pair'], keep='first')
        
        # Drop the temporary column
        match_df = match_df.drop(columns=['match_type_priority'])
    
    print(f"Unique matches after de-duplication: {len(match_df)}")
    
    # Step 6: Count matches by type
    email_match_count = sum(match_df['match_type'] == 'email')
    name_dob_match_count = sum(match_df['match_type'] == 'full_name_dob')
    
    print(f"Final match breakdown:")
    print(f"  Email matches: {email_match_count}")
    print(f"  Full name + DOB matches: {name_dob_match_count}")
    print(f"  Total unique matches: {len(match_df)}")
    
    # Step 7: Create merged dataset
    print("\n5. Creating merged dataset")
    
    # Check for duplicate pre_idx (one pre-enrollment record matched to multiple post-course records)
    pre_idx_counts = match_df['pre_idx'].value_counts()
    duplicate_pre = pre_idx_counts[pre_idx_counts > 1]
    
    if len(duplicate_pre) > 0:
        print(f"Warning: {len(duplicate_pre)} pre-enrollment records are matched to multiple post-course records")
        print("Keeping only one match per pre-enrollment record")
        
        # For each pre_idx with multiple matches, keep only the match with the highest confidence
        # (email matches over name+DOB matches)
        for pre_idx in duplicate_pre.index:
            matches = match_df[match_df['pre_idx'] == pre_idx]
            # If there's an email match, keep it; otherwise keep the first name+DOB match
            if any(matches['match_type'] == 'email'):
                email_match = matches[matches['match_type'] == 'email'].iloc[0]
                match_df = match_df[~((match_df['pre_idx'] == pre_idx) & (match_df['match_type'] == 'full_name_dob'))]
            else:
                # Keep only the first name+DOB match
                first_match = matches.iloc[0]
                match_df = match_df[~((match_df['pre_idx'] == pre_idx) & (match_df.index != first_match.name))]
    
    # Perform the merge
    merged_data = []
    
    # Prefix mappings to avoid column name conflicts
    pre_prefix_map = {}
    post_prefix_map = {}
    
    # Add prefixes to pre columns
    for col in pre_df.columns:
        if col.startswith('pre_'):
            pre_prefix_map[col] = col
        else:
            pre_prefix_map[col] = f"pre_{col}"
    
    # Add prefixes to post columns
    for col in post_df.columns:
        if col.startswith('post_'):
            post_prefix_map[col] = col
        else:
            post_prefix_map[col] = f"post_{col}"
    
    # Merge the data
    for _, match in match_df.iterrows():
        pre_idx = match['pre_idx']
        post_idx = match['post_idx']
        
        pre_row = pre_df.loc[pre_idx].to_dict()
        post_row = post_df.loc[post_idx].to_dict()
        
        merged_row = {}
        
        # Add prefixed pre columns
        for col, value in pre_row.items():
            merged_row[pre_prefix_map[col]] = value
        
        # Add prefixed post columns
        for col, value in post_row.items():
            merged_row[post_prefix_map[col]] = value
        
        # Add match metadata
        merged_row['match_type'] = match['match_type']
        merged_row['match_confidence'] = 100 if match['match_type'] == 'email' else 90
        
        merged_data.append(merged_row)
    
    # Create the final merged dataframe
    merged_df = pd.DataFrame(merged_data)
    
    # Step 8: Quality checking
    print("\n6. Performing quality checks on merged dataset")
    
    # Age calculation where possible
    if ('pre_dob_standardized' in merged_df.columns and 'post_course_year' in merged_df.columns):
        def calculate_age(row):
            try:
                if pd.notna(row['pre_dob_standardized']):
                    dob_str = row['pre_dob_standardized']
                    dob_year = int(dob_str.split('-')[0])
                    course_year = row['post_course_year']
                    if pd.notna(course_year):
                        age = course_year - dob_year
                        # Sanity check for reasonable age
                        if 0 <= age <= 120:
                            return age
                return None
            except:
                return None
        
        merged_df['age_at_course'] = merged_df.apply(calculate_age, axis=1)
        
        # Identify age outliers
        age_stats = merged_df['age_at_course'].describe()
        print("\nAge at course statistics:")
        print(f"  Min: {age_stats['min']}")
        print(f"  Max: {age_stats['max']}")
        print(f"  Mean: {age_stats['mean']:.1f}")
        
        young_outliers = merged_df[merged_df['age_at_course'] < 5].shape[0]
        old_outliers = merged_df[merged_df['age_at_course'] > 90].shape[0]
        
        if young_outliers > 0 or old_outliers > 0:
            print(f"  Found {young_outliers} records with age < 5")
            print(f"  Found {old_outliers} records with age > 90")
            print("  Consider addressing these age outliers separately as mentioned")
    
    # Course year consistency
    if 'pre_course_year' in merged_df.columns and 'post_course_year' in merged_df.columns:
        timeline_issues = sum(merged_df['post_course_year'] < merged_df['pre_course_year'])
        
        if timeline_issues > 0:
            print(f"\nFound {timeline_issues} records with pre-course year after post-course year")
            print("Consider addressing these timeline issues separately as mentioned")
    
    # Step 9: Save the merged dataset
    print(f"\n7. Saving high-quality merged dataset to {output_file}")
    merged_df.to_excel(output_file, index=False)
    
    print("\nHigh-quality merged dataset created successfully!")
    print(f"Total records: {len(merged_df)}")
    print("Match breakdown:")
    print(f"  Email matches: {sum(merged_df['match_type'] == 'email')}")
    print(f"  Full name + DOB matches: {sum(merged_df['match_type'] == 'full_name_dob')}")
    
    # Return the merged dataframe
    return merged_df

# Run the function with your original files
pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
high_quality_merged_df = create_high_quality_merged_dataset(pre_course_path, post_course_path)

Creating high-quality merged dataset from original files

1. Loading datasets
Loading pre-enrollment data from: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx
Loaded 29700 pre-enrollment records
Loading post-course data from: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx
Loaded 7806 post-course records

2. Standardizing and cleaning data
Using pre-enrollment email column: email_address
Using post-course email column: email_address
Using pre-enrollment full name column: full_name
Using post-course full name column: full_name
Using pre-enrollment DOB column: date_of_birth_standardized
Using post-course DOB column: date_of_birth
Standardized 281 pre-enrollment emails
Standardized 51 post-course emails
Normalized 29614 pre-enrollment names
Normalized 7749 post-course names
Standardized 29583 pre-enrollment DOBs
Standardized 7708 post-course DOBs

3. Pe

In [28]:
#creating gold standard data
import pandas as pd

# Load the comprehensive dataset
comprehensive_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Direct_High_Quality_Merged_Dataset.xlsx'
comprehensive_df = pd.read_excel(comprehensive_path)

# Create the Gold Standard dataset (email matches only)
email_only_df = comprehensive_df[comprehensive_df['match_type'] == 'email']

# Save to a new Excel file
email_only_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Gold_Standard_Email_Only_Dataset.xlsx'
email_only_df.to_excel(email_only_path, index=False)

print(f"Gold Standard dataset created with {len(email_only_df)} records")
print(f"Saved to: {email_only_path}")

Gold Standard dataset created with 3241 records
Saved to: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Gold_Standard_Email_Only_Dataset.xlsx


In [30]:
pip install python-docx matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [32]:
#Creating documentation
from docx import Document
from docx.shared import Pt, Inches, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
import matplotlib.pyplot as plt
import numpy as np
import os

# Create output directory for documentation
output_dir = os.path.dirname(comprehensive_path)
doc_path = os.path.join(output_dir, "Data_Matching_Methodology.docx")

# Create visualizations for the documentation
viz_dir = os.path.join(output_dir, "Visualizations")
os.makedirs(viz_dir, exist_ok=True)

# Create a visualization of the match types
match_counts = comprehensive_df['match_type'].value_counts()
labels = match_counts.index
sizes = match_counts.values
colors = ['#3498db', '#2ecc71']

plt.figure(figsize=(10, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Distribution of Match Types in Comprehensive Dataset')
pie_chart_path = os.path.join(viz_dir, "match_type_distribution.png")
plt.savefig(pie_chart_path, dpi=300, bbox_inches='tight')
plt.close()

# Create a bar chart comparing dataset sizes
datasets = ['Pre-enrollment', 'Post-course', 'Comprehensive Matches', 'Gold Standard Matches']
counts = [29700, 7806, len(comprehensive_df), len(email_only_df)]

plt.figure(figsize=(12, 6))
bars = plt.bar(datasets, counts, color=['#3498db', '#2ecc71', '#9b59b6', '#f1c40f'])

# Add count labels on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{int(height):,}', ha='center', va='bottom', fontsize=11)

plt.title('Dataset Sizes')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
dataset_chart_path = os.path.join(viz_dir, "dataset_sizes.png")
plt.savefig(dataset_chart_path, dpi=300, bbox_inches='tight')
plt.close()

# Create the Word document
doc = Document()

# Title
title = doc.add_heading('Data Processing and Matching Methodology', 0)
title.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Introduction
doc.add_heading('1. Introduction', 1)
p = doc.add_paragraph()
p.add_run('This document details the methodology used to process and match pre-enrollment and post-course survey data for analysis. Two datasets were created:')
p.add_run().add_break()
p.add_run('• Comprehensive Dataset: ').bold = True
p.add_run(f'Contains {len(comprehensive_df):,} records matched using both email and full name + date of birth')
p.add_run().add_break()
p.add_run('• Gold Standard Dataset: ').bold = True
p.add_run(f'Contains {len(email_only_df):,} records matched exclusively using email addresses')

# Original Data Overview
doc.add_heading('2. Original Data Overview', 1)
p = doc.add_paragraph()
p.add_run('The original data consisted of two separate datasets:')
p.add_run().add_break()
p.add_run('• Pre-enrollment Survey: ').bold = True
p.add_run(f'{29700:,} records')
p.add_run().add_break()
p.add_run('• Post-course Survey: ').bold = True
p.add_run(f'{7806:,} records')

p = doc.add_paragraph()
p.add_run('Key data quality issues identified in the original datasets:')
doc.add_paragraph('• Inconsistent email formats and domains', style='List Bullet')
doc.add_paragraph('• Varied name formatting (capitalization, special characters)', style='List Bullet')
doc.add_paragraph('• Inconsistent date of birth formats', style='List Bullet')
doc.add_paragraph('• Duplicate email addresses', style='List Bullet')
doc.add_paragraph('• Timeline inconsistencies between pre and post course dates', style='List Bullet')

# Data Standardization
doc.add_heading('3. Data Standardization Process', 1)

# Email standardization
doc.add_heading('3.1 Email Standardization', 2)
p = doc.add_paragraph()
p.add_run('The email standardization process involved:')
doc.add_paragraph('• Converting all emails to lowercase', style='List Bullet')
doc.add_paragraph('• Removing spaces', style='List Bullet')
doc.add_paragraph('• Correcting common domain misspellings (e.g., "gmail.com" vs "gmai.com")', style='List Bullet')
doc.add_paragraph('• Validating basic email format (must contain @ and .)', style='List Bullet')

p = doc.add_paragraph()
p.add_run('Results of email standardization:')
doc.add_paragraph(f'• {281} pre-enrollment emails standardized', style='List Bullet')
doc.add_paragraph(f'• {51} post-course emails standardized', style='List Bullet')
doc.add_paragraph('• Common corrections included fixing variations of gmail.com, yahoo.com, and hotmail.com', style='List Bullet')

# Name standardization
doc.add_heading('3.2 Name Standardization', 2)
p = doc.add_paragraph()
p.add_run('The name standardization process involved:')
doc.add_paragraph('• Converting all names to lowercase', style='List Bullet')
doc.add_paragraph('• Removing extra spaces', style='List Bullet')
doc.add_paragraph('• Removing non-alphabetic characters', style='List Bullet')
doc.add_paragraph('• Standardizing spacing between name parts', style='List Bullet')

p = doc.add_paragraph()
p.add_run('Results of name standardization:')
doc.add_paragraph(f'• {29614} pre-enrollment names normalized', style='List Bullet')
doc.add_paragraph(f'• {7749} post-course names normalized', style='List Bullet')

# DOB standardization
doc.add_heading('3.3 Date of Birth Standardization', 2)
p = doc.add_paragraph()
p.add_run('The date of birth standardization process involved:')
doc.add_paragraph('• Converting all dates to YYYY-MM-DD format', style='List Bullet')
doc.add_paragraph('• Handling various input formats (MM/DD/YYYY, MM-DD-YYYY, etc.)', style='List Bullet')
doc.add_paragraph('• Properly interpreting two-digit years (e.g., "74" as 1974)', style='List Bullet')
doc.add_paragraph('• Filtering out implausible dates (e.g., birth years before 1900)', style='List Bullet')

p = doc.add_paragraph()
p.add_run('Results of date of birth standardization:')
doc.add_paragraph(f'• {29583} pre-enrollment DOBs standardized', style='List Bullet')
doc.add_paragraph(f'• {7708} post-course DOBs standardized', style='List Bullet')

# Matching Methodology
doc.add_heading('4. Matching Methodology', 1)

# Email matching
doc.add_heading('4.1 Email Matching', 2)
p = doc.add_paragraph()
p.add_run('Email matching process:')
doc.add_paragraph('• Used standardized email addresses as the primary identifier', style='List Bullet')
doc.add_paragraph('• Created an exact match between pre-enrollment and post-course records', style='List Bullet')
doc.add_paragraph('• Assigned 100% confidence to email matches', style='List Bullet')

p = doc.add_paragraph()
p.add_run('Results of email matching:')
doc.add_paragraph(f'• {3241} unique matches identified by email', style='List Bullet')

# Name+DOB matching
doc.add_heading('4.2 Full Name + DOB Matching', 2)
p = doc.add_paragraph()
p.add_run('Full name + DOB matching process:')
doc.add_paragraph('• Used normalized full names combined with standardized date of birth', style='List Bullet')
doc.add_paragraph('• Required exact match on both name and DOB', style='List Bullet')
doc.add_paragraph('• Assigned 90% confidence to name+DOB matches', style='List Bullet')

p = doc.add_paragraph()
p.add_run('Results of name+DOB matching:')
doc.add_paragraph(f'• {5314} initial matches identified by name+DOB', style='List Bullet')
doc.add_paragraph(f'• {2854} unique matches by name+DOB after deduplication', style='List Bullet')
doc.add_paragraph(f'• Final count: {2726} name+DOB matches in the comprehensive dataset', style='List Bullet')

# Deduplication
doc.add_heading('4.3 Deduplication Process', 2)
p = doc.add_paragraph()
p.add_run('The deduplication process involved:')
doc.add_paragraph('• Identifying records matched by both email and name+DOB', style='List Bullet')
doc.add_paragraph('• Prioritizing email matches (higher confidence) over name+DOB matches', style='List Bullet')
doc.add_paragraph('• Ensuring each pre-enrollment record was matched to only one post-course record', style='List Bullet')

p = doc.add_paragraph()
p.add_run('Deduplication results:')
doc.add_paragraph(f'• {8555} total raw matches before deduplication', style='List Bullet')
doc.add_paragraph(f'• {2460} duplicate match pairs removed', style='List Bullet')
doc.add_paragraph(f'• {123} pre-enrollment records matched to multiple post-course records (kept highest quality match)', style='List Bullet')
doc.add_paragraph(f'• {6095} unique matches after initial deduplication', style='List Bullet')
doc.add_paragraph(f'• {5967} final matches after resolving all duplications', style='List Bullet')

# Final Datasets
doc.add_heading('5. Final Datasets', 1)

# Comprehensive Dataset
doc.add_heading('5.1 Comprehensive Dataset', 2)
p = doc.add_paragraph()
p.add_run('Comprehensive Dataset characteristics:')
doc.add_paragraph(f'• Total records: {len(comprehensive_df):,}', style='List Bullet')
doc.add_paragraph(f'• Email matches: {sum(comprehensive_df["match_type"] == "email"):,} ({sum(comprehensive_df["match_type"] == "email")/len(comprehensive_df)*100:.1f}%)', style='List Bullet')
doc.add_paragraph(f'• Name+DOB matches: {sum(comprehensive_df["match_type"] == "full_name_dob"):,} ({sum(comprehensive_df["match_type"] == "full_name_dob")/len(comprehensive_df)*100:.1f}%)', style='List Bullet')
doc.add_paragraph('• Includes both high and very high confidence matches', style='List Bullet')
doc.add_paragraph('• Provides maximum sample size while maintaining high reliability', style='List Bullet')
doc.add_paragraph('• Recommended for most analytical purposes', style='List Bullet')

# Gold Standard Dataset
doc.add_heading('5.2 Gold Standard Dataset', 2)
p = doc.add_paragraph()
p.add_run('Gold Standard Dataset characteristics:')
doc.add_paragraph(f'• Total records: {len(email_only_df):,}', style='List Bullet')
doc.add_paragraph('• Email matches only (100% confidence)', style='List Bullet')
doc.add_paragraph('• Highest level of confidence in matches', style='List Bullet')
doc.add_paragraph('• Recommended for analyses requiring absolute certainty in matches', style='List Bullet')
doc.add_paragraph('• Can be used for validation of findings from the Comprehensive Dataset', style='List Bullet')

# Data Quality Issues
doc.add_heading('6. Remaining Data Quality Issues', 1)

p = doc.add_paragraph()
p.add_run('The following quality issues were identified in the final datasets:')

# Age outliers
doc.add_heading('6.1 Age Outliers', 2)
p = doc.add_paragraph()
p.add_run('Age calculation was performed based on birth year and course year:')
doc.add_paragraph(f'• Age range: {0} to {93} years', style='List Bullet')
doc.add_paragraph(f'• Mean age: {44.4:.1f} years', style='List Bullet')
doc.add_paragraph(f'• {19} records with age < 5 years', style='List Bullet')
doc.add_paragraph(f'• {7} records with age > 90 years', style='List Bullet')
p = doc.add_paragraph()
p.add_run('Recommendation: ').bold = True
p.add_run('These age outliers should be reviewed and potentially excluded from age-dependent analyses.')

# Timeline inconsistencies
doc.add_heading('6.2 Timeline Inconsistencies', 2)
p = doc.add_paragraph()
p.add_run('Timeline inconsistencies between pre-enrollment and post-course dates:')
doc.add_paragraph(f'• {90} records where pre-course year is after post-course year', style='List Bullet')
p = doc.add_paragraph()
p.add_run('Recommendation: ').bold = True
p.add_run('These records should be reviewed and potentially excluded from time-sensitive analyses.')

# Add visualizations
doc.add_heading('7. Visualizations', 1)

# Add match type distribution chart
doc.add_heading('7.1 Match Type Distribution', 2)
doc.add_paragraph('The following chart shows the distribution of match types in the Comprehensive Dataset:')
doc.add_picture(pie_chart_path, width=Inches(6))
p = doc.add_paragraph()
p.add_run(f'Email Matches: {sum(comprehensive_df["match_type"] == "email"):,} ({sum(comprehensive_df["match_type"] == "email")/len(comprehensive_df)*100:.1f}%)')
p.add_run().add_break()
p.add_run(f'Name+DOB Matches: {sum(comprehensive_df["match_type"] == "full_name_dob"):,} ({sum(comprehensive_df["match_type"] == "full_name_dob")/len(comprehensive_df)*100:.1f}%)')

# Add dataset size comparison chart
doc.add_heading('7.2 Dataset Size Comparison', 2)
doc.add_paragraph('The following chart compares the sizes of the original and matched datasets:')
doc.add_picture(dataset_chart_path, width=Inches(6))

# Recommendations
doc.add_heading('8. Recommendations for Analysis', 1)
p = doc.add_paragraph()
p.add_run('Based on the data quality assessment, the following recommendations are provided:')

doc.add_paragraph('• Use the Comprehensive Dataset for most analyses to maximize statistical power while maintaining reliability.', style='List Bullet')
doc.add_paragraph('• Use the Gold Standard Dataset for analyses requiring absolute certainty in matches or as validation for sensitive findings.', style='List Bullet')
doc.add_paragraph('• Consider excluding records with timeline inconsistencies from analyses that depend on temporal sequence.', style='List Bullet')
doc.add_paragraph('• Review and potentially exclude age outliers from analyses where age is a factor.', style='List Bullet')
doc.add_paragraph('• Document which dataset was used when reporting results.', style='List Bullet')
doc.add_paragraph('• Analyze key metrics using both datasets to ensure robustness of findings.', style='List Bullet')

# Save the document
doc.save(doc_path)
print(f"Documentation created and saved to: {doc_path}")

Documentation created and saved to: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Data_Matching_Methodology.docx


In [1]:
#Final Standardization of Pre enrollment and Post Course

import pandas as pd
import os
import re
from datetime import datetime
import numpy as np

def standardize_original_datasets(pre_course_path, post_course_path):
    """
    Standardize the original pre-enrollment and post-course datasets.
    
    Parameters:
    pre_course_path (str): Path to the pre-enrollment Excel file
    post_course_path (str): Path to the post-course Excel file
    """
    print("Standardizing original pre-enrollment and post-course datasets")
    
    # Step 1: Load the datasets
    print("\n1. Loading datasets")
    print(f"Loading pre-enrollment data from: {pre_course_path}")
    pre_df = pd.read_excel(pre_course_path)
    print(f"Loaded {len(pre_df)} pre-enrollment records")
    
    print(f"Loading post-course data from: {post_course_path}")
    post_df = pd.read_excel(post_course_path)
    print(f"Loaded {len(post_df)} post-course records")
    
    # Create output file paths
    output_dir = os.path.dirname(pre_course_path)
    pre_output_file = os.path.join(output_dir, "Standardized_Pre_Enrollment_Data.xlsx")
    post_output_file = os.path.join(output_dir, "Standardized_Post_Course_Data.xlsx")
    
    # Step 2: Define standardization functions
    print("\n2. Defining standardization functions")
    
    # Domain corrections dictionary
    domain_corrections = {
        # Gmail variations
        'gmai.com': 'gmail.com',
        'gmil.com': 'gmail.com',
        'gmal.com': 'gmail.com',
        'gamail.com': 'gmail.com',
        'gimail.com': 'gmail.com',
        'gmaill.com': 'gmail.com',
        'gmail.comm': 'gmail.com',
        'gmial.com': 'gmail.com',
        'gmail.cm': 'gmail.com',
        'gmail.org': 'gmail.com',
        'gemail.com': 'gmail.com',
        'gmail.com.com': 'gmail.com',
        'gmai.coml': 'gmail.com',
        'gmail.coom': 'gmail.com',
        'jmail.com': 'gmail.com',
        'gmeil.com': 'gmail.com',
        'gmall.com': 'gmail.com',
        'gail.com': 'gmail.com',
        'gmail.co': 'gmail.com',
        'gmaiil.com': 'gmail.com',
        'gmaim.com': 'gmail.com',
        'gmail.om': 'gmail.com',
        'gmail.ccom': 'gmail.com',
        'g.com': 'gmail.com',
        'gmail.vom': 'gmail.com',
        'gmali.com': 'gmail.com',
        'gmaio.com': 'gmail.com',
        'g-mail.com': 'gmail.com',
        'gamill.com': 'gmail.com',
        'gmaol.com': 'gmail.com',
        'gmsil.com': 'gmail.com',
        'gmaii.com': 'gmail.com',
        'gmail.oeg': 'gmail.com',
        'gmanil.com': 'gmail.com',
        'gmaili.com': 'gmail.com',
        'gmail.crom': 'gmail.com',
        'gmail.com11': 'gmail.com',
        'gmill.com': 'gmail.com',
        'gmail.com3': 'gmail.com',
        'gnmail.com': 'gmail.com',
        'gmaij.com': 'gmail.com',
        'gmail.coml.com': 'gmail.com',
        
        # Yahoo variations
        'yahoo.con': 'yahoo.com',
        'yaoo.com': 'yahoo.com',
        'hayoo.com': 'yahoo.com',
        'yhoo.com': 'yahoo.com',
        'yahoo.comm': 'yahoo.com',
        'myyahoo.com': 'yahoo.com',
        'yahoo.comb': 'yahoo.com',
        'yahool.com': 'yahoo.com',
        'yaho.cm': 'yahoo.com',
        'yahio.un': 'yahoo.com',
        
        # Hotmail variations
        'hmail.com': 'hotmail.com',
        'hotmai.com': 'hotmail.com',
        'hotamil.com': 'hotmail.com',
        'hotmail.con': 'hotmail.com',
        'hoo.com': 'hotmail.com',
        'hotmiail.com': 'hotmail.com',
        'htomail.com': 'hotmail.com',
        'hiotmail.com': 'hotmail.com',
        'hitmail.com': 'hotmail.com',
        
        # iCloud variations
        'icould.com': 'icloud.com',
        'iclod.com': 'icloud.com',
        'cloud.com': 'icloud.com',
        'icoud.com': 'icloud.com',
        'ichoud.com': 'icloud.com',
        'iclou.com': 'icloud.com',
        
        # AOL variations
        'ao.com': 'aol.com',
        'alo.com': 'aol.com',
        'aol.co': 'aol.com',
        'aol.om': 'aol.com',
        
        # Boston Public Schools variations
        'bostonk12.com': 'bostonk12.org',
        'boston12.org': 'bostonk12.org',
        'bostobpublicschools.org': 'bostonpublicschools.org',
        'bostonnpublicschools.org': 'bostonpublicschools.org',
        'bostonnk12.org': 'bostonk12.org',
        'bostonpulicschools.org': 'bostonpublicschools.org',
        'bostonpublicschool.org.org': 'bostonpublicschools.org',
        
        # Other common corrections
        'live.con': 'live.com',
        'live.cm': 'live.com',
        'outlook.pt': 'outlook.com',
        'verizon.com': 'verizon.net',
        'comcast.com': 'comcast.net',
    }
    
    # Email standardization
    def standardize_email(email):
        if pd.isna(email):
            return None
        
        # Convert to string, lowercase, and strip spaces
        email_str = str(email).lower().strip()
        
        # Remove any spaces
        email_str = email_str.replace(' ', '')
        
        # Basic validation
        if '@' not in email_str or '.' not in email_str:
            return None
        
        # Extract username and domain
        username, domain = email_str.split('@', 1)
        
        # Apply domain correction
        corrected_domain = domain_corrections.get(domain, domain)
        
        # Rebuild corrected email
        return f"{username}@{corrected_domain}"
    
    # Phone number standardization
    def standardize_phone(phone):
        if pd.isna(phone):
            return None
        
        # Convert to string and remove non-digit characters
        phone_str = re.sub(r'\D', '', str(phone))
        
        # Ensure it's a valid US phone (10 digits)
        if len(phone_str) == 10:
            # Format as XXX-XXX-XXXX
            return f"{phone_str[:3]}-{phone_str[3:6]}-{phone_str[6:]}"
        elif len(phone_str) == 11 and phone_str[0] == '1':
            # Remove country code and format
            return f"{phone_str[1:4]}-{phone_str[4:7]}-{phone_str[7:]}"
        else:
            # If format is unclear, return cleaned digits
            return phone_str
    
    # DOB standardization for age calculation
    def standardize_dob(dob):
        if pd.isna(dob):
            return None
        
        try:
            # If it's already a datetime
            if isinstance(dob, (pd.Timestamp, datetime)):
                return dob
            
            # Convert to string
            dob_str = str(dob).strip()
            
            # Try parsing with pandas
            try:
                dob_date = pd.to_datetime(dob_str)
                
                # Adjust years for two-digit years (19xx vs 20xx)
                if dob_date.year > datetime.now().year and dob_date.year < 2100:
                    dob_date = dob_date.replace(year=dob_date.year - 100)
                
                # Sanity check for very old birth years
                if dob_date.year < 1900:
                    if dob_date.year < 1800:
                        return None
                
                return dob_date
            except:
                pass
            
            # Try specialized formats
            # Handle MM-DD-YYYY format common in your data
            if '-' in dob_str:
                parts = dob_str.split('-')
                if len(parts) == 3:
                    try:
                        month, day, year = parts
                        month = int(month)
                        day = int(day)
                        year = int(year)
                        
                        # Adjust if it's a two-digit year
                        if year < 100:
                            year = 1900 + year if year >= 50 else 2000 + year
                            
                        return pd.Timestamp(year=year, month=month, day=day)
                    except:
                        pass
        except:
            pass
        
        # If we can't parse it properly
        return None
    
    # Function to calculate age at course
    def calculate_age_at_course(dob, course_year):
        if pd.isna(dob) or pd.isna(course_year):
            return None
        
        try:
            # Ensure dob is a datetime
            if isinstance(dob, str):
                dob = standardize_dob(dob)
            
            if dob is None:
                return None
                
            # Calculate age as of the course year (December 31st)
            course_date = pd.Timestamp(year=int(course_year), month=12, day=31)
            age = course_date.year - dob.year
            
            # Adjust if birthday hasn't occurred yet that year
            if course_date.month < dob.month or (course_date.month == dob.month and course_date.day < dob.day):
                age -= 1
                
            # Sanity check for reasonable age
            if 0 <= age <= 120:
                return age
                
            return None
        except:
            return None
    
    # ZIP code standardization
    def standardize_zip(zip_code):
        if pd.isna(zip_code):
            return None
            
        # Convert to string and remove non-digits
        zip_str = re.sub(r'\D', '', str(zip_code))
        
        # Handle 5-digit ZIP codes
        if len(zip_str) >= 5:
            return zip_str[:5]  # Keep only first 5 digits
        
        # If less than 5 digits, pad with leading zeros
        elif len(zip_str) > 0:
            return zip_str.zfill(5)
            
        return None
    
    # Step 3: Apply standardizations to pre-enrollment dataset
    print("\n3. Standardizing pre-enrollment dataset")
    
    # Email standardization
    if 'email_address' in pre_df.columns:
        pre_df['email_address_original'] = pre_df['email_address'].copy()
        pre_df['email_address'] = pre_df['email_address'].apply(standardize_email)
        pre_email_changes = sum((pre_df['email_address'] != pre_df['email_address_original']) & 
                               pre_df['email_address'].notna())
        print(f"Standardized {pre_email_changes} pre-enrollment emails")
    
    # Phone number standardization
    if 'phone_number' in pre_df.columns:
        pre_df['phone_number_original'] = pre_df['phone_number'].copy()
        pre_df['phone_number'] = pre_df['phone_number'].apply(standardize_phone)
        pre_phone_changes = sum((pre_df['phone_number'] != pre_df['phone_number_original']) & 
                               pre_df['phone_number'].notna())
        print(f"Standardized {pre_phone_changes} pre-enrollment phone numbers")
    
    # ZIP code standardization (added as a bonus)
    if 'zip_code_fixed' in pre_df.columns:
        pre_df['zip_code_original'] = pre_df['zip_code_fixed'].copy()
        pre_df['zip_code_fixed'] = pre_df['zip_code_fixed'].apply(standardize_zip)
        pre_zip_changes = sum((pre_df['zip_code_fixed'] != pre_df['zip_code_original']) & 
                             pre_df['zip_code_fixed'].notna())
        print(f"Standardized {pre_zip_changes} pre-enrollment ZIP codes")
    
    # Age at course calculation
    if 'date_of_birth_standardized' in pre_df.columns and 'course_year' in pre_df.columns:
        pre_df['date_of_birth_datetime'] = pre_df['date_of_birth_standardized'].apply(standardize_dob)
        pre_df['age_at_course'] = pre_df.apply(
            lambda row: calculate_age_at_course(row['date_of_birth_datetime'], row['course_year']), 
            axis=1
        )
        age_count = pre_df['age_at_course'].notna().sum()
        print(f"Calculated age at course for {age_count} pre-enrollment records")
        
        # Report on age distribution
        if age_count > 0:
            age_stats = pre_df['age_at_course'].describe()
            print(f"  Age range: {age_stats['min']} to {age_stats['max']} years")
            print(f"  Mean age: {age_stats['mean']:.1f} years")
            
            # Flag potential age outliers
            young_outliers = sum(pre_df['age_at_course'] < 5)
            old_outliers = sum(pre_df['age_at_course'] > 90)
            if young_outliers > 0 or old_outliers > 0:
                print(f"  Found {young_outliers} records with age < 5 years")
                print(f"  Found {old_outliers} records with age > 90 years")
    
    # Step 4: Apply standardizations to post-course dataset
    print("\n4. Standardizing post-course dataset")
    
    # Email standardization
    if 'email_address' in post_df.columns:
        post_df['email_address_original'] = post_df['email_address'].copy()
        post_df['email_address'] = post_df['email_address'].apply(standardize_email)
        post_email_changes = sum((post_df['email_address'] != post_df['email_address_original']) & 
                                post_df['email_address'].notna())
        print(f"Standardized {post_email_changes} post-course emails")
    
    # Optional: Additional post-course standardizations
    
    # Step 5: Save standardized datasets
    print(f"\n5. Saving standardized datasets")
    
    # Save pre-enrollment dataset
    pre_df.to_excel(pre_output_file, index=False)
    print(f"Saved standardized pre-enrollment dataset to: {pre_output_file}")
    
    # Save post-course dataset
    post_df.to_excel(post_output_file, index=False)
    print(f"Saved standardized post-course dataset to: {post_output_file}")
    
    print("\nStandardization complete!")
    
    return {
        'pre_df': pre_df,
        'post_df': post_df,
        'pre_output_file': pre_output_file,
        'post_output_file': post_output_file
    }

# Run the standardization
pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
results = standardize_original_datasets(pre_course_path, post_course_path)

Standardizing original pre-enrollment and post-course datasets

1. Loading datasets
Loading pre-enrollment data from: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx
Loaded 29700 pre-enrollment records
Loading post-course data from: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx
Loaded 7806 post-course records

2. Defining standardization functions

3. Standardizing pre-enrollment dataset
Standardized 281 pre-enrollment emails
Standardized 25975 pre-enrollment phone numbers
Standardized 29610 pre-enrollment ZIP codes
Calculated age at course for 19237 pre-enrollment records
  Age range: 0.0 to 95.0 years
  Mean age: 43.0 years
  Found 74 records with age < 5 years
  Found 26 records with age > 90 years

4. Standardizing post-course dataset
Standardized 51 post-course emails

5. Saving standardized datasets
Saved standardized pre-enrollment dataset to

In [7]:
import pandas as pd
import matplotlib.pyplot as plt

def analyze_course_years(matched_dataset_path):
    """
    Analyze the distribution of course years in the matched dataset.
    
    Parameters:
    matched_dataset_path (str): Path to the matched dataset Excel file
    """
    print(f"Analyzing course years in: {matched_dataset_path}")
    
    # Load the matched dataset
    df = pd.read_excel(matched_dataset_path)
    print(f"Loaded {len(df)} records")
    
    # Identify the course year column
    # Try different possible column names
    course_year_columns = ['post_course_year', 'course_year', 'pre_course_year']
    
    course_year_col = None
    for col in course_year_columns:
        if col in df.columns:
            course_year_col = col
            break
    
    if not course_year_col:
        print("Error: Could not find course year column in the dataset")
        print(f"Available columns: {df.columns.tolist()}")
        return
    
    print(f"Using column '{course_year_col}' for course year analysis")
    
    # Count records by course year
    year_counts = df[course_year_col].value_counts().sort_index()
    total_records = len(df)
    
    # Calculate percentages
    year_percentages = (year_counts / total_records * 100).round(2)
    
    # Create a DataFrame for the results
    results_df = pd.DataFrame({
        'Course Year': year_counts.index,
        'Count': year_counts.values,
        'Percentage': year_percentages.values
    })
    
    # Sort by year
    results_df = results_df.sort_values('Course Year')
    
    # Display the results
    print("\nCourse Year Distribution:")
    print("-" * 50)
    print(f"{'Year':<10} {'Count':<10} {'Percentage':<10}")
    print("-" * 50)
    
    for _, row in results_df.iterrows():
        print(f"{int(row['Course Year']):<10} {row['Count']:<10} {row['Percentage']}%")
    
    print("-" * 50)
    print(f"Total: {total_records} records")
    
    # Create a visualization
    plt.figure(figsize=(12, 6))
    
    # Bar chart
    bars = plt.bar(results_df['Course Year'].astype(str), results_df['Count'])
    
    # Add count and percentage labels
    for i, bar in enumerate(bars):
        height = bar.get_height()
        percentage = results_df['Percentage'].iloc[i]
        plt.text(bar.get_x() + bar.get_width()/2., height + 5,
                f'{int(height)}\n({percentage}%)', 
                ha='center', va='bottom')
    
    plt.title('Distribution of Records by Course Year', fontsize=15)
    plt.xlabel('Course Year', fontsize=12)
    plt.ylabel('Number of Records', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(rotation=45)
    
    # Save the chart
    chart_path = matched_dataset_path.replace('.xlsx', '_course_year_distribution.png')
    plt.tight_layout()
    plt.savefig(chart_path, dpi=300)
    plt.close()
    
    print(f"\nVisualization saved to: {chart_path}")
    
    # Save the results to Excel
    excel_path = matched_dataset_path.replace('.xlsx', '_course_year_analysis.xlsx')
    results_df.to_excel(excel_path, index=False)
    print(f"Analysis saved to: {excel_path}")
    
    return results_df

# Run the analysis
matched_dataset_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/Matched Data/Comprehensive_ES:PCS_matched.xlsx'
course_year_analysis = analyze_course_years(matched_dataset_path)

Analyzing course years in: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/Matched Data/Comprehensive_ES:PCS_matched.xlsx
Loaded 5967 records
Using column 'post_course_year' for course year analysis

Course Year Distribution:
--------------------------------------------------
Year       Count      Percentage
--------------------------------------------------
2003       1.0        0.02%
2016       5.0        0.08%
2020       1.0        0.02%
2022       951.0      15.94%
2023       1968.0     32.98%
2024       2394.0     40.12%
2025       192.0      3.22%
--------------------------------------------------
Total: 5967 records

Visualization saved to: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/Matched Data/Comprehensive_ES:PCS_matched_course_year_distribution.png
Analysis saved to: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/Matched Data/Comprehensive_ES:PCS_matched_course_year_analysis

In [9]:
#course year distribution analysis

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

def analyze_course_year_distributions(pre_course_path, post_course_path, matched_dataset_path):
    """
    Analyze and compare the distribution of course years across pre-enrollment,
    post-course, and matched datasets.
    
    Parameters:
    pre_course_path (str): Path to pre-enrollment dataset
    post_course_path (str): Path to post-course dataset
    matched_dataset_path (str): Path to matched dataset
    """
    print("Analyzing course year distributions across all datasets")
    
    # Load the datasets
    print("\nLoading datasets...")
    pre_df = pd.read_excel(pre_course_path)
    post_df = pd.read_excel(post_course_path)
    matched_df = pd.read_excel(matched_dataset_path)
    
    print(f"Pre-enrollment dataset: {len(pre_df)} records")
    print(f"Post-course dataset: {len(post_df)} records")
    print(f"Matched dataset: {len(matched_df)} records")
    
    # Identify course year columns in each dataset
    # These might have different names in each dataset
    pre_year_cols = [col for col in pre_df.columns if 'course_year' in col.lower() or 'year' in col.lower()]
    post_year_cols = [col for col in post_df.columns if 'course_year' in col.lower() or 'year' in col.lower()]
    matched_year_cols = [col for col in matched_df.columns if 'course_year' in col.lower() or 'year' in col.lower()]
    
    # Select the appropriate column for each dataset
    pre_year_col = None
    if 'pre_course_year' in pre_df.columns:
        pre_year_col = 'pre_course_year'
    elif 'course_year' in pre_df.columns:
        pre_year_col = 'course_year'
    elif len(pre_year_cols) > 0:
        pre_year_col = pre_year_cols[0]
        
    post_year_col = None
    if 'post_course_year' in post_df.columns:
        post_year_col = 'post_course_year'
    elif 'course_year' in post_df.columns:
        post_year_col = 'course_year'
    elif len(post_year_cols) > 0:
        post_year_col = post_year_cols[0]
    
    matched_year_col = None
    if 'post_course_year' in matched_df.columns:
        matched_year_col = 'post_course_year'
    elif 'pre_course_year' in matched_df.columns:
        matched_year_col = 'pre_course_year'
    elif 'course_year' in matched_df.columns:
        matched_year_col = 'course_year'
    elif len(matched_year_cols) > 0:
        matched_year_col = matched_year_cols[0]
    
    # Check if we found suitable columns
    print("\nSelected year columns for analysis:")
    print(f"Pre-enrollment: {pre_year_col}")
    print(f"Post-course: {post_year_col}")
    print(f"Matched dataset: {matched_year_col}")
    
    if not all([pre_year_col, post_year_col, matched_year_col]):
        print("\nWarning: Could not find course year columns in all datasets")
        if not pre_year_col:
            print(f"Pre-enrollment columns: {pre_df.columns.tolist()}")
        if not post_year_col:
            print(f"Post-course columns: {post_df.columns.tolist()}")
        if not matched_year_col:
            print(f"Matched columns: {matched_df.columns.tolist()}")
        return
    
    # Function to calculate year distribution for a dataset
    def get_year_distribution(df, year_col):
        # Filter out null values
        valid_years = df[df[year_col].notna()]
        
        # Count by year
        year_counts = valid_years[year_col].value_counts().sort_index()
        
        # Calculate percentages
        total = len(valid_years)
        year_pcts = (year_counts / total * 100).round(2)
        
        return pd.DataFrame({
            'Year': year_counts.index,
            'Count': year_counts.values,
            'Percentage': year_pcts.values
        })
    
    # Get distributions for each dataset
    pre_dist = get_year_distribution(pre_df, pre_year_col)
    post_dist = get_year_distribution(post_df, post_year_col)
    matched_dist = get_year_distribution(matched_df, matched_year_col)
    
    # Print the distributions
    print("\nPre-enrollment Course Year Distribution:")
    print(pre_dist.to_string(index=False))
    
    print("\nPost-course Year Distribution:")
    print(post_dist.to_string(index=False))
    
    print("\nMatched Dataset Year Distribution:")
    print(matched_dist.to_string(index=False))
    
    # Create a visualization directory
    output_dir = os.path.dirname(matched_dataset_path)
    viz_dir = os.path.join(output_dir, "Course_Year_Analysis")
    os.makedirs(viz_dir, exist_ok=True)
    
    # Get all unique years across all datasets
    all_years = sorted(set(
        list(pre_dist['Year']) + 
        list(post_dist['Year']) + 
        list(matched_dist['Year'])
    ))
    
    # Function to create consistent dataframe with all years
    def standardize_dist(dist_df):
        std_df = pd.DataFrame({'Year': all_years})
        merged = std_df.merge(dist_df, on='Year', how='left').fillna(0)
        return merged
    
    pre_dist_std = standardize_dist(pre_dist)
    post_dist_std = standardize_dist(post_dist)
    matched_dist_std = standardize_dist(matched_dist)
    
    # Create side-by-side bar chart
    plt.figure(figsize=(14, 8))
    
    x = np.arange(len(all_years))
    width = 0.25
    
    pre_bars = plt.bar(x - width, pre_dist_std['Percentage'], width, label='Pre-enrollment')
    post_bars = plt.bar(x, post_dist_std['Percentage'], width, label='Post-course')
    matched_bars = plt.bar(x + width, matched_dist_std['Percentage'], width, label='Matched')
    
    plt.xlabel('Course Year', fontsize=12)
    plt.ylabel('Percentage of Records', fontsize=12)
    plt.title('Course Year Distribution Comparison', fontsize=15)
    plt.xticks(x, [str(int(year)) for year in all_years])
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Save the comparison chart
    comparison_chart_path = os.path.join(viz_dir, "course_year_comparison.png")
    plt.tight_layout()
    plt.savefig(comparison_chart_path, dpi=300)
    plt.close()
    
    # Create individual charts for each dataset
    def create_year_chart(dist_df, title, filename):
        plt.figure(figsize=(10, 6))
        bars = plt.bar(dist_df['Year'].astype(str), dist_df['Percentage'])
        
        # Add percentage labels
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{height}%', ha='center', va='bottom')
        
        plt.title(title, fontsize=15)
        plt.xlabel('Course Year', fontsize=12)
        plt.ylabel('Percentage of Records', fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(viz_dir, filename), dpi=300)
        plt.close()
    
    create_year_chart(pre_dist, 'Pre-enrollment Course Year Distribution', 'pre_course_years.png')
    create_year_chart(post_dist, 'Post-course Year Distribution', 'post_course_years.png')
    create_year_chart(matched_dist, 'Matched Dataset Year Distribution', 'matched_course_years.png')
    
    # Create a complete analysis Excel file
    analysis_path = os.path.join(viz_dir, "course_year_analysis.xlsx")
    
    with pd.ExcelWriter(analysis_path, engine='openpyxl') as writer:
        # Create comparison sheet
        comparison_df = pd.DataFrame({'Year': all_years})
        
        # Add pre-enrollment percentages
        pre_pcts = pre_dist_std[['Year', 'Percentage']].rename(columns={'Percentage': 'Pre-enrollment %'})
        comparison_df = comparison_df.merge(pre_pcts, on='Year', how='left')
        
        # Add post-course percentages
        post_pcts = post_dist_std[['Year', 'Percentage']].rename(columns={'Percentage': 'Post-course %'})
        comparison_df = comparison_df.merge(post_pcts, on='Year', how='left')
        
        # Add matched percentages
        matched_pcts = matched_dist_std[['Year', 'Percentage']].rename(columns={'Percentage': 'Matched %'})
        comparison_df = comparison_df.merge(matched_pcts, on='Year', how='left')
        
        # Add counts
        pre_counts = pre_dist_std[['Year', 'Count']].rename(columns={'Count': 'Pre-enrollment Count'})
        comparison_df = comparison_df.merge(pre_counts, on='Year', how='left')
        
        post_counts = post_dist_std[['Year', 'Count']].rename(columns={'Count': 'Post-course Count'})
        comparison_df = comparison_df.merge(post_counts, on='Year', how='left')
        
        matched_counts = matched_dist_std[['Year', 'Count']].rename(columns={'Count': 'Matched Count'})
        comparison_df = comparison_df.merge(matched_counts, on='Year', how='left')
        
        # Calculate match rates
        comparison_df['Match Rate (% of Pre)'] = (comparison_df['Matched Count'] / comparison_df['Pre-enrollment Count'] * 100).round(2)
        comparison_df['Match Rate (% of Post)'] = (comparison_df['Matched Count'] / comparison_df['Post-course Count'] * 100).round(2)
        
        # Save each sheet
        comparison_df.to_excel(writer, sheet_name='Comparison', index=False)
        pre_dist.to_excel(writer, sheet_name='Pre-enrollment', index=False)
        post_dist.to_excel(writer, sheet_name='Post-course', index=False)
        matched_dist.to_excel(writer, sheet_name='Matched', index=False)
    
    print(f"\nAnalysis saved to: {analysis_path}")
    print(f"Visualizations saved to: {viz_dir}")
    
    return {
        'pre_distribution': pre_dist,
        'post_distribution': post_dist,
        'matched_distribution': matched_dist,
        'comparison': comparison_df
    }

# Run the analysis
pre_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
post_course_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/PCS/Cleaned_Merged_PCS_2cleaned.xlsx'
matched_dataset_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/Matched Data/Comprehensive_ES_PCS_matched.xlsx'

results = analyze_course_year_distributions(pre_course_path, post_course_path, matched_dataset_path)

Analyzing course year distributions across all datasets

Loading datasets...
Pre-enrollment dataset: 29700 records
Post-course dataset: 7806 records
Matched dataset: 5967 records

Selected year columns for analysis:
Pre-enrollment: course_year
Post-course: course_year
Matched dataset: post_course_year

Pre-enrollment Course Year Distribution:
  Year  Count  Percentage
2003.0      8        0.04
2016.0     14        0.07
2020.0   1340        6.92
2021.0   3460       17.87
2022.0   3419       17.66
2023.0   4712       24.34
2024.0   5861       30.27
2025.0    549        2.84

Post-course Year Distribution:
  Year  Count  Percentage
2003.0      1        0.01
2016.0      4        0.06
2020.0      1        0.01
2022.0   1022       14.76
2023.0   2585       37.34
2024.0   3027       43.73
2025.0    282        4.07

Matched Dataset Year Distribution:
  Year  Count  Percentage
2003.0      1        0.02
2016.0      5        0.09
2020.0      1        0.02
2022.0    951       17.25
2023.0   1968  

In [11]:
#investigating missing records for pre and post covid

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import re

def audit_year_distributions(raw_file_path, cleaned_file_path):
    """
    Audit year distributions in pre-enrollment datasets to identify potential 
    missing data issues, comparing course_year and submitted_date columns.
    
    Parameters:
    raw_file_path (str): Path to the raw pre-enrollment dataset with duplicates
    cleaned_file_path (str): Path to the cleaned pre-enrollment dataset
    """
    print("Auditing year distributions in pre-enrollment datasets")
    
    # Create output directory for results
    output_dir = os.path.dirname(raw_file_path)
    audit_dir = os.path.join(output_dir, "Year_Distribution_Audit")
    os.makedirs(audit_dir, exist_ok=True)
    
    # Step 1: Load the datasets
    print("\n1. Loading datasets...")
    try:
        raw_df = pd.read_excel(raw_file_path)
        print(f"Loaded raw dataset: {len(raw_df)} records")
    except Exception as e:
        print(f"Error loading raw dataset: {e}")
        return
        
    try:
        cleaned_df = pd.read_excel(cleaned_file_path)
        print(f"Loaded cleaned dataset: {len(cleaned_df)} records")
    except Exception as e:
        print(f"Error loading cleaned dataset: {e}")
        return
    
    # Step 2: Identify relevant columns
    print("\n2. Identifying relevant columns...")
    
    # Check for course_year columns
    raw_year_cols = [col for col in raw_df.columns if 'course_year' in col.lower() or 'year' in col.lower()]
    cleaned_year_cols = [col for col in cleaned_df.columns if 'course_year' in col.lower() or 'year' in col.lower()]
    
    raw_year_col = None
    if 'course_year' in raw_df.columns:
        raw_year_col = 'course_year'
    elif 'pre_course_year' in raw_df.columns:
        raw_year_col = 'pre_course_year'
    elif len(raw_year_cols) > 0:
        raw_year_col = raw_year_cols[0]
    
    cleaned_year_col = None
    if 'course_year' in cleaned_df.columns:
        cleaned_year_col = 'course_year'
    elif 'pre_course_year' in cleaned_df.columns:
        cleaned_year_col = 'pre_course_year'
    elif len(cleaned_year_cols) > 0:
        cleaned_year_col = cleaned_year_cols[0]
    
    # Check for submitted_date columns
    raw_date_cols = [col for col in raw_df.columns if 'submitted' in col.lower() or 'date' in col.lower()]
    cleaned_date_cols = [col for col in cleaned_df.columns if 'submitted' in col.lower() or 'date' in col.lower()]
    
    raw_date_col = None
    if 'pre_submitted_date' in raw_df.columns:
        raw_date_col = 'pre_submitted_date'
    elif 'submitted_date' in raw_df.columns:
        raw_date_col = 'submitted_date'
    elif len(raw_date_cols) > 0:
        # Try to find the most likely submission date column
        for col in raw_date_cols:
            if 'submit' in col.lower():
                raw_date_col = col
                break
        if not raw_date_col and raw_date_cols:
            raw_date_col = raw_date_cols[0]
    
    cleaned_date_col = None
    if 'pre_submitted_date' in cleaned_df.columns:
        cleaned_date_col = 'pre_submitted_date'
    elif 'submitted_date' in cleaned_df.columns:
        cleaned_date_col = 'submitted_date'
    elif len(cleaned_date_cols) > 0:
        # Try to find the most likely submission date column
        for col in cleaned_date_cols:
            if 'submit' in col.lower():
                cleaned_date_col = col
                break
        if not cleaned_date_col and cleaned_date_cols:
            cleaned_date_col = cleaned_date_cols[0]
    
    print(f"Raw dataset - Course year column: {raw_year_col}")
    print(f"Raw dataset - Submitted date column: {raw_date_col}")
    print(f"Cleaned dataset - Course year column: {cleaned_year_col}")
    print(f"Cleaned dataset - Submitted date column: {cleaned_date_col}")
    
    if not all([raw_year_col, raw_date_col, cleaned_year_col, cleaned_date_col]):
        print("\nWarning: Could not identify all necessary columns")
        # Print the first few column names if we couldn't find what we need
        if not raw_year_col or not raw_date_col:
            print(f"Raw dataset columns: {raw_df.columns.tolist()[:20]}...")
        if not cleaned_year_col or not cleaned_date_col:
            print(f"Cleaned dataset columns: {cleaned_df.columns.tolist()[:20]}...")
        return
    
    # Step 3: Define function to extract year from various date formats
    def extract_year_from_date(date_val):
        if pd.isna(date_val):
            return None
        
        # If already a datetime
        if isinstance(date_val, (pd.Timestamp, datetime)):
            return date_val.year
        
        # Convert to string
        date_str = str(date_val).strip()
        
        # Try direct year extraction with regex
        # Look for 4-digit year
        year_match = re.search(r'20\d{2}', date_str)
        if year_match:
            return int(year_match.group(0))
            
        # Look for 2-digit year and assume 20YY
        yy_match = re.search(r'\b\d{1,2}/\d{1,2}/(\d{2})\b', date_str)
        if yy_match:
            year = int(yy_match.group(1))
            return 2000 + year
        
        # Try standard date parsing
        try:
            date_obj = pd.to_datetime(date_str)
            return date_obj.year
        except:
            pass
        
        # Try various common formats
        formats = ['%m/%d/%Y', '%Y-%m-%d', '%m-%d-%Y', '%d/%m/%Y', '%m/%d/%y']
        for fmt in formats:
            try:
                date_obj = datetime.strptime(date_str, fmt)
                return date_obj.year
            except:
                continue
        
        # If nothing works, return None
        return None
    
    # Step 4: Extract years from course_year and submitted_date
    print("\n3. Extracting years from columns...")
    
    # For raw dataset
    raw_df['course_year_val'] = raw_df[raw_year_col].copy()
    # Make sure course_year is numeric
    raw_df['course_year_val'] = pd.to_numeric(raw_df['course_year_val'], errors='coerce')
    
    raw_df['submitted_year'] = raw_df[raw_date_col].apply(extract_year_from_date)
    
    # For cleaned dataset
    cleaned_df['course_year_val'] = cleaned_df[cleaned_year_col].copy()
    # Make sure course_year is numeric
    cleaned_df['course_year_val'] = pd.to_numeric(cleaned_df['course_year_val'], errors='coerce')
    
    cleaned_df['submitted_year'] = cleaned_df[cleaned_date_col].apply(extract_year_from_date)
    
    # Step 5: Analyze the distributions
    print("\n4. Analyzing year distributions...")
    
    # Function to get year distribution from a dataset
    def get_year_distribution(df, year_col, year_name):
        year_counts = df[year_col].value_counts().sort_index()
        total = year_counts.sum()
        year_pcts = (year_counts / total * 100).round(2)
        
        result_df = pd.DataFrame({
            'Year': year_counts.index,
            'Count': year_counts.values,
            'Percentage': year_pcts.values,
            'Source': year_name
        })
        return result_df
    
    # Get distributions
    raw_course_dist = get_year_distribution(
        raw_df.dropna(subset=['course_year_val']), 
        'course_year_val', 
        'Raw - Course Year'
    )
    
    raw_submitted_dist = get_year_distribution(
        raw_df.dropna(subset=['submitted_year']), 
        'submitted_year', 
        'Raw - Submitted Year'
    )
    
    cleaned_course_dist = get_year_distribution(
        cleaned_df.dropna(subset=['course_year_val']), 
        'course_year_val', 
        'Cleaned - Course Year'
    )
    
    cleaned_submitted_dist = get_year_distribution(
        cleaned_df.dropna(subset=['submitted_year']), 
        'submitted_year', 
        'Cleaned - Submitted Year'
    )
    
    # Print the distributions
    print("\nRaw Dataset - Course Year Distribution:")
    print(raw_course_dist[['Year', 'Count', 'Percentage']].to_string(index=False))
    
    print("\nRaw Dataset - Submitted Year Distribution:")
    print(raw_submitted_dist[['Year', 'Count', 'Percentage']].to_string(index=False))
    
    print("\nCleaned Dataset - Course Year Distribution:")
    print(cleaned_course_dist[['Year', 'Count', 'Percentage']].to_string(index=False))
    
    print("\nCleaned Dataset - Submitted Year Distribution:")
    print(cleaned_submitted_dist[['Year', 'Count', 'Percentage']].to_string(index=False))
    
    # Step 6: Cross-tabulate course_year vs submitted_year
    print("\n5. Cross-tabulating course year vs submitted year...")
    
    # For raw dataset
    raw_cross = pd.crosstab(
        raw_df['course_year_val'], 
        raw_df['submitted_year'],
        margins=True,
        normalize=False
    )
    
    # For cleaned dataset
    cleaned_cross = pd.crosstab(
        cleaned_df['course_year_val'], 
        cleaned_df['submitted_year'],
        margins=True,
        normalize=False
    )
    
    print("\nRaw Dataset - Cross-tabulation of Course Year vs Submitted Year:")
    print(raw_cross)
    
    print("\nCleaned Dataset - Cross-tabulation of Course Year vs Submitted Year:")
    print(cleaned_cross)
    
    # Step 7: Visualize the distributions
    print("\n6. Creating visualizations...")
    
    # Combine all distributions for comparison
    all_dist = pd.concat([
        raw_course_dist, 
        raw_submitted_dist, 
        cleaned_course_dist, 
        cleaned_submitted_dist
    ])
    
    # Get all unique years
    all_years = sorted(all_dist['Year'].unique())
    
    # Create standardized dataframes with all years
    distributions = {}
    for source in all_dist['Source'].unique():
        dist = all_dist[all_dist['Source'] == source]
        
        # Create a standardized dataframe with all years
        std_df = pd.DataFrame({'Year': all_years})
        merged = std_df.merge(dist[['Year', 'Count', 'Percentage']], on='Year', how='left').fillna(0)
        distributions[source] = merged
    
    # Create comparison chart
    plt.figure(figsize=(14, 8))
    
    # Line chart with markers
    for source, dist in distributions.items():
        plt.plot(dist['Year'], dist['Percentage'], marker='o', linewidth=2, label=source)
    
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Percentage of Records', fontsize=12)
    plt.title('Year Distribution Comparison', fontsize=15)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    plt.xticks(all_years)
    
    # Add vertical line around 2019-2021 to highlight the years with issues
    plt.axvspan(2019, 2021, alpha=0.2, color='red')
    plt.text(2020, 5, 'Years with potential issues', ha='center', fontsize=10)
    
    # Save the comparison chart
    comparison_chart_path = os.path.join(audit_dir, "year_distribution_comparison.png")
    plt.tight_layout()
    plt.savefig(comparison_chart_path, dpi=300)
    plt.close()
    
    # Create individual charts for each distribution
    def create_year_chart(dist_df, title, filename):
        plt.figure(figsize=(12, 6))
        bars = plt.bar(dist_df['Year'].astype(str), dist_df['Percentage'])
        
        # Add percentage labels
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{height:.1f}%', ha='center', va='bottom')
        
        plt.title(title, fontsize=15)
        plt.xlabel('Year', fontsize=12)
        plt.ylabel('Percentage of Records', fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(audit_dir, filename), dpi=300)
        plt.close()
    
    for source, dist in distributions.items():
        create_year_chart(dist, f'{source} Distribution', f"{source.lower().replace(' - ', '_').replace(' ', '_')}_distribution.png")
    
    # Create heatmap for cross-tabulations
    def create_cross_heatmap(cross_df, title, filename):
        plt.figure(figsize=(12, 8))
        
        # Remove the 'All' row and column for the visualization
        cross_viz = cross_df.iloc[:-1, :-1].copy()
        
        # Create the heatmap
        plt.imshow(cross_viz, cmap='YlOrRd')
        
        # Add text annotations
        for i in range(cross_viz.shape[0]):
            for j in range(cross_viz.shape[1]):
                plt.text(j, i, f'{cross_viz.iloc[i, j]:.0f}', 
                         ha='center', va='center', 
                         color='black' if cross_viz.iloc[i, j] < cross_viz.max().max()/2 else 'white')
        
        plt.colorbar(label='Count')
        plt.title(title, fontsize=15)
        plt.xlabel('Submitted Year', fontsize=12)
        plt.ylabel('Course Year', fontsize=12)
        
        # Set tick labels
        plt.xticks(range(len(cross_viz.columns)), cross_viz.columns)
        plt.yticks(range(len(cross_viz.index)), cross_viz.index)
        
        plt.tight_layout()
        plt.savefig(os.path.join(audit_dir, filename), dpi=300)
        plt.close()
    
    create_cross_heatmap(raw_cross, 'Raw Dataset: Course Year vs Submitted Year', 'raw_cross_heatmap.png')
    create_cross_heatmap(cleaned_cross, 'Cleaned Dataset: Course Year vs Submitted Year', 'cleaned_cross_heatmap.png')
    
    # Step 8: Save comprehensive analysis to Excel
    print("\n7. Saving analysis to Excel...")
    
    excel_path = os.path.join(audit_dir, "year_distribution_audit.xlsx")
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Summary sheet
        summary_data = []
        
        # Add row counts
        summary_data.append({
            'Metric': 'Raw Dataset - Total Records', 
            'Value': len(raw_df)
        })
        summary_data.append({
            'Metric': 'Raw Dataset - Records with Course Year', 
            'Value': raw_df['course_year_val'].notna().sum()
        })
        summary_data.append({
            'Metric': 'Raw Dataset - Records with Submitted Year', 
            'Value': raw_df['submitted_year'].notna().sum()
        })
        summary_data.append({
            'Metric': 'Cleaned Dataset - Total Records', 
            'Value': len(cleaned_df)
        })
        summary_data.append({
            'Metric': 'Cleaned Dataset - Records with Course Year', 
            'Value': cleaned_df['course_year_val'].notna().sum()
        })
        summary_data.append({
            'Metric': 'Cleaned Dataset - Records with Submitted Year', 
            'Value': cleaned_df['submitted_year'].notna().sum()
        })
        
        # Add record counts for 2019-2021
        for year in [2019, 2020, 2021]:
            raw_course_count = sum(raw_df['course_year_val'] == year)
            raw_submitted_count = sum(raw_df['submitted_year'] == year)
            cleaned_course_count = sum(cleaned_df['course_year_val'] == year)
            cleaned_submitted_count = sum(cleaned_df['submitted_year'] == year)
            
            summary_data.append({
                'Metric': f'Raw Dataset - {year} Course Year Records', 
                'Value': raw_course_count
            })
            summary_data.append({
                'Metric': f'Raw Dataset - {year} Submitted Year Records', 
                'Value': raw_submitted_count
            })
            summary_data.append({
                'Metric': f'Cleaned Dataset - {year} Course Year Records', 
                'Value': cleaned_course_count
            })
            summary_data.append({
                'Metric': f'Cleaned Dataset - {year} Submitted Year Records', 
                'Value': cleaned_submitted_count
            })
        
        pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
        
        # Year distributions
        all_dist.to_excel(writer, sheet_name='Year Distributions', index=False)
        
        # Cross-tabulations
        raw_cross.to_excel(writer, sheet_name='Raw Cross-tabulation')
        cleaned_cross.to_excel(writer, sheet_name='Cleaned Cross-tabulation')
        
        # Records for problematic years
        for year in [2019, 2020, 2021]:
            # Raw records for this year
            raw_year_records = raw_df[
                (raw_df['course_year_val'] == year) | 
                (raw_df['submitted_year'] == year)
            ]
            if len(raw_year_records) > 0:
                raw_year_records.to_excel(writer, sheet_name=f'Raw {year} Records', index=False)
            
            # Cleaned records for this year
            cleaned_year_records = cleaned_df[
                (cleaned_df['course_year_val'] == year) | 
                (cleaned_df['submitted_year'] == year)
            ]
            if len(cleaned_year_records) > 0:
                cleaned_year_records.to_excel(writer, sheet_name=f'Cleaned {year} Records', index=False)
    
    print(f"\nAudit completed. Results saved to: {audit_dir}")
    print(f"Analysis Excel file: {excel_path}")
    
    # Return results for further analysis if needed
    return {
        'raw_course_dist': raw_course_dist,
        'raw_submitted_dist': raw_submitted_dist,
        'cleaned_course_dist': cleaned_course_dist,
        'cleaned_submitted_dist': cleaned_submitted_dist,
        'raw_cross': raw_cross,
        'cleaned_cross': cleaned_cross
    }

# Run the audit
raw_file_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Merged_ES_Raw.xlsx'
cleaned_file_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/2nd_cleaned_data_excel.xlsx'

results = audit_year_distributions(raw_file_path, cleaned_file_path)

Auditing year distributions in pre-enrollment datasets

1. Loading datasets...
Loaded raw dataset: 34742 records
Loaded cleaned dataset: 29700 records

2. Identifying relevant columns...
Raw dataset - Course year column: have_you_received_a_tgh_device_chromebook_or_ipad_in_the_last_two_years
Raw dataset - Submitted date column: submitted_date
Cleaned dataset - Course year column: course_year
Cleaned dataset - Submitted date column: submitted_date

3. Extracting years from columns...

4. Analyzing year distributions...

Raw Dataset - Course Year Distribution:
Empty DataFrame
Columns: [Year, Count, Percentage]
Index: []

Raw Dataset - Submitted Year Distribution:
  Year  Count  Percentage
2016.0      3        0.01
2017.0   1047        3.03
2018.0   4820       13.96
2019.0   3186        9.22
2020.0   3825       11.08
2021.0   4428       12.82
2022.0   4065       11.77
2023.0   5271       15.26
2024.0   7278       21.07
2025.0    614        1.78

Cleaned Dataset - Course Year Distribution:

/var/folders/q6/v__5dqvn4fsgnld9cp_x0v1m0000gn/T/ipykernel_33164/230460191.py:329: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  plt.imshow(cross_viz, cmap='YlOrRd')
/var/folders/q6/v__5dqvn4fsgnld9cp_x0v1m0000gn/T/ipykernel_33164/230460191.py:329: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  plt.imshow(cross_viz, cmap='YlOrRd')



7. Saving analysis to Excel...

Audit completed. Results saved to: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Year_Distribution_Audit
Analysis Excel file: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Year_Distribution_Audit/year_distribution_audit.xlsx


In [13]:
#fixing course years

import pandas as pd
import os

def fill_missing_course_years(input_file_path, output_file_path=None):
    """
    Fill missing course years using submission date as a proxy.
    
    Parameters:
    input_file_path (str): Path to the dataset with missing course years
    output_file_path (str, optional): Path to save the corrected dataset
    """
    print(f"Processing dataset: {input_file_path}")
    
    # Load the dataset
    df = pd.read_excel(input_file_path)
    print(f"Loaded {len(df)} records")
    
    # Identify the course year and submitted date columns
    course_year_col = 'course_year' if 'course_year' in df.columns else None
    submitted_date_col = 'submitted_date' if 'submitted_date' in df.columns else None
    
    if not course_year_col or not submitted_date_col:
        print("Error: Could not find required columns")
        return df
    
    # Count missing course years
    missing_years = df[course_year_col].isna().sum()
    print(f"Found {missing_years} records with missing course years")
    
    # Create a copy of the dataframe
    df_updated = df.copy()
    
    # Function to extract year from date
    def extract_year(date_val):
        if pd.isna(date_val):
            return None
        
        try:
            # Convert to datetime and extract year
            date_obj = pd.to_datetime(date_val)
            return date_obj.year
        except:
            # If conversion fails, return None
            return None
    
    # Extract submission years
    df_updated['submission_year'] = df_updated[submitted_date_col].apply(extract_year)
    
    # Count records by submission year (before filling)
    print("\nDistribution of submission years (for records with missing course years):")
    missing_year_counts = df_updated[df_updated[course_year_col].isna()]['submission_year'].value_counts().sort_index()
    for year, count in missing_year_counts.items():
        print(f"  {year}: {count} records")
    
    # Fill missing course years with submission years
    before_count = df_updated[course_year_col].notna().sum()
    
    # Only fill where course_year is missing but submission_year exists
    mask = (df_updated[course_year_col].isna()) & (df_updated['submission_year'].notna())
    df_updated.loc[mask, course_year_col] = df_updated.loc[mask, 'submission_year']
    
    after_count = df_updated[course_year_col].notna().sum()
    filled_count = after_count - before_count
    
    print(f"\nFilled {filled_count} missing course years using submission years")
    
    # Distribution of course years after filling
    print("\nCourse year distribution after filling:")
    year_counts = df_updated[course_year_col].value_counts().sort_index()
    for year, count in year_counts.items():
        print(f"  {year}: {count} records")
    
    # Save the updated dataset if output path provided
    if output_file_path:
        # Drop the temporary column
        df_updated = df_updated.drop(columns=['submission_year'])
        
        df_updated.to_excel(output_file_path, index=False)
        print(f"\nSaved updated dataset to: {output_file_path}")
    
    return df_updated

# Set paths
input_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx'
output_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned_Enhanced_ES_Data_with_Fixed_Years.xlsx'

# Run the function
updated_df = fill_missing_course_years(input_path, output_path)

Processing dataset: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned & Enhanced ES Data .xlsx
Loaded 29700 records
Found 10337 records with missing course years

Distribution of submission years (for records with missing course years):
  2016.0: 1 records
  2017.0: 918 records
  2018.0: 4216 records
  2019.0: 2819 records
  2020.0: 1749 records
  2021.0: 100 records
  2022.0: 61 records
  2023.0: 91 records
  2024.0: 372 records
  2025.0: 10 records

Filled 10337 missing course years using submission years

Course year distribution after filling:
  2003.0: 8 records
  2016.0: 15 records
  2017.0: 918 records
  2018.0: 4216 records
  2019.0: 2819 records
  2020.0: 3089 records
  2021.0: 3560 records
  2022.0: 3480 records
  2023.0: 4803 records
  2024.0: 6233 records
  2025.0: 559 records

Saved updated dataset to: /Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/ES/Cleaned_Enhanced_ES_Data_with_Fixed_Years.xlsx


In [15]:
#updating matched data course years
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

def update_course_years_in_matched_datasets(comprehensive_path, gold_standard_path):
    """
    Update course years in matched datasets using pre_submitted_date for missing values.
    Analyze course year distribution before and after the update.
    
    Parameters:
    comprehensive_path (str): Path to the comprehensive matched dataset
    gold_standard_path (str): Path to the gold standard matched dataset
    """
    print("Updating course years in matched datasets using submission dates")
    
    # Create output directory for results
    output_dir = os.path.dirname(comprehensive_path)
    analysis_dir = os.path.join(output_dir, "Course_Year_Update_Analysis")
    os.makedirs(analysis_dir, exist_ok=True)
    
    # Step 1: Load the datasets
    print("\n1. Loading datasets...")
    comprehensive_df = pd.read_excel(comprehensive_path)
    gold_standard_df = pd.read_excel(gold_standard_path)
    
    print(f"Loaded comprehensive dataset: {len(comprehensive_df)} records")
    print(f"Loaded gold standard dataset: {len(gold_standard_df)} records")
    
    # Step 2: Identify the relevant columns
    print("\n2. Identifying relevant columns...")
    
    # Common column patterns
    course_year_patterns = ['course_year', 'pre_course_year']
    submitted_date_patterns = ['pre_submitted_date', 'submitted_date']
    
    # Find course year columns
    comprehensive_year_col = None
    for pattern in course_year_patterns:
        for col in comprehensive_df.columns:
            if pattern in col.lower():
                comprehensive_year_col = col
                break
        if comprehensive_year_col:
            break
            
    gold_standard_year_col = None
    for pattern in course_year_patterns:
        for col in gold_standard_df.columns:
            if pattern in col.lower():
                gold_standard_year_col = col
                break
        if gold_standard_year_col:
            break
    
    # Find submitted date columns
    comprehensive_date_col = None
    for pattern in submitted_date_patterns:
        for col in comprehensive_df.columns:
            if pattern in col.lower():
                comprehensive_date_col = col
                break
        if comprehensive_date_col:
            break
            
    gold_standard_date_col = None
    for pattern in submitted_date_patterns:
        for col in gold_standard_df.columns:
            if pattern in col.lower():
                gold_standard_date_col = col
                break
        if gold_standard_date_col:
            break
    
    print(f"Comprehensive dataset - Course year column: {comprehensive_year_col}")
    print(f"Comprehensive dataset - Submitted date column: {comprehensive_date_col}")
    print(f"Gold standard dataset - Course year column: {gold_standard_year_col}")
    print(f"Gold standard dataset - Submitted date column: {gold_standard_date_col}")
    
    if not all([comprehensive_year_col, comprehensive_date_col, gold_standard_year_col, gold_standard_date_col]):
        print("\nWarning: Could not find all necessary columns")
        return
    
    # Step 3: Analyze course year distribution before update
    print("\n3. Analyzing course year distribution before update...")
    
    def get_year_distribution(df, year_col, dataset_name):
        # Count by year
        year_counts = df[year_col].value_counts().sort_index()
        
        # Calculate percentages
        total = len(df)
        year_pcts = (year_counts / total * 100).round(2)
        
        result_df = pd.DataFrame({
            'Year': year_counts.index,
            'Count': year_counts.values,
            'Percentage': year_pcts.values
        })
        
        print(f"\n{dataset_name} - Course Year Distribution (Before Update):")
        print(result_df.to_string(index=False))
        
        return result_df
    
    comp_before_dist = get_year_distribution(comprehensive_df, comprehensive_year_col, "Comprehensive Dataset")
    gold_before_dist = get_year_distribution(gold_standard_df, gold_standard_year_col, "Gold Standard Dataset")
    
    # Step 4: Extract year from submitted date
    print("\n4. Extracting years from submitted dates...")
    
    def extract_year_from_date(date_val):
        if pd.isna(date_val):
            return None
        
        try:
            # Convert to datetime and extract year
            date_obj = pd.to_datetime(date_val)
            return date_obj.year
        except:
            # If conversion fails, return None
            return None
    
    # Process comprehensive dataset
    comprehensive_df['submission_year'] = comprehensive_df[comprehensive_date_col].apply(extract_year_from_date)
    
    # Process gold standard dataset
    gold_standard_df['submission_year'] = gold_standard_df[gold_standard_date_col].apply(extract_year_from_date)
    
    # Count records by submission year
    print("\nComprehensive Dataset - Submission Year Distribution:")
    comp_subm_counts = comprehensive_df['submission_year'].value_counts().sort_index()
    for year, count in comp_subm_counts.items():
        print(f"  {year}: {count} records")
    
    print("\nGold Standard Dataset - Submission Year Distribution:")
    gold_subm_counts = gold_standard_df['submission_year'].value_counts().sort_index()
    for year, count in gold_subm_counts.items():
        print(f"  {year}: {count} records")
    
    # Step 5: Fill missing course years with submission years
    print("\n5. Filling missing course years...")
    
    # For comprehensive dataset
    comp_before_count = comprehensive_df[comprehensive_year_col].notna().sum()
    
    # Only fill where course_year is missing and submission_year exists
    comp_mask = (comprehensive_df[comprehensive_year_col].isna()) & (comprehensive_df['submission_year'].notna())
    comprehensive_df.loc[comp_mask, comprehensive_year_col] = comprehensive_df.loc[comp_mask, 'submission_year']
    
    comp_after_count = comprehensive_df[comprehensive_year_col].notna().sum()
    comp_filled_count = comp_after_count - comp_before_count
    
    print(f"Comprehensive dataset: Filled {comp_filled_count} missing course years")
    
    # For gold standard dataset
    gold_before_count = gold_standard_df[gold_standard_year_col].notna().sum()
    
    # Only fill where course_year is missing and submission_year exists
    gold_mask = (gold_standard_df[gold_standard_year_col].isna()) & (gold_standard_df['submission_year'].notna())
    gold_standard_df.loc[gold_mask, gold_standard_year_col] = gold_standard_df.loc[gold_mask, 'submission_year']
    
    gold_after_count = gold_standard_df[gold_standard_year_col].notna().sum()
    gold_filled_count = gold_after_count - gold_before_count
    
    print(f"Gold standard dataset: Filled {gold_filled_count} missing course years")
    
    # Step 6: Analyze course year distribution after update
    print("\n6. Analyzing course year distribution after update...")
    
    comp_after_dist = get_year_distribution(comprehensive_df, comprehensive_year_col, "Comprehensive Dataset")
    gold_after_dist = get_year_distribution(gold_standard_df, gold_standard_year_col, "Gold Standard Dataset")
    
    # Step 7: Visualize before and after distributions
    print("\n7. Creating visualizations...")
    
    # Function to create comparative visualization
    def create_comparison_chart(before_dist, after_dist, title, filename):
        plt.figure(figsize=(14, 8))
        
        # Get all unique years
        all_years = sorted(set(list(before_dist['Year']) + list(after_dist['Year'])))
        
        # Create standardized dataframes with all years
        before_std = pd.DataFrame({'Year': all_years})
        before_std = before_std.merge(before_dist[['Year', 'Percentage']], on='Year', how='left').fillna(0)
        
        after_std = pd.DataFrame({'Year': all_years})
        after_std = after_std.merge(after_dist[['Year', 'Percentage']], on='Year', how='left').fillna(0)
        
        # Create bar chart
        x = np.arange(len(all_years))
        width = 0.35
        
        fig, ax = plt.subplots(figsize=(14, 8))
        before_bars = ax.bar(x - width/2, before_std['Percentage'], width, label='Before Update')
        after_bars = ax.bar(x + width/2, after_std['Percentage'], width, label='After Update')
        
        # Add labels and title
        ax.set_xlabel('Course Year')
        ax.set_ylabel('Percentage of Records')
        ax.set_title(title)
        ax.set_xticks(x)
        ax.set_xticklabels([str(int(year)) for year in all_years])
        ax.legend()
        
        # Add value labels
        def add_labels(bars):
            for bar in bars:
                height = bar.get_height()
                if height > 0:
                    ax.annotate(f'{height:.1f}%',
                                xy=(bar.get_x() + bar.get_width() / 2, height),
                                xytext=(0, 3),  # 3 points vertical offset
                                textcoords="offset points",
                                ha='center', va='bottom')
        
        add_labels(before_bars)
        add_labels(after_bars)
        
        # Highlight 2019-2021 period
        ax.axvspan(x[all_years.index(2019)] - width, x[all_years.index(2021)] + width, alpha=0.2, color='red')
        
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(analysis_dir, filename), dpi=300)
        plt.close()
    
    create_comparison_chart(
        comp_before_dist, 
        comp_after_dist, 
        'Comprehensive Dataset - Course Year Distribution Before and After Update', 
        'comprehensive_before_after.png'
    )
    
    create_comparison_chart(
        gold_before_dist, 
        gold_after_dist, 
        'Gold Standard Dataset - Course Year Distribution Before and After Update', 
        'gold_standard_before_after.png'
    )
    
    # Step 8: Save updated datasets
    print("\n8. Saving updated datasets...")
    
    # Remove temporary columns
    comprehensive_df = comprehensive_df.drop(columns=['submission_year'])
    gold_standard_df = gold_standard_df.drop(columns=['submission_year'])
    
    # Create output paths
    comprehensive_output = os.path.join(output_dir, "Comprehensive_ES_PCS_matched_updated.xlsx")
    gold_standard_output = os.path.join(output_dir, "Gold_Standard_Email_Only_Dataset_updated.xlsx")
    
    comprehensive_df.to_excel(comprehensive_output, index=False)
    gold_standard_df.to_excel(gold_standard_output, index=False)
    
    print(f"Saved updated comprehensive dataset to: {comprehensive_output}")
    print(f"Saved updated gold standard dataset to: {gold_standard_output}")
    
    # Step 9: Save analysis to Excel
    print("\n9. Saving analysis to Excel...")
    
    excel_path = os.path.join(analysis_dir, "course_year_update_analysis.xlsx")
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Summary sheet
        summary_data = []
        
        # Add dataset counts
        summary_data.append({
            'Metric': 'Comprehensive Dataset - Total Records', 
            'Value': len(comprehensive_df)
        })
        summary_data.append({
            'Metric': 'Comprehensive Dataset - Records with Course Year (Before)', 
            'Value': comp_before_count
        })
        summary_data.append({
            'Metric': 'Comprehensive Dataset - Records with Course Year (After)', 
            'Value': comp_after_count
        })
        summary_data.append({
            'Metric': 'Comprehensive Dataset - Course Years Filled', 
            'Value': comp_filled_count
        })
        summary_data.append({
            'Metric': 'Gold Standard Dataset - Total Records', 
            'Value': len(gold_standard_df)
        })
        summary_data.append({
            'Metric': 'Gold Standard Dataset - Records with Course Year (Before)', 
            'Value': gold_before_count
        })
        summary_data.append({
            'Metric': 'Gold Standard Dataset - Records with Course Year (After)', 
            'Value': gold_after_count
        })
        summary_data.append({
            'Metric': 'Gold Standard Dataset - Course Years Filled', 
            'Value': gold_filled_count
        })
        
        # Add counts for 2019-2021
        for year in [2019, 2020, 2021]:
            # Comprehensive dataset
            comp_before = len(comp_before_dist[comp_before_dist['Year'] == year]) > 0
            comp_before_count = comp_before_dist[comp_before_dist['Year'] == year]['Count'].sum() if comp_before else 0
            
            comp_after = len(comp_after_dist[comp_after_dist['Year'] == year]) > 0
            comp_after_count = comp_after_dist[comp_after_dist['Year'] == year]['Count'].sum() if comp_after else 0
            
            summary_data.append({
                'Metric': f'Comprehensive Dataset - {year} Records (Before)', 
                'Value': comp_before_count
            })
            summary_data.append({
                'Metric': f'Comprehensive Dataset - {year} Records (After)', 
                'Value': comp_after_count
            })
            summary_data.append({
                'Metric': f'Comprehensive Dataset - {year} Records Added', 
                'Value': comp_after_count - comp_before_count
            })
            
            # Gold standard dataset
            gold_before = len(gold_before_dist[gold_before_dist['Year'] == year]) > 0
            gold_before_count = gold_before_dist[gold_before_dist['Year'] == year]['Count'].sum() if gold_before else 0
            
            gold_after = len(gold_after_dist[gold_after_dist['Year'] == year]) > 0
            gold_after_count = gold_after_dist[gold_after_dist['Year'] == year]['Count'].sum() if gold_after else 0
            
            summary_data.append({
                'Metric': f'Gold Standard Dataset - {year} Records (Before)', 
                'Value': gold_before_count
            })
            summary_data.append({
                'Metric': f'Gold Standard Dataset - {year} Records (After)', 
                'Value': gold_after_count
            })
            summary_data.append({
                'Metric': f'Gold Standard Dataset - {year} Records Added', 
                'Value': gold_after_count - gold_before_count
            })
        
        pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
        
        # Distributions
        comp_before_dist.to_excel(writer, sheet_name='Comp Before', index=False)
        comp_after_dist.to_excel(writer, sheet_name='Comp After', index=False)
        gold_before_dist.to_excel(writer, sheet_name='Gold Before', index=False)
        gold_after_dist.to_excel(writer, sheet_name='Gold After', index=False)
    
    print(f"Saved analysis to: {excel_path}")
    print("\nUpdate completed successfully!")
    
    return {
        'comprehensive_before': comp_before_dist,
        'comprehensive_after': comp_after_dist,
        'gold_standard_before': gold_before_dist,
        'gold_standard_after': gold_after_dist,
        'comprehensive_df': comprehensive_df,
        'gold_standard_df': gold_standard_df
    }

# Run the update
comprehensive_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/Matched Data/Comprehensive_ES_PCS_matched.xlsx'
gold_standard_path = '/Users/sreeharsha/Documents/TGH Data Management Cleaning/FA Data/MERGED FA Data/Matched Data/Gold_Standard_Email_Only_Dataset.xlsx'

results = update_course_years_in_matched_datasets(comprehensive_path, gold_standard_path)

Updating course years in matched datasets using submission dates

1. Loading datasets...
Loaded comprehensive dataset: 5967 records
Loaded gold standard dataset: 3241 records

2. Identifying relevant columns...
Comprehensive dataset - Course year column: pre_course_year
Comprehensive dataset - Submitted date column: pre_submitted_date
Gold standard dataset - Course year column: pre_course_year
Gold standard dataset - Submitted date column: pre_submitted_date

3. Analyzing course year distribution before update...

Comprehensive Dataset - Course Year Distribution (Before Update):
  Year  Count  Percentage
2003.0      3        0.05
2016.0      5        0.08
2020.0     38        0.64
2021.0    131        2.20
2022.0    959       16.07
2023.0   1858       31.14
2024.0   2426       40.66
2025.0    184        3.08

Gold Standard Dataset - Course Year Distribution (Before Update):
  Year  Count  Percentage
2003.0      1        0.03
2016.0      2        0.06
2020.0     28        0.86
2021.0   

<Figure size 1400x800 with 0 Axes>

<Figure size 1400x800 with 0 Axes>